**Mount Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os, pathlib

BASE = "/content/drive/MyDrive/A2A_prediction_system"
PKG  = f"{BASE}/backend/a2a"
os.makedirs(PKG, exist_ok=True)


open(f"{BASE}/backend/__init__.py", "a").close()
open(f"{PKG}/__init__.py", "a").close()

print("Created:", PKG)
for p in pathlib.Path(f"{BASE}/backend").rglob("*"):
    print(" -", p)



**Install Libraries**

In [ ]:
!pip install langgraph yfinance numpy pandas pandas-datareader matplotlib

**Create index.html file in frontend folder**

In [ ]:
from pathlib import Path

ui_dir = Path("/content/drive/MyDrive/A2A_prediction_system/frontend")
ui_dir.mkdir(parents=True, exist_ok=True)

html = """<!doctype html>
<html><head><meta charset="utf-8"><title>A2A Demo UI</title></head>
<body>
  <h1>A2A Stock Analytics – Demo</h1>
  <p>If you can see this, the UI is mounted at <code>/ui/index.html</code>.</p>
</body></html>
"""
(ui_dir / "index.html").write_text(html, encoding="utf-8")
print("Wrote:", ui_dir / "index.html")



**Clean reload orchestrator and collector**

In [ ]:
import sys, importlib, os
sys.path.append("/content/drive/MyDrive/A2A_prediction_system/backend")


import a2a.data_collector_agent as dc
import a2a.orchestrator as orch
importlib.reload(dc)
importlib.reload(orch)
from a2a.orchestrator import run_orchestrator


**Run the Full pipeline via orchestrator**

In [ ]:
import sys, importlib, os, datetime
sys.path.append("/content/drive/MyDrive/A2A_prediction_system/backend")


import a2a.orchestrator as orch
import a2a.model_evaluation_agent as me
importlib.reload(me)
importlib.reload(orch)

from a2a.orchestrator import run_orchestrator

res = run_orchestrator({
    "tickers": ["AAPL"],

    "opt_args": {"allow_shorts": True, "snr_min": 0.7, "strong_pct": 60, "w_max": 0.40},
})
print("RUN_BASE:", res.get("run_base"))
print("report_path:", res.get("report_path"))
for k in ["reception_status","collector_status","fe_status","predict_status","eval_status",
          "xai_status","risk_status","opt_status","summary_status","report_status"]:
    print(k, "->", res.get(k))


**index.html file for the UI**

In [ ]:
%%writefile /content/drive/MyDrive/A2A_prediction_system/frontend/index.html

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8" />
  <meta name="viewport" content="width=device-width, initial-scale=1"/>
  <title>A2A Explainable Stock Analytics Platform</title>
  <style>
    body{font-family:system-ui,-apple-system,Segoe UI,Roboto,Inter,Arial,sans-serif;max-width:900px;margin:32px auto;padding:0 16px}
    .card{border:1px solid #e5e7eb;border-radius:14px;padding:16px;box-shadow:0 1px 2px rgba(0,0,0,.05)}
    .row{display:flex;gap:12px;flex-wrap:wrap}
    label{font-weight:600;margin-top:10px;display:block}
    input,select,textarea,button{font:inherit}
    input,select,textarea{width:100%;padding:10px;border:1px solid #d1d5db;border-radius:10px}
    button{padding:10px 14px;border-radius:10px;border:1px solid #1f2937;background:#111827;color:#fff;cursor:pointer}
    button[disabled]{opacity:.6;cursor:not-allowed}
    .muted{color:#6b7280}
    .ok{color:#059669}
    .err{color:#b91c1c}
    .link{word-break:break-all}
    .spinner{display:inline-block;width:16px;height:16px;border:2px solid #d1d5db;border-top-color:#111827;border-radius:50%;animation:spin .7s linear infinite;vertical-align:middle;margin-right:6px}
    @keyframes spin{to{transform:rotate(360deg)}}
  </style>
</head>
<body>
  <h1>A2A Explainable Stock Analytics Platform</h1>
  <p class="muted">AI-powered stock analytics platform with forecasting, explainability, evaluation, and risk insights</p>

  <div class="card">
    <label for="prompt">What Do You Want to Analyze?</label>
    <textarea id="prompt" rows="4">Analyze AAPL for the last 5 years and forecast the next 7 days. Include evaluation, explainability, risk, and produce an HTML report.</textarea>

    <div class="row">
      <div style="flex:1">
        <label for="tickers">Tickers</label>
        <input id="tickers" placeholder="Type your Ticker"/>
      </div>
      <div style="width:160px">
        <label for="forecast">Forecast days</label>
        <select id="forecast">
          <option value="3">3</option>
          <option value="5">5</option>
          <option value="7"selected>7</option>
        </select>
      </div>
    </div>

    <div style="margin-top:12px">
      <button id="runBtn">Run analysis</button>
      <span id="status" class="muted" style="margin-left:8px"></span>
    </div>

    <div id="result" style="margin-top:14px"></div>
  </div>

  <!-- Embedded report viewer -->
  <div id="viewer" style="margin-top:16px">
    <iframe id="reportFrame"
            style="width:100%; height:70vh; border:1px solid #e5e7eb; border-radius:12px"
            src="about:blank"></iframe>
  </div>

  <script>
    const statusEl = document.getElementById('status');
    const resEl    = document.getElementById('result');
    const runBtn   = document.getElementById('runBtn');
    const frameEl  = document.getElementById('reportFrame');

    document.getElementById('forecast').addEventListener('change', syncPrompt);
    document.getElementById('tickers').addEventListener('blur', syncPrompt);

    function syncPrompt(){
      const base = "Analyze {TICS} for the last 5 years and forecast the next {D} days. Include evaluation, explainability, risk, and produce an HTML report.";
      const d = document.getElementById('forecast').value;
      const t = (document.getElementById('tickers').value || "AAPL").trim();
      document.getElementById('prompt').value = base.replace("{TICS}", t).replace("{D}", d);
    }

    async function pollStatus(run_id, timeoutMs=10*60*1000, intervalMs=3000){
      const start = Date.now();
      while (Date.now() - start < timeoutMs){
        try{
          const r = await fetch(`/status/${encodeURIComponent(run_id)}`);
          const s = await r.json();
          if (s.ready) return s.report;
        }catch(e){ /* ignore and keep polling */ }
        await new Promise(res => setTimeout(res, intervalMs));
      }
      throw new Error("Timed out waiting for report.");
    }

    runBtn.onclick = async () => {
      resEl.innerHTML = "";
      frameEl.src = "about:blank";
      statusEl.innerHTML = `<span class="spinner"></span>Submitting…`;
      runBtn.disabled = true;

      try {
        const user_request = document.getElementById('prompt').value.trim();
        const tickers = (document.getElementById('tickers').value || "")
                        .split(/[,\s]+/).filter(Boolean);

        const body = { user_request, report_out_name: "llm_report.html" };
        if (tickers.length) body.tickers = tickers;


        const r = await fetch("/run-nl-async", {
          method: "POST",
          headers: {"Content-Type":"application/json"},
          body: JSON.stringify(body)
        });
        const data = await r.json();
        if (!r.ok) throw new Error(data.error || JSON.stringify(data));

        const run_id = data.run_id || (data.run_base || "").split("/").pop();
        if (!run_id) throw new Error("No run_id in response.");

        statusEl.innerHTML = `<span class="spinner"></span>Working… generating report…`;

        // Wait until the report exists, then display it
        const reportUrl = await pollStatus(run_id);
        resEl.innerHTML =
          `<div><b class="ok">Done.</b> Report: <a class="link" href="${reportUrl}" target="_blank">${reportUrl}</a></div>`;
        frameEl.src = reportUrl;

        frameEl.scrollIntoView({behavior:"smooth", block:"start"});
        statusEl.textContent = "";
      } catch (err) {
        statusEl.textContent = "";
        resEl.innerHTML = `<div class="err">Error: ${err.message || err}</div>`;
      } finally {
        runBtn.disabled = false;
      }
    };
  </script>
</body>
</html>


**app.py - Pipeline entry point**

In [ ]:
%%writefile /content/drive/MyDrive/A2A_prediction_system/backend/app.py

# backend/app.py
import os
import datetime
from fastapi import FastAPI, Body, HTTPException, BackgroundTasks
from fastapi.middleware.cors import CORSMiddleware
from fastapi.staticfiles import StaticFiles
from fastapi.responses import FileResponse, JSONResponse, HTMLResponse

from backend.a2a.orchestrator import run_orchestrator

app = FastAPI(title="A2A Orchestrator API")

# --- CORS  ---
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# --- Paths ---
A2A_ROOT = os.environ.get("A2A_ROOT", "/content/drive/MyDrive/A2A_prediction_system")
FRONTEND_DIR = os.path.join(A2A_ROOT, "frontend")

# Serve Drive so images in the report load
if os.path.isdir(A2A_ROOT):
    app.mount("/files", StaticFiles(directory=A2A_ROOT), name="files")

# Serve the static UI: <A2A_ROOT>/frontend/index.html -> /ui/index.html
if os.path.isdir(FRONTEND_DIR):
    app.mount("/ui", StaticFiles(directory=FRONTEND_DIR, html=True), name="ui")


# ---------------------------------------------------------------------
# Helpers
# ---------------------------------------------------------------------
def _new_run_base() -> tuple[str, str]:
    """Create a fresh run folder and return (run_id, run_base)."""
    run_id = datetime.datetime.now().strftime("RUN_%Y%m%d_%H%M%S")
    run_base = os.path.join(A2A_ROOT, run_id)
    os.makedirs(os.path.join(run_base, "reports"), exist_ok=True)
    return run_id, run_base


# ---------------------------------------------------------------------
# Routes
# ---------------------------------------------------------------------
@app.get("/")
def root_redirect():
    if os.path.isdir(FRONTEND_DIR):
        return HTMLResponse('<meta http-equiv="refresh" content="0; url=/ui/index.html">')
    return HTMLResponse('<meta http-equiv="refresh" content="0; url=/docs">')


@app.get("/health")
def health():
    return {"ok": True}


# Synchronous run with structured JSON
@app.post("/run")
def run_pipeline(payload: dict = Body(...)):
    payload = dict(payload or {})
    payload.setdefault("report_out_name", "llm_report.html")
    # Ensure a stable run_id in the response
    if "run_base" not in payload:
        run_id, run_base = _new_run_base()
        payload["run_base"] = run_base
    else:
        run_id = os.path.basename(payload["run_base"].rstrip("/"))
    res = run_orchestrator(payload)
    res["run_id"] = run_id
    return JSONResponse(res)


# Synchronous natural-language run
@app.post("/run-nl")
def run_pipeline_nl(
    user_request: str = Body(..., embed=True),
    report_out_name: str | None = Body(None),
    tickers: list[str] | None = Body(None),
):
    run_id, run_base = _new_run_base()
    cfg = {
        "user_request": user_request,
        "report_out_name": report_out_name or "llm_report.html",
        "run_base": run_base,
    }
    if tickers:
        cfg["tickers"] = tickers
    res = run_orchestrator(cfg)
    res["run_id"] = run_id
    return JSONResponse(res)


# Asynchronous natural-language run
@app.post("/run-nl-async")
def run_pipeline_nl_async(
    background_tasks: BackgroundTasks,
    user_request: str = Body(..., embed=True),
    report_out_name: str | None = Body(None),
    tickers: list[str] | None = Body(None),
):
    run_id, run_base = _new_run_base()
    cfg = {
        "user_request": user_request,
        "report_out_name": report_out_name or "llm_report.html",
        "run_base": run_base,
    }
    if tickers:
        cfg["tickers"] = tickers

    # Launch heavy pipeline in the background and return immediately
    background_tasks.add_task(run_orchestrator, cfg)

    return {
        "accepted": True,
        "run_id": run_id,
        "run_base": run_base,
        "status_url": f"/status/{run_id}",
        "report_url": f"/report-html/{run_id}",
        "message": "Run started in background.",
    }


# Is the HTML report there yet?
@app.get("/status/{run_id}")
def run_status(run_id: str):
    rpt = os.path.join(A2A_ROOT, run_id, "reports", "llm_report.html")
    ready = os.path.exists(rpt)
    return {
        "run_id": run_id,
        "ready": ready,
        "report": f"/report-html/{run_id}" if ready else None,
    }


# Download any report file (HTML or other)
@app.get("/report/{run_id}/{filename}")
def get_report(run_id: str, filename: str):
    path = os.path.join(A2A_ROOT, run_id, "reports", filename)
    if not os.path.exists(path):
        raise HTTPException(404, f"Report not found: {path}")
    return FileResponse(path)


# HTML preview: rewrite absolute Drive paths to files
@app.get("/report-html/{run_id}")
def preview_html(run_id: str):
    path = os.path.join(A2A_ROOT, run_id, "reports", "llm_report.html")
    if not os.path.exists(path):
        raise HTTPException(404, f"Report not found: {path}")
    with open(path, "r", encoding="utf-8") as f:
        content = f.read()
    content = content.replace(A2A_ROOT, "/files")
    return HTMLResponse(content)





In [ ]:
# backend/app.py
from fastapi import FastAPI
from a2a.orchestrator import run_orchestrator

app = FastAPI()

@app.get("/health")
def health():
    return {"ok": True}

@app.post("/run")
def run(payload: dict):
    return run_orchestrator(payload or {})


**orchestrator.py**

In [ ]:
%%writefile /content/drive/MyDrive/A2A_prediction_system/backend/a2a/orchestrator.py

import os, datetime
from typing import Dict, Any, Tuple

from langgraph.graph import StateGraph

# import agent modules
from . import reception                 as rec
from . import data_collector_agent      as dc
from . import feature_engineering_agent as fe
from . import predictive_model_agent    as pm
from . import model_evaluation_agent    as me
from . import explainability_agent      as xai
from . import risk_assessment_agent     as risk
from . import optimisation_agent        as opt
from . import summarizer_agent          as summ
from .report_agent import render_report


# =========================
# Helpers
# =========================
def _ts() -> str:
    return datetime.datetime.now().strftime("RUN_%Y%m%d_%H%M%S")

def _ensure_run_base(state: Dict[str, Any]) -> str:
    rb = state.get("run_base") or os.environ.get("RUN_BASE")
    if not rb:
        root = os.environ.get("A2A_ROOT", "/content/drive/MyDrive/A2A_prediction_system")
        rb = os.path.join(root, _ts())
    os.makedirs(rb, exist_ok=True)
    os.environ["RUN_BASE"] = rb
    return rb

def _ok(res: Dict[str, Any]) -> bool:
    """Consider a step ok unless it explicitly reports an error/failure."""
    if not isinstance(res, dict): return False
    if res.get("error"): return False
    s = str(res.get("status", "")).lower()
    if any(b in s for b in ("fail","error","exception","not found")):
        return False
    return True

def _invoke(build_fn, args: Dict[str, Any]) -> Tuple[bool, Dict[str, Any]]:
    app = build_fn()
    out = app.invoke(args)
    return _ok(out), out

def _flag(state: Dict[str, Any], *keys: str, default: bool=True) -> bool:
    """Return first present flag among synonyms, else default."""
    for k in keys:
        if k in state: return bool(state[k])
    return default


# =========================
# Nodes
# =========================
def node_init(state: Dict[str, Any]) -> Dict[str, Any]:
    rb = _ensure_run_base(state)

    # Map common top-level keys
    st = {**state, "run_base": rb}

    # Collector args: support both start/end and date_start/date_end
    ca = {**st.get("collector_args", {})}
    if "start_date" in st: ca["start"] = st["start_date"]
    if "end_date"   in st: ca["end"]   = st["end_date"]
    if "date_start" in st: ca["start"] = st["date_start"]
    if "date_end"   in st: ca["end"]   = st["date_end"]
    if "tickers"    in st: ca["tickers"] = st["tickers"]
    st["collector_args"] = ca

    # Predict args
    pa = {**st.get("predict_args", {})}
    if "seq_len" in st: pa["seq_len"] = st["seq_len"]
    st["predict_args"] = pa

    return {**st, "init_status": {"ok": True, "run_base": rb}}

def node_reception(state: Dict[str, Any]) -> Dict[str, Any]:
    """
    Parse natural-language request:
      - tickers
      - date_start/date_end
      - seq_len
    """
    ok, out = _invoke(rec.build_llm_reception_workflow, {
        "user_request": state.get("user_request", ""),
        "default_seq_len": state.get("default_seq_len", 40),
        "run_base": state["run_base"],
        "tickers": state.get("tickers"),
    })

    new_state = {**state, "reception_status": {"ok": ok}, "reception_out": out}
    print("[RECEPTION] ack:", out.get("ack_text"))

    # tickers
    tickers = out.get("tickers") or new_state.get("tickers")
    if tickers:
        new_state["tickers"] = tickers

    # date window
    ca = {**new_state.get("collector_args", {})}
    if out.get("date_start"): ca["start"] = out["date_start"]
    if out.get("date_end"):   ca["end"]   = out["date_end"]

    if out.get("start"): ca["start"] = out["start"]
    if out.get("end"):   ca["end"]   = out["end"]
    new_state["collector_args"] = ca

    # seq_len -> predict args
    pa = {**new_state.get("predict_args", {})}
    if out.get("seq_len"): pa["seq_len"] = out["seq_len"]
    new_state["predict_args"] = pa

    # run_base
    if out.get("run_base"):
        new_state["run_base"] = out["run_base"]
        os.environ["RUN_BASE"] = out["run_base"]

    return new_state

def node_collect(state: Dict[str, Any]) -> Dict[str, Any]:
    if not _flag(state, "run_collector", default=True):
        return state

    ok0, out = _invoke(dc.build_data_collector_workflow, {
        **state.get("collector_args", {}),
        "run_base": state["run_base"],
        "tickers": state.get("tickers"),
    })

    data_path = out.get("data_path")

    if not data_path:
        cand = os.path.join(state["run_base"], "data", "yfinance_raw_data.csv")
        if os.path.exists(cand): data_path = cand

    exists = bool(data_path and os.path.exists(data_path))
    ok = ok0 or exists or ("success" in str(out.get("status","")).lower())

    if ok and exists:
        fe_args = {**state.get("fe_args", {})}
        fe_args["data_path"]     = data_path
        fe_args["raw_data_path"] = data_path
        fe_args["input_csv"]     = data_path
        fe_args["run_base"]      = state["run_base"]
        return {**state,
                "collector_status": {"ok": True},
                "collector_out": out,
                "fe_args": fe_args}
    else:
        return {**state,
                "collector_status": {"ok": False, "error": out.get("error")},
                "collector_out": out,
                # disable downstream steps if collection failed
                "run_fe": False, "run_predict": False, "run_eval": False,
                "run_xai": False, "run_risk": False, "run_opt": False,
                "run_summary": False, "run_report": False}

def node_fe(state: Dict[str, Any]) -> Dict[str, Any]:
    if not _flag(state, "run_fe", default=True):
        return state
    ok, out = _invoke(fe.build_enhanced_feature_workflow, {
        **state.get("fe_args", {}),
        "run_base": state["run_base"],
    })
    return {**state, "fe_status": {"ok": ok}, "fe_out": out}

def node_predict(state: Dict[str, Any]) -> Dict[str, Any]:
    if not _flag(state, "run_predict", default=True):
        return state
    ok, out = _invoke(pm.build_predictive_workflow, {
        **state.get("predict_args", {}),
        "run_base": state["run_base"],
    })
    return {**state, "predict_status": {"ok": ok}, "predict_out": out}

def node_eval(state: Dict[str, Any]) -> Dict[str, Any]:
    if not _flag(state, "run_eval", default=True):
        return state
    ok, out = _invoke(me.build_evaluation_workflow, {
        **state.get("eval_args", {}),
        "run_base": state["run_base"],
    })
    return {**state, "eval_status": {"ok": ok}, "eval_out": out}

def node_xai(state: Dict[str, Any]) -> Dict[str, Any]:
    # accept multiple flag spellings
    if not _flag(state, "run_xai", "run_xai_shap", "run_xai_ig", default=True):
        return state
    ok, out = _invoke(xai.build_explainability_workflow, {
        **state.get("xai_args", {}),
        "run_base": state["run_base"],
    })
    return {**state, "xai_status": {"ok": ok}, "xai_out": out}

def node_risk(state: Dict[str, Any]) -> Dict[str, Any]:
    if not _flag(state, "run_risk", default=True):
        return state
    ok, out = _invoke(risk.build_risk_workflow, {
        **state.get("risk_args", {}),
        "run_base": state["run_base"],
    })
    return {**state, "risk_status": {"ok": ok}, "risk_out": out}

def node_opt(state: Dict[str, Any]) -> Dict[str, Any]:
    if not _flag(state, "run_opt", default=True):
        return state
    ok, out = _invoke(opt.build_optimization_workflow, {
        **state.get("opt_args", {}),
        "run_base": state["run_base"],
    })
    return {**state, "opt_status": {"ok": ok}, "opt_out": out}

def node_summary(state: Dict[str, Any]) -> Dict[str, Any]:
    if not _flag(state, "run_summary", "run_summarizer", default=True):
        return state
    ok, out = _invoke(summ.build_summarizer_workflow, {
        **state.get("summary_args", {}),
        "run_base": state["run_base"],
    })
    return {**state, "summary_status": {"ok": ok}, "summary_out": out}

def node_report(state: Dict[str, Any]) -> Dict[str, Any]:
    if not _flag(state, "run_report", "run_llm_report", default=True):
        return state
    path = render_report(
        run_base=state["run_base"],
        out_name=state.get("report_out_name","llm_report.html"),
        order=state.get("tickers"),
    )
    return {**state, "report_status": {"ok": bool(path)}, "report_path": path}


# =========================
# Workflow
# =========================
def build_orchestrator_workflow():
    g = StateGraph(dict)
    g.add_node("init",      node_init)
    g.add_node("reception", node_reception)
    g.add_node("collect",   node_collect)
    g.add_node("features",  node_fe)
    g.add_node("predict",   node_predict)
    g.add_node("eval",      node_eval)
    g.add_node("xai",       node_xai)
    g.add_node("risk",      node_risk)
    g.add_node("opt",       node_opt)
    g.add_node("summary",   node_summary)
    g.add_node("report",    node_report)

    g.set_entry_point("init")
    g.add_edge("init", "reception")
    g.add_edge("reception","collect")
    g.add_edge("collect","features")
    g.add_edge("features","predict")
    g.add_edge("predict","eval")
    g.add_edge("eval","xai")
    g.add_edge("xai","risk")
    g.add_edge("risk","opt")
    g.add_edge("opt","summary")
    g.add_edge("summary","report")
    g.set_finish_point("report")
    return g.compile()

def run_orchestrator(config: Dict[str, Any]) -> Dict[str, Any]:
    print("[ENTRY]", "NL mode" if config.get("user_request") else "Structured mode")
    app = build_orchestrator_workflow()
    out = app.invoke(config or {})
    import os
    run_base = out.get("run_base") or ""
    run_id   = os.path.basename(run_base) if run_base else None
    return {
        "status": "done",
        "run_id": run_id,
        "run_base": run_base,
        "report_path": out.get("report_path"),
        "reception_status": out.get("reception_status"),
        "reception_out": out.get("reception_out"),
        "collector_status": out.get("collector_status"),
        "fe_status": out.get("fe_status"),
        "predict_status": out.get("predict_status"),
        "eval_status": out.get("eval_status"),
        "xai_status": out.get("xai_status"),
        "risk_status": out.get("risk_status"),
        "opt_status": out.get("opt_status"),
        "summary_status": out.get("summary_status"),
        "report_status": out.get("report_status"),
    }


In [ ]:
%%writefile /content/drive/MyDrive/A2A_prediction_system/backend/a2a/reception.py
import re
from typing import Dict, Any, List, Tuple, Optional
import pandas as pd

try:
    from langgraph.graph import StateGraph
    HAS_LG = True
except Exception:
    HAS_LG = False


# ---------------- helpers ----------------

def _now_date() -> pd.Timestamp:
    return pd.Timestamp.today().normalize()

def _parse_rel_window(text: str) -> Tuple[Optional[pd.Timestamp], Optional[pd.Timestamp]]:
    """Parse phrases like 'last 5 years', 'last 6 months', 'last 30 days'."""
    t = text.lower()
    end = _now_date()
    if re.search(r'\bytd\b|\byear[-\s]*to[-\s]*date\b', t):
        start = pd.Timestamp(end.year, 1, 1)
        return start, end
    m = re.search(r'last\s+(\d+)\s*(year|years|yr|yrs|month|months|mo|mos|week|weeks|day|days)', t)
    if m:
        n = int(m.group(1))
        unit = m.group(2)
        if unit.startswith(('year', 'yr')):
            start = end - pd.DateOffset(years=n)
        elif unit.startswith('month') or unit.startswith('mo'):
            start = end - pd.DateOffset(months=n)
        elif unit.startswith('week'):
            start = end - pd.Timedelta(days=7 * n)
        else:
            start = end - pd.Timedelta(days=n)
        return start.normalize(), end
    return None, None

def _parse_explicit_dates(text: str) -> Tuple[Optional[pd.Timestamp], Optional[pd.Timestamp]]:
    """Parse explicit 'from YYYY-MM-DD' / 'to YYYY-MM-DD' spans."""
    start = end = None
    m = re.search(r'(from|since|start[,:\s]*)\s*([0-9]{4}[-/][0-9]{1,2}[-/][0-9]{1,2})', text, flags=re.I)
    if m:
        try: start = pd.to_datetime(m.group(2))
        except Exception: pass
    m = re.search(r'(to|until|end[,:\s]*)\s*([0-9]{4}[-/][0-9]{1,2}[-/][0-9]{1,2})', text, flags=re.I)
    if m:
        try: end = pd.to_datetime(m.group(2))
        except Exception: pass
    return start, end

def _default_dates(start: Optional[pd.Timestamp], end: Optional[pd.Timestamp]) -> Tuple[pd.Timestamp, pd.Timestamp]:
    """If dates missing, default to last 3 years."""
    e = (end or _now_date()).normalize()
    s = (start or (e - pd.DateOffset(years=3))).normalize()
    if s > e:
        s, e = e - pd.DateOffset(years=1), e
    return s, e

def _parse_seq_len(text: str, default: int = 30) -> int:
    m = re.search(r'(seq[_\s-]*len|sequence\s*length)\s*[:=]?\s*(\d+)', text, flags=re.I)
    if m:
        try:
            return max(2, int(m.group(2)))
        except Exception:
            return default
    return default

def _parse_tickers(text: str) -> List[str]:
    """General ticker parser."""
    t = text.upper()
    STOP = {
        "I","ME","MY","WE","OUR","YOU","YOUR","PLEASE",
        "WANT","KNOW","SHOW","TELL","TREND","TRENDS","STOCK","MARKET",
        "FOR","OF","THE","A","AN","AND","LAST","YEARS","YEAR",
        "MONTHS","MONTH","WEEKS","WEEK","DAYS","DAY",
        "IS","IT","OKAY","TO","TRADE","NOW","ABOUT","ON","IN","OVER",
        "RUN","FE","SEQ","LEN","YTD","FROM","SINCE","TO","UNTIL","END",
        "PREDICT","EVAL","RISK","OPT","IG","SHAP","SUMMARY","REPORT",
        "NASDAQ","NYSE","LSE","EURONEXT","AMEX","TSX","ASX","BSE","NSE","SHOULD",
    }
    toks: List[str] = []
    for m in re.finditer(r'\b(?:NYSE|NASDAQ|LSE|EURONEXT|AMEX|TSX|ASX|BSE|NSE)[:\s]+([A-Z.-]{1,6})\b', t):
        tok = m.group(1).replace('-', '.'); toks.append(tok)
    for m in re.finditer(r'\b[A-Z][A-Z.-]{0,5}\b', t):
        tok = m.group(0)
        if tok in STOP or len(tok) <= 1: continue
        toks.append(tok.replace('-', '.'))
    m = re.search(r'(?:\brun\b|\btickers?)\s*[:=]\s*([A-Z0-9,.\s-]+)', t)
    if m:
        more = [s.strip().replace('-', '.') for s in re.split(r'[,\s]+', m.group(1)) if s.strip()]
        toks.extend(more)
    seen, out = set(), []
    for tok in toks:
        if re.fullmatch(r'[A-Z.]{1,6}', tok) and tok not in seen and tok not in STOP:
            seen.add(tok); out.append(tok)
    return out or ["AAPL"]

def _user_intends_decision(text: str) -> bool:
    t = text.lower()
    return any(kw in t for kw in [
        "is it okay to trade", "should i trade", "go or no-go", "buy", "enter", "trade now", "okay to trade"
    ])

def _parse_steps(text: str) -> Dict[str, bool]:
    """Turn words in the prompt into pipeline flags."""
    t = text.lower()
    mentioned = any(k in t for k in ["collect","fe","feature","predict","eval","risk","opt","ig","shap","summary","report"])
    defaults = dict(
        run_collector=True, run_fe=True, run_predict=True, run_eval=True,
        run_xai_ig=True, run_xai_shap=True, run_risk=True, run_opt=True,
        run_portfolio=False, run_summarizer=True, run_llm_report=True,
    )
    if not mentioned: return defaults
    flags = {k: False for k in defaults}
    def has(*keys): return any(k in t for k in keys)
    flags["run_collector"]  = has("collect","collector","download")
    flags["run_fe"]         = has("fe","feature")
    flags["run_predict"]    = has("predict")
    flags["run_eval"]       = has("eval","evaluation")
    flags["run_xai_ig"]     = has(" ig ","integrated gradients")
    flags["run_xai_shap"]   = has("shap")
    flags["run_risk"]       = has("risk")
    flags["run_opt"]        = has("opt","optimization")
    flags["run_portfolio"]  = has("portfolio")
    flags["run_summarizer"] = has("summary","summarizer")
    flags["run_llm_report"] = has("report")
    for k, v in defaults.items():
        if k not in flags: flags[k] = v
    return flags

def _make_run_base(base: Optional[str] = None) -> str:
    if base: return base
    return pd.Timestamp.now().strftime("RUN_%Y%m%d_%H%M%S")


# ---------------- main NL node ----------------

def node_parse(state: Dict[str, Any]) -> Dict[str, Any]:
    """
    Reception: parse tickers, dates (explicit/relative), seq_len and flags.
    Falls back to sensible defaults when the user doesn't specify.
    """
    txt = (state.get("user_request", "") or "").strip()
    if not txt:
        txt = "Analyze AAPL last 3 years; seq_len 30"

    # tickers
    tickers = state.get("tickers") or _parse_tickers(txt) or ["AAPL"]

    # dates
    s1, e1 = _parse_explicit_dates(txt)
    s2, e2 = _parse_rel_window(txt)
    start, end = _default_dates(s1 or s2, e1 or e2)

    # seq_len
    seq_len = _parse_seq_len(txt, default=int(state.get("default_seq_len", 30)))

    # pipeline flags
    flags = _parse_steps(txt)

    # intent
    wants_decision = _user_intends_decision(txt)

    # run folder
    run_base = _make_run_base(state.get("run_base"))

    # UX text
    pretty_tics = ", ".join(tickers)
    daterange = f"{start.date()} \u2192 {end.date()}"
    ack = (f"Thanks! I’ll analyze {pretty_tics} over {daterange}, then coordinate "
           f"data → features → prediction → evaluation → IG & SHAP → risk → optimization, and return a report.")
    plan = (f"Plan: tickers={tickers}, window={start.date()}..{end.date()}, "
            f"seq_len={seq_len}, steps={[k for k, v in flags.items() if v]}, run_base='{run_base}', "
            f"decision_requested={wants_decision}.")

    # Per-stage args
    base_args = {"run_base": run_base, "tickers": tickers}
    collector_args = {**base_args, "start": str(start.date()), "end": str(end.date())}
    fe_args = {**base_args}
    predict_args = {**base_args, "seq_len": seq_len}
    eval_args = {**base_args}
    risk_args = {**base_args}
    ig_args = {**base_args, "ig_head": "close"}
    shap_args = {**base_args, "head": "close"}
    opt_args = {**base_args, "mode": "historical", "head": "close"}

    return {
        **state,
        "status": "parsed",
        "ack_text": ack,
        "plan_text": plan,
        "run_base": run_base,
        "tickers": tickers,
        "seq_len": seq_len,
        "date_start": str(start.date()),
        "date_end": str(end.date()),
        "wants_decision": wants_decision,
        **flags,
        "collector_args": collector_args,
        "fe_args": fe_args,
        "predict_args": predict_args,
        "eval_args": eval_args,
        "risk_args": risk_args,
        "ig_args": ig_args,
        "shap_args": shap_args,
        "opt_args": opt_args,
    }


# ---------------- optional workflow ----------------

def build_llm_reception_workflow():
    if not HAS_LG:
        raise RuntimeError("LangGraph not available. Install with: pip install langgraph")
    g = StateGraph(dict)
    g.add_node("parse", node_parse)
    g.set_entry_point("parse")
    g.set_finish_point("parse")
    return g.compile()


**Data_collector_agent.py**

In [ ]:
%%writefile /content/drive/MyDrive/A2A_prediction_system/backend/a2a/data_collector_agent.py
import os
from datetime import datetime, timedelta
from typing import TypedDict, Optional, List

import pandas as pd
import yfinance as yf


try:
    from langgraph.graph import StateGraph
    HAS_LG = True
except Exception:
    HAS_LG = False


class DataCollectorState(TypedDict, total=False):
    run_base: Optional[str]
    tickers: Optional[List[str]]
    date_start: Optional[str]
    date_end: Optional[str]
    data_path: Optional[str]
    status: Optional[str]
    error: Optional[str]


def _drive_base() -> str:

    drive_root = "/content/drive/MyDrive/A2A_prediction_system"
    local_root = "/content/A2A_prediction_system"
    if os.path.isdir("/content/drive/MyDrive"):
        os.makedirs(drive_root, exist_ok=True)
        return drive_root
    os.makedirs(local_root, exist_ok=True)
    return local_root


def _compute_run_base(state: DataCollectorState) -> str:

    if state.get("run_base"):
        return state["run_base"]

    env_rb = os.environ.get("RUN_BASE")
    if env_rb:
        os.makedirs(env_rb, exist_ok=True)
        return env_rb

    run_id = pd.Timestamp.now().strftime("RUN_%Y%m%d_%H%M%S")
    return os.path.join(_drive_base(), run_id)



def _as_date(s: Optional[str], fallback: Optional[datetime]) -> datetime:
    if s:
        try:
            return pd.to_datetime(s).to_pydatetime()
        except Exception:
            pass
    return fallback or datetime.today()


def _normalize_downloaded_df(df: pd.DataFrame, ticker: str) -> Optional[pd.DataFrame]:
    """
    Return a DataFrame with columns: Date, Open, Close, High, Low, Volume
    Works for:
      - standard columns (Open, High, Low, Close, Volume)
      - MultiIndex flattened to either Open_AAPL or AAPL_Open
    """
    if df is None or df.empty:
        return None

    df = df.reset_index()
    base_cols = {"Open", "High", "Low", "Close", "Volume"}


    if not isinstance(df.columns, pd.MultiIndex):
        if base_cols.issubset(set(df.columns)):
            return df[["Date", "Open", "Close", "High", "Low", "Volume"]].copy()

        return None

    # MultiIndex case: flatten to strings
    flat_cols = []
    for col in df.columns:
        if isinstance(col, tuple):
            flat_cols.append("_".join([c for c in col if c]).strip())
        else:
            flat_cols.append(str(col))
    df.columns = flat_cols

    fields = ["Open", "Close", "High", "Low", "Volume"]
    # Accept BOTH orders
    needed_field_first = ["Date"] + [f"{f}_{ticker}" for f in fields]   # Open_AAPL
    needed_ticker_first = ["Date"] + [f"{ticker}_{f}" for f in fields]  # AAPL_Open

    if all(c in df.columns for c in needed_field_first):
        out = df[needed_field_first].copy()
        out.columns = ["Date", "Open", "Close", "High", "Low", "Volume"]
        return out
    if all(c in df.columns for c in needed_ticker_first):
        tmp = df[needed_ticker_first].copy()
        rename_map = {f"{ticker}_Open": "Open",
                      f"{ticker}_Close": "Close",
                      f"{ticker}_High": "High",
                      f"{ticker}_Low": "Low",
                      f"{ticker}_Volume": "Volume"}
        tmp = tmp.rename(columns=rename_map)
        return tmp[["Date", "Open", "Close", "High", "Low", "Volume"]].copy()


    return None


def fetch_5yr_stock_data(state: DataCollectorState) -> DataCollectorState:
    print("[DataCollector] incoming run_base:", state.get("run_base"), "| env RUN_BASE:", os.environ.get("RUN_BASE"))

    try:
        # Resolve config (with safe fallbacks)
        tickers = state.get("tickers") or ["AAPL"]
        end_date = _as_date(state.get("date_end"), datetime.today())
        start_date = _as_date(state.get("date_start"), end_date - timedelta(days=5 * 365))
        run_base = _compute_run_base(state)

        output_dir = os.path.join(run_base, "data")
        os.makedirs(output_dir, exist_ok=True)
        output_path = os.path.join(output_dir, "yfinance_raw_data.csv")

        print(f"\n[DataCollector] Tickers: {tickers}")
        print(f"[DataCollector] Window : {start_date.date()} → {end_date.date()}")
        print(f"[DataCollector] Output : {output_path}")

        # Download per ticker
        all_data = []
        success_count = 0

        for ticker in tickers:
            print(f"\nDownloading: {ticker}")
            try:
                # Use field-first layout
                df = yf.download(
                    tickers=ticker,
                    start=start_date,
                    end=end_date,
                    progress=False,
                    group_by="column",
                    auto_adjust=False,
                    actions=False,
                    rounding=False,
                    threads=True,
                )

                df_clean = _normalize_downloaded_df(df, ticker)
                if df_clean is None or df_clean.empty:

                    df2 = yf.download(
                        tickers=ticker,
                        start=start_date,
                        end=end_date,
                        progress=False,
                        auto_adjust=False,
                        actions=False,
                        rounding=False,
                        threads=True,
                    )
                    df_clean = _normalize_downloaded_df(df2, ticker)

                if df_clean is None or df_clean.empty:
                    print(f"  {ticker}: Could not normalize columns.")
                    continue

                # Clean types
                df_clean["Symbol"] = ticker
                df_clean["Date"] = pd.to_datetime(df_clean["Date"], errors="coerce")
                for col in ["Open", "Close", "High", "Low", "Volume"]:
                    df_clean[col] = pd.to_numeric(df_clean[col], errors="coerce")
                df_clean.dropna(subset=["Date", "Open", "Close", "High", "Low", "Volume"], inplace=True)

                if df_clean.empty:
                    print(f"  {ticker}: No valid rows after cleaning.")
                    continue

                print(f"  {ticker}: {len(df_clean)} rows ready.")
                all_data.append(df_clean)
                success_count += 1

            except Exception as e:
                print(f"  Error fetching {ticker}: {e}")
                continue

        if not all_data:
            return {**state, "run_base": run_base, "status": "Failed", "error": "No valid stock data collected."}

        # Concatenate & save
        final_df = pd.concat(all_data, ignore_index=True)
        final_df.sort_values(["Symbol", "Date"], inplace=True)
        final_df.to_csv(output_path, index=False)

        print(f"\n[DataCollector] Saved: {output_path}")
        print(f"[DataCollector] Records: {len(final_df)} from {success_count} ticker(s)")
        print(f"[DataCollector] Columns: {list(final_df.columns)}")

        return {
            **state,
            "run_base": run_base,
            "data_path": output_path,
            "status": f"Success: {success_count} ticker(s) collected",
        }

    except Exception as e:
        return {**state, "status": "Error", "error": str(e)}


def build_data_collector_workflow():
    if not HAS_LG:
        raise RuntimeError("LangGraph not available. Install with: pip install langgraph")
    graph = StateGraph(dict)
    graph.add_node("FetchData", fetch_5yr_stock_data)
    graph.set_entry_point("FetchData")
    graph.set_finish_point("FetchData")
    return graph.compile()


if __name__ == "__main__":
    print("Starting 5-Year Stock Data Collection Agent...\n")
    demo_state: DataCollectorState = {
        "tickers": ["AAPL"],
        "date_start": (datetime.today() - timedelta(days=5*365)).date().isoformat(),
        "date_end": datetime.today().date().isoformat(),

    }
    try:
        if HAS_LG:
            app = build_data_collector_workflow()
            result = app.invoke(demo_state)
        else:
            result = fetch_5yr_stock_data(demo_state)

        print("\nPipeline Complete")
        print(f" Status: {result.get('status')}")
        if result.get("error"):
            print(f" Error: {result['error']}")
        else:
            print(f" Data saved to: {result['data_path']}")
    except Exception as e:
        print("Fatal error:", e)





**feature_engineering_agent.py**

In [ ]:
%%writefile /content/drive/MyDrive/A2A_prediction_system/backend/a2a/feature_engineering_agent.py
import os, json
from pathlib import Path
from datetime import datetime
from typing import TypedDict, Optional, List, Dict, Any

import numpy as np
import pandas as pd
import logging


try:
    from langgraph.graph import StateGraph
    HAS_LG = True
except Exception:
    HAS_LG = False

# ---------------- Logging ----------------
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# --------------- State type --------------
class FeatureState(TypedDict, total=False):
    run_base: Optional[str]
    data_path: Optional[str]
    fe_path: Optional[str]
    fe_model_path: Optional[str]
    model_features: Optional[List[str]]
    status: Optional[str]
    error: Optional[str]

    data: Optional[pd.DataFrame]

# --------------- Helpers -----------------
def _infer_run_base(state: FeatureState) -> Path:

    if state.get("run_base"):
        return Path(str(state["run_base"]))
    dp = state.get("data_path")
    if dp:
        p = Path(dp)
        # expect .../<run_base>/data/filename.csv
        return p.parent.parent
    return Path(".")

def _atomic_write_csv(df: pd.DataFrame, path: Path) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)
    tmp = path.with_suffix(path.suffix + ".tmp")
    df.to_csv(tmp, index=False)
    tmp.replace(path)

def winsorize_series(s: pd.Series, lower_q=0.01, upper_q=0.99) -> pd.Series:
    s = s.astype(float)
    lo = s.quantile(lower_q)
    hi = s.quantile(upper_q)
    return s.clip(lower=lo, upper=hi)

# ============== Load data =============
def load_stock_data(state: FeatureState) -> FeatureState:
    logger.info("Loading stock data...")
    try:
        data_path = state.get("data_path")
        if not data_path or not os.path.exists(data_path):
            raise FileNotFoundError(f"Raw data file not found: {data_path}")

        df = pd.read_csv(data_path, parse_dates=["Date"])
        logger.info(f"Loaded: {data_path} shape={df.shape}")

        # Normalize column names
        column_mapping = {
            'symbol': 'Symbol', 'ticker': 'Symbol', 'SYMBOL': 'Symbol',
            'date': 'Date', 'DATE': 'Date',
            'open': 'Open', 'OPEN': 'Open',
            'close': 'Close', 'CLOSE': 'Close',
            'high': 'High', 'HIGH': 'High',
            'low': 'Low', 'LOW': 'Low',
            'volume': 'Volume', 'VOLUME': 'Volume',
        }
        df.rename(columns=column_mapping, inplace=True)

        expected = ["Date", "Symbol", "Open", "Close", "High", "Low", "Volume"]
        miss = [c for c in expected if c not in df.columns]
        if miss:
            raise ValueError(f"Missing required columns: {miss}")

        df = df[expected].copy()
        df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
        for c in ["Open", "Close", "High", "Low", "Volume"]:
            df[c] = pd.to_numeric(df[c], errors="coerce")

        # Basic validation & cleaning
        n0 = len(df)
        df = df.dropna(subset=["Date", "Symbol", "Open", "Close", "High", "Low", "Volume"])
        df = df[df["Volume"] > 0]
        df = df[df["High"] >= df["Low"]]
        df = df[df["Close"] > 0]
        df = df.drop_duplicates(["Symbol", "Date"]).sort_values(["Symbol", "Date"]).reset_index(drop=True)
        logger.info(f"Validation: {n0} -> {len(df)} rows")

        return {**state, "data": df, "status": "Data loaded successfully"}
    except Exception as e:
        logger.error(f"Error loading data: {e}")
        return {**state, "status": "Failed", "error": f"Error loading data: {e}"}

# ============== Create features =======
def create_enhanced_features(state: FeatureState) -> FeatureState:
    logger.info("Creating features...")
    try:
        df = state["data"]
        syms = df["Symbol"].unique()
        logger.info(f"Symbols: {list(syms)}")

        processed: List[pd.DataFrame] = []

        for sym in syms:
            s = df[df["Symbol"] == sym].copy().sort_values("Date")
            if len(s) < 50:
                logger.warning(f"{sym}: too few rows ({len(s)}), skip")
                continue

            # Returns
            s["Return_1d"] = s["Close"].pct_change(1)

            # MAs
            s["MA_5"]  = s["Close"].rolling(5).mean()
            s["MA_20"] = s["Close"].rolling(20).mean()
            s["Close_MA5_Ratio"]  = s["Close"] / s["MA_5"].replace(0, np.nan)
            s["Close_MA20_Ratio"] = s["Close"] / s["MA_20"].replace(0, np.nan)
            s["MA5_MA20_Ratio"]   = s["MA_5"] / s["MA_20"].replace(0, np.nan)
            s["MA_5_Slope"]  = s["MA_5"].diff(5) / s["MA_5"].shift(5).replace(0, np.nan)
            s["MA_20_Slope"] = s["MA_20"].diff(5) / s["MA_20"].shift(5).replace(0, np.nan)

            # RSI
            delta = s["Close"].diff()
            gain = delta.clip(lower=0).rolling(14, min_periods=7).mean()
            loss = (-delta.clip(upper=0)).rolling(14, min_periods=7).mean().replace(0, np.nan)
            rs = gain / loss
            s["RSI"] = 100 - (100 / (1 + rs))

            # MACD (histogram)
            exp1 = s["Close"].ewm(span=12, adjust=False).mean()
            exp2 = s["Close"].ewm(span=26, adjust=False).mean()
            macd = exp1 - exp2
            macd_sig = macd.ewm(span=9, adjust=False).mean()
            s["MACD_Histogram"] = macd - macd_sig

            # Bollinger position
            bb_ma = s["Close"].rolling(20).mean()
            bb_sd = s["Close"].rolling(20).std()
            bb_up = bb_ma + 2 * bb_sd
            bb_lo = bb_ma - 2 * bb_sd
            span = (bb_up - bb_lo).replace(0, np.nan)
            s["BB_Position"] = (s["Close"] - bb_lo) / span

            # Ratios / ranges
            denom_hl = (s["High"] - s["Low"]).replace(0, np.nan)
            s["High_Low_Ratio"] = (s["Close"] - s["Low"]) / denom_hl
            s["Daily_Range"] = (s["High"] - s["Low"]) / s["Close"].replace(0, np.nan)
            s["Open_Close_Ratio"] = s["Open"] / s["Close"].replace(0, np.nan)

            # Volatility & volume signal
            s["Volatility_20d"] = s["Return_1d"].rolling(20, min_periods=10).std()
            vol_ma10 = s["Volume"].rolling(10, min_periods=5).mean()
            s["Volume_Ratio"] = s["Volume"] / vol_ma10.replace(0, np.nan)

            # Winsorize (cap outliers)
            for col in ["Return_1d", "Volume_Ratio", "Daily_Range"]:
                if col in s.columns:
                    s[col] = winsorize_series(s[col])

            # Targets (analysis only)
            s["Next_Day_Return"] = s["Return_1d"].shift(-1)
            s["Next_Day_Direction"] = (s["Next_Day_Return"] > 0).astype(int)

            s = s.dropna(subset=["Open", "Close", "High", "Low", "Volume"])
            processed.append(s)

        if not processed:
            raise ValueError("No symbols processed successfully")

        final_df = pd.concat(processed, ignore_index=True)
        final_df = final_df.sort_values(["Symbol", "Date"]).replace([np.inf, -np.inf], np.nan)

        # Core features used for modeling (no targets)
        CORE_MODEL_FEATURES = [
            "Open","Close","High","Low","Volume",
            "Return_1d","Volatility_20d",
            "Close_MA5_Ratio","Close_MA20_Ratio","MA5_MA20_Ratio",
            "MA_5_Slope","MA_20_Slope",
            "RSI","MACD_Histogram","BB_Position",
            "High_Low_Ratio","Daily_Range","Open_Close_Ratio",
            "Volume_Ratio"
        ]
        available_core = [c for c in CORE_MODEL_FEATURES if c in final_df.columns]
        logger.info(f"Model features: {len(available_core)}/{len(CORE_MODEL_FEATURES)} available")

        return {**state, "data": final_df, "model_features": available_core,
                "status": "Features created successfully"}
    except Exception as e:
        logger.error(f"Error in create_enhanced_features: {e}")
        return {**state, "status": "Failed", "error": f"Error in create_features: {e}"}

# ============== Handle NaNs ===========
def handle_missing_after_fe(state: FeatureState) -> FeatureState:
    logger.info("Handling NaNs after FE...")
    try:
        df = state["data"].copy().sort_values(["Symbol","Date"])
        target_cols = {"Next_Day_Return", "Next_Day_Direction"}

        num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
        feat_cols = [c for c in num_cols if c not in target_cols]

        # ffill/bfill per symbol
        df[feat_cols] = df.groupby("Symbol", group_keys=False)[feat_cols].transform(lambda g: g.ffill().bfill())

        # median per symbol then global
        per_sym_med = df.groupby("Symbol", group_keys=False)[feat_cols].transform("median")
        df[feat_cols] = df[feat_cols].fillna(per_sym_med)
        df[feat_cols] = df[feat_cols].fillna(df[feat_cols].median(numeric_only=True))

        # drop rows with critical NA and tail NA
        before = len(df)
        df = df.dropna(subset=["Close", "Return_1d"])
        dropped = before - len(df)
        if "Next_Day_Return" in df.columns:
            before2 = len(df)
            df = df.dropna(subset=["Next_Day_Return"])
            dropped += before2 - len(df)
        logger.info(f"Dropped rows due to NA: {dropped}")

        # report remaining NaNs
        rem = df.isna().sum()
        rem = rem[rem > 0]
        if len(rem):
            logger.warning("Remaining NaNs:\n" + "\n".join([f"  {k}: {v}" for k,v in rem.items()]))
        else:
            logger.info("All NaNs handled.")

        return {**state, "data": df, "status": "NaNs handled after FE"}
    except Exception as e:
        logger.error(f"Error in handle_missing_after_fe: {e}")
        return {**state, "status": "Failed", "error": f"Error in NaN handling: {e}"}

# ============== Validate/Summarize =====
def validate_and_summarize(state: FeatureState) -> FeatureState:
    logger.info("Validating & summarizing...")
    try:
        df = state["data"]
        logger.info(f"Rows: {len(df):,}")
        logger.info(f"Symbols: {df['Symbol'].nunique()}")
        logger.info(f"Date range: {df['Date'].min()} → {df['Date'].max()}")

        inf_cols = [c for c in df.select_dtypes(include=[np.number]).columns if np.isinf(df[c]).any()]
        if inf_cols:
            logger.warning(f"Infinite values in: {inf_cols}")

        const_cols = [c for c in df.select_dtypes(include=[np.number]).columns if df[c].nunique(dropna=True) <= 1]
        if const_cols:
            logger.warning(f"Constant columns (consider drop): {const_cols}")

        return {**state, "status": "Features validated and summarized"}
    except Exception as e:
        logger.error(f"Error in validate_and_summarize: {e}")
        return {**state, "status": "Failed", "error": f"Error in validation: {e}"}

# ============== Save outputs ===========
def save_enhanced_features(state: FeatureState) -> FeatureState:
    logger.info("Saving features...")
    try:
        df = state["data"].copy()
        model_feats = state.get("model_features", []) or []

        run_base = _infer_run_base(state)
        out_dir  = run_base / "FE_Agent"
        out_dir.mkdir(parents=True, exist_ok=True)

        # Full file
        full_path = out_dir / "features_engineered.csv"
        _atomic_write_csv(df, full_path)
        logger.info(f"Full features: {full_path}")

        # model file
        keep_cols = ["Date", "Symbol"] + model_feats
        slim = df[keep_cols].copy()
        slim_path = out_dir / "features_for_model.csv"
        _atomic_write_csv(slim, slim_path)
        logger.info(f"Model features: {slim_path}")

        # Artifacts near full_path
        base = full_path.with_suffix("")
        ts = datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ")

        all_feats = [c for c in df.columns if c not in ["Date","Symbol"]]
        (base.with_name(base.name + "_feature_list.json")
         ).write_text(json.dumps({"generated_at": ts, "count": len(all_feats), "features": all_feats}, indent=2))

        (base.with_name(base.name + "_model_feature_list.json")
         ).write_text(json.dumps({"generated_at": ts, "count": len(model_feats), "features": model_feats}, indent=2))

        schema = {c: str(t) for c, t in df.dtypes.items()}
        (base.with_name(base.name + "_schema.json")
         ).write_text(json.dumps({"generated_at": ts, "schema": schema}, indent=2))

        df.describe().to_csv(base.with_name(base.name + "_summary.csv"), index=True)

        (base.with_name(base.name + "_VERSION.txt")
         ).write_text(
            f"features_version: 2.0\n"
            f"generated_at_utc: {ts}\n"
            f"rows: {len(df)}\n"
            f"columns_full: {len(df.columns)}\n"
            f"columns_for_model: {len(keep_cols)}\n"
        )

        # keep state small for next agent/orchestrator
        out_state = {
            **state,
            "fe_path": str(full_path),
            "fe_model_path": str(slim_path),
            "model_features": model_feats,
            "status": "Features saved successfully",
        }
        out_state.pop("data", None)
        return out_state

    except Exception as e:
        logger.error(f"Error saving features: {e}")
        return {**state, "status": "Failed", "error": f"Error saving features: {e}"}

# ======= Simple function for orchestrator =======
def run_fe(state: Dict[str, Any]) -> Dict[str, Any]:

    s = FeatureState(**state)

    for step in (load_stock_data, create_enhanced_features, handle_missing_after_fe,
                 validate_and_summarize, save_enhanced_features):
        s = step(s)
        if str(s.get("status","")).startswith("Failed"):
            return s
    return s

# ============ LangGraph ============
def build_enhanced_feature_workflow():
    if not HAS_LG:
        raise RuntimeError("LangGraph not available. Install with: pip install langgraph")
    graph = StateGraph(FeatureState)
    graph.add_node("load", load_stock_data)
    graph.add_node("create", create_enhanced_features)
    graph.add_node("nans", handle_missing_after_fe)
    graph.add_node("validate", validate_and_summarize)
    graph.add_node("save", save_enhanced_features)

    graph.set_entry_point("load")
    graph.add_edge("load", "create")
    graph.add_edge("create", "nans")
    graph.add_edge("nans", "validate")
    graph.add_edge("validate", "save")
    graph.set_finish_point("save")
    return graph.compile()

# =============== Demo run =================
if __name__ == "__main__":
    print("Starting Feature Engineering Agent...\n")

    example_state: FeatureState = {
        "data_path": "data/Yfinance_real_time_data.csv",
        "run_base": ".",
    }
    out = run_fe(example_state)
    print("\nStatus:", out.get("status"))
    if out.get("error"):
        print("Error:", out["error"])
    else:
        print("FE path:", out.get("fe_path"))
        print("Model features path:", out.get("fe_model_path"))


**predictive_model_agent.py**

In [ ]:
%%writefile /content/drive/MyDrive/A2A_prediction_system/backend/a2a/predictive_model_agent.py
import os, json, warnings, shutil
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from typing import Dict, Any, List, Optional
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import joblib

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# LangGraph
try:
    from langgraph.graph import StateGraph
    HAS_LG = True
except Exception:
    HAS_LG = False

np.random.seed(42)
tf.random.set_seed(42)


# ============================== Utilities ==============================
def _infer_run_base(state: Dict[str, Any]) -> str:
    if state.get("run_base"):
        return str(state["run_base"])
    fe_path = state.get("fe_path")
    if fe_path:
        # path../<run_base>/FE_Agent/features_engineered.csv
        return os.path.dirname(os.path.dirname(fe_path))

    return "."

def _atomic_write_df(df: pd.DataFrame, path: str) -> None:
    os.makedirs(os.path.dirname(path), exist_ok=True)
    tmp = path + ".tmp"
    df.to_csv(tmp, index=False)
    os.replace(tmp, path)


# ============================== Trainer ==============================
class TwoHeadTrainer:
    def __init__(self, seq_len: int, run_base: str):
        self.sequence_length = int(seq_len)
        self.RUN_BASE = run_base
        self.ROOT = os.path.join(run_base, "Predictive_Model")
        self.DIR_MODELS = os.path.join(self.ROOT, "lstm_models")
        self.DIR_PRED   = os.path.join(self.ROOT, "predictions")
        self.DIR_FCAST  = os.path.join(self.ROOT, "advanced_forecasts")
        self.DIR_PLOTS  = os.path.join(self.ROOT, "evaluation_plots")
        self.RESULTS_CSV= os.path.join(self.ROOT, "lstm_results.csv")
        for d in [self.ROOT, self.DIR_MODELS, self.DIR_PRED, self.DIR_FCAST, self.DIR_PLOTS]:
            os.makedirs(d, exist_ok=True)

        self.models: Dict[str, tf.keras.Model] = {}
        self.scalers: Dict[str, Dict[str, Any]] = {}
        self.results: Dict[str, Any] = {}

    # -------------------------- helpers --------------------------
    def _next_business_day(self, d):
        d = pd.Timestamp(d)
        nxt = d + pd.Timedelta(days=1)
        while nxt.weekday() >= 5:
            nxt += pd.Timedelta(days=1)
        return nxt

    # ----------------------- feature builder ----------------------
    def build_features(self, df: pd.DataFrame):
        data = df.copy()

        roll50 = data["Close"].rolling(50)
        data["Price_Norm"] = (data["Close"] - roll50.mean()) / roll50.std()
        data["High_Low_Ratio"]  = data["High"] / data["Low"].replace(0, np.nan)
        data["Open_Close_Ratio"] = data["Open"] / data["Close"].replace(0, np.nan)

        for w in [5, 10, 20]:
            ma = data["Close"].rolling(w).mean()
            data[f"MA_{w}"] = ma
            data[f"Close_MA{w}_Ratio"] = data["Close"] / ma.replace(0, np.nan)
            data[f"MA{w}_Slope"] = ma.diff(5) / ma.shift(5).replace(0, np.nan)

        for p in [1, 3, 5]:
            prev = data["Close"].shift(p).replace(0, np.nan)
            data[f"Log_Return_{p}d"] = np.log(data["Close"] / prev)
            data[f"Return_Volatility_{p}d"] = data[f"Log_Return_{p}d"].rolling(10).std()

        delta = data["Close"].diff()
        gain = delta.clip(lower=0).rolling(14).mean()
        loss = (-delta.clip(upper=0)).rolling(14).mean().replace(0, np.nan)
        rs = gain / loss
        data["RSI"] = 100 - (100 / (1 + rs))
        data["RSI_Norm"] = (data["RSI"] - 50) / 50

        exp1 = data["Close"].ewm(span=12, adjust=False).mean()
        exp2 = data["Close"].ewm(span=26, adjust=False).mean()
        data["MACD"] = exp1 - exp2
        data["MACD_Signal"] = data["MACD"].ewm(span=9, adjust=False).mean()
        data["MACD_Histogram"] = data["MACD"] - data["MACD_Signal"]
        data["MACD_Norm"] = data["MACD"] / data["Close"].replace(0, np.nan)

        vol_ma = data["Volume"].rolling(20).mean().replace(0, np.nan)
        data["Volume_MA"] = vol_ma
        data["Volume_Ratio"] = data["Volume"] / vol_ma
        data["Volume_Price_Correlation"] = data["Volume"].rolling(20).corr(data["Close"])

        data["Price_Volatility"] = data["Log_Return_1d"].rolling(20).std()
        data["High_Low_Volatility"] = np.log((data["High"] / data["Low"]).replace(0, np.nan)).rolling(10).mean()

        roll_min = data["Close"].rolling(20).min()
        roll_max = data["Close"].rolling(20).max()
        den = (roll_max - roll_min).replace(0, np.nan)
        data["Price_Position"] = (data["Close"] - roll_min) / den
        data["Momentum_5"] = data["Close"] / data["Close"].shift(5).replace(0, np.nan)
        data["Momentum_10"] = data["Close"] / data["Close"].shift(10).replace(0, np.nan)

        feature_cols = [
            "Price_Norm","High_Low_Ratio","Open_Close_Ratio",
            "Close_MA5_Ratio","Close_MA10_Ratio","Close_MA20_Ratio",
            "MA5_Slope","MA10_Slope","MA20_Slope",
            "Log_Return_1d","Log_Return_3d","Log_Return_5d",
            "Return_Volatility_1d","Return_Volatility_3d",
            "RSI_Norm","MACD_Norm","MACD_Histogram",
            "Volume_Ratio","Volume_Price_Correlation",
            "Price_Volatility","High_Low_Volatility",
            "Price_Position","Momentum_5","Momentum_10",
        ]
        avail = [c for c in feature_cols if c in data.columns]
        data[avail] = (data[avail].replace([np.inf, -np.inf], np.nan)
                                 .fillna(method="ffill")
                                 .fillna(method="bfill"))
        return data, avail

    def _prepare_sequences(self, X: np.ndarray, Y: np.ndarray, seq: int):
        xs, ys = [], []
        for i in range(seq, len(X)):
            xs.append(X[i-seq:i])
            ys.append(Y[i])
        return np.array(xs), np.array(ys)

    def _build_model(self, input_shape, *,
                     arch="lstm_gru", layers=2,
                     units1=64, units2=32, units3=16,
                     dropout=0.2, batch_norm=True,
                     dense_units=16, lr=1e-3, optimizer="adam"):
        m = Sequential()
        if arch == "lstm_gru":
            m.add(LSTM(units1, return_sequences=True, input_shape=input_shape))
            if batch_norm: m.add(BatchNormalization())
            m.add(Dropout(dropout))
            m.add(GRU(units2, return_sequences=(layers > 2)))
            if batch_norm: m.add(BatchNormalization())
            m.add(Dropout(dropout))
            if layers >= 3:
                m.add(GRU(units3))
                if batch_norm: m.add(BatchNormalization())
                m.add(Dropout(dropout))
        else:
            m.add(GRU(units1, return_sequences=(layers > 1), input_shape=input_shape))
            if batch_norm: m.add(BatchNormalization())
            m.add(Dropout(dropout))
            if layers >= 2:
                m.add(LSTM(units2, return_sequences=(layers > 2)))
                if batch_norm: m.add(BatchNormalization())
                m.add(Dropout(dropout))
            if layers >= 3:
                m.add(LSTM(units3))
                if batch_norm: m.add(BatchNormalization())
                m.add(Dropout(dropout))
        if dense_units > 0:
            m.add(Dense(dense_units, activation="relu"))
            m.add(Dropout(dropout))
        m.add(Dense(2, activation="linear"))
        opt = Adam(learning_rate=lr) if optimizer == "adam" else RMSprop(learning_rate=lr)
        m.compile(optimizer=opt, loss="mse", metrics=["mae"])
        return m

    def _tiny_search(self, Xtr, Ytr, Xva, Yva, input_shape):
        grid = [
            dict(arch="lstm_gru", layers=2, units1=64, units2=32, dropout=0.2, batch_norm=True,  dense_units=16, lr=1e-3, optimizer="adam"),
            dict(arch="lstm_gru", layers=2, units1=64, units2=32, dropout=0.3, batch_norm=True,  dense_units=16, lr=1e-3, optimizer="adam"),
            dict(arch="lstm_gru", layers=3, units1=64, units2=32, dropout=0.2, batch_norm=True,  dense_units=16, lr=5e-4, optimizer="adam"),
        ]
        best = {"val_loss": np.inf, "model": None, "config": None, "history": None}
        for i, cfg in enumerate(grid, 1):
            m = self._build_model(input_shape, **cfg)
            cbs = [
                EarlyStopping(monitor="val_loss", patience=8, restore_best_weights=True),
                ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=4, min_lr=1e-6),
            ]
            h = m.fit(Xtr, Ytr, validation_data=(Xva, Yva),
                      epochs=60, batch_size=min(64, max(8, len(Xtr)//4)),
                      verbose=0, callbacks=cbs)
            v = float(np.min(h.history["val_loss"]))
            print(f"   tried {i}/{len(grid)} -> val_loss={v:.6f}")
            if v < best["val_loss"]:
                best = {"val_loss": v, "model": m, "config": cfg, "history": h}
        print(f"   best config: {best['config']} (val_loss={best['val_loss']:.6f})")
        return best

    # -------------------------- train one ticker --------------------------
    def train_model(self, ticker: str, fe_path: str):
        print(f"\n Training model for {ticker}...")
        try:
            allfe = pd.read_csv(fe_path)
            tdf = allfe[allfe["Symbol"] == ticker].copy().sort_values("Date").reset_index(drop=True)
            if len(tdf) < 80:
                print(f"   Not enough rows for {ticker}")
                return None

            fe, feats = self.build_features(tdf)
            prev_close = fe["Close"].shift(1)
            fe["Open_Ret"]  = fe["Open"]/prev_close - 1.0
            fe["Close_Ret"] = fe["Close"]/prev_close - 1.0

            cols_needed = feats + ["Open_Ret", "Close_Ret", "Date", "Open", "Close"]
            fe = fe.dropna(subset=cols_needed).reset_index(drop=True)

            dates = pd.to_datetime(fe["Date"]).values
            X = fe[feats].values
            Y = fe[["Open_Ret","Close_Ret"]].values

            N = len(X); trn = int(0.70*N); val = int(0.15*N)
            Xtr, Xva, Xte = X[:trn], X[trn:trn+val], X[trn+val:]
            Ytr, Yva, Yte = Y[:trn], Y[trn:trn+val], Y[trn+val:]
            dates_te = dates[trn+val:]

            fs = StandardScaler()
            Xtr_s = fs.fit_transform(pd.DataFrame(Xtr, columns=feats))
            Xva_s = fs.transform(pd.DataFrame(Xva, columns=feats))
            Xte_s = fs.transform(pd.DataFrame(Xte, columns=feats))

            ys = StandardScaler()
            Ytr_s = ys.fit_transform(Ytr)
            Yva_s = ys.transform(Yva)
            Yte_s = ys.transform(Yte)

            seq = self.sequence_length
            Xtr_seq, Ytr_seq = self._prepare_sequences(Xtr_s, Ytr_s, seq)
            Xva_seq, Yva_seq = self._prepare_sequences(Xva_s, Yva_s, seq)
            Xte_seq, Yte_seq = self._prepare_sequences(Xte_s, Yte_s, seq)
            print(f"   Sequences - Train: {len(Xtr_seq)}, Val: {len(Xva_seq)}, Test: {len(Xte_seq)}")

            input_shape = (seq, len(feats))
            best = self._tiny_search(Xtr_seq, Ytr_seq, Xva_seq, Yva_seq, input_shape)
            model = best["model"]; hist = best["history"]

            tr_pred = ys.inverse_transform(model.predict(Xtr_seq, verbose=0))
            va_pred = ys.inverse_transform(model.predict(Xva_seq, verbose=0))
            te_pred = ys.inverse_transform(model.predict(Xte_seq, verbose=0))

            tr_true = Ytr[seq:]
            va_true = Yva[seq:]
            te_true = Yte[seq:]

            def head_metrics(y_true, y_pred, head=0):
                if len(y_true) < 2:
                    return 0.0, float("nan"), float("nan")
                r2  = r2_score(y_true[:, head], y_pred[:, head])
                rmse= float(np.sqrt(mean_squared_error(y_true[:, head], y_pred[:, head])))
                dir = float(np.mean((y_true[:, head] > 0) == (y_pred[:, head] > 0))*100.0)
                return r2, rmse, dir

            m_tr_open = head_metrics(tr_true, tr_pred, 0)
            m_tr_close= head_metrics(tr_true, tr_pred, 1)
            m_va_open = head_metrics(va_true, va_pred, 0)
            m_va_close= head_metrics(va_true, va_pred, 1)
            m_te_open = head_metrics(te_true, te_pred, 0)
            m_te_close= head_metrics(te_true, te_pred, 1)

            print("   Results (Open/Close heads):")
            print(f"      Train Open : R²={m_tr_open[0]:.4f} RMSE={m_tr_open[1]:.4f} DirAcc={m_tr_open[2]:.1f}%")
            print(f"      Train Close: R²={m_tr_close[0]:.4f} RMSE={m_tr_close[1]:.4f} DirAcc={m_tr_close[2]:.1f}%")
            print(f"      Val   Open : R²={m_va_open[0]:.4f} RMSE={m_va_open[1]:.4f} DirAcc={m_va_open[2]:.1f}%")
            print(f"      Val   Close: R²={m_va_close[0]:.4f} RMSE={m_va_close[1]:.4f} DirAcc={m_va_close[2]:.1f}%")
            print(f"      Test  Open : R²={m_te_open[0]:.4f} RMSE={m_te_open[1]:.4f} DirAcc={m_te_open[2]:.1f}%")
            print(f"      Test  Close: R²={m_te_close[0]:.4f} RMSE={m_te_close[1]:.4f} DirAcc={m_te_close[2]:.1f}%")

            # Align for price reconstruction
            L = len(te_pred)
            base_close = fe["Close"].iloc[trn+val+seq-1 : trn+val+seq-1 + (L+1)].values
            L = min(L, len(base_close)-1)
            te_pred = te_pred[:L]; te_true = te_true[:L]
            dates_te_adj = pd.to_datetime(dates_te[seq: seq+L])

            pred_close_price = base_close[:-1] * (1.0 + te_pred[:, 1])
            actual_close_price = base_close[1:]
            pred_open_price = base_close[:-1] * (1.0 + te_pred[:, 0])
            actual_open_price = fe["Open"].iloc[trn+val+seq : trn+val+seq+L].values

            out_pred = pd.DataFrame({
                "Date": dates_te_adj.astype("datetime64[ns]").astype(str),
                "Actual_Open":  actual_open_price,
                "Pred_Open":    pred_open_price,
                "Actual_Close": actual_close_price,
                "Pred_Close":   pred_close_price,
                "Actual_Return":  te_true[:,1],
                "Pred_Return":    te_pred[:,1]
            })
            _atomic_write_df(out_pred, os.path.join(self.DIR_PRED, f"{ticker}_test_predictions.csv"))

            self.models[ticker] = model
            self.scalers[ticker] = {"feature_scaler": fs, "target_scaler": ys, "features": feats}
            self.results[ticker] = {
                "train_open": m_tr_open, "train_close": m_tr_close,
                "val_open": m_va_open, "val_close": m_va_close,
                "test_open": m_te_open, "test_close": m_te_close,
                "best_config": best["config"], "history": hist.history,
            }

            # Save generated outputs
            model.save(os.path.join(self.DIR_MODELS, f"{ticker}_twohead.keras"))
            joblib.dump(fs, os.path.join(self.DIR_MODELS, f"{ticker}_feature_scaler.pkl"))
            joblib.dump(ys, os.path.join(self.DIR_MODELS, f"{ticker}_target_scaler.pkl"))
            with open(os.path.join(self.DIR_MODELS, f"{ticker}_feature_names.json"), "w") as f:
                json.dump(feats, f)
            with open(os.path.join(self.DIR_MODELS, f"{ticker}_seq_len.txt"), "w") as f:
                f.write(str(self.sequence_length))
            with open(os.path.join(self.DIR_MODELS, f"{ticker}_best_config.json"), "w") as f:
                json.dump(best["config"], f, indent=2)

            # Plot
            plt.figure(figsize=(12,5))
            plt.plot(dates_te_adj, actual_close_price, label="Actual Close")
            plt.plot(dates_te_adj, pred_close_price, label="Pred Close")
            plt.title(f"{ticker} — Test Close: Actual vs Predicted")
            plt.legend(); plt.grid(alpha=0.3); plt.tight_layout()
            plt.savefig(os.path.join(self.DIR_PLOTS, f"{ticker}_test_close.png"), dpi=220)
            plt.close()

            # Tomorrow-only prediction
            fe["Date"] = pd.to_datetime(fe["Date"])
            latest_date = fe["Date"].iloc[-1]
            latest_close = float(fe["Close"].iloc[-1])
            latest_open  = float(fe["Open"].iloc[-1])

            last_win = fe[feats].tail(self.sequence_length).values
            last_win_s = fs.transform(last_win).reshape(1, self.sequence_length, len(feats))
            tom_pred = ys.inverse_transform(model.predict(last_win_s, verbose=0))[0]
            pred_open_ret, pred_close_ret = float(tom_pred[0]), float(tom_pred[1])

            tom_pred_open  = latest_close * (1.0 + pred_open_ret)
            tom_pred_close = latest_close * (1.0 + pred_close_ret)

            for_date = self._next_business_day(latest_date)
            tomorrow_df = pd.DataFrame([{
                "For_Date": for_date.strftime("%Y-%m-%d"),
                "Latest_Actual_Open":  latest_open,
                "Latest_Actual_Close": latest_close,
                "Pred_Open_Return":  pred_open_ret,
                "Pred_Close_Return": pred_close_ret,
                "Pred_Open":  tom_pred_open,
                "Pred_Close": tom_pred_close
            }])
            _atomic_write_df(tomorrow_df, os.path.join(self.DIR_PRED, f"{ticker}_tomorrow.csv"))
            print(f"   Tomorrow-only prediction saved → {os.path.join(self.DIR_PRED, f'{ticker}_tomorrow.csv')}")

            # XAI compatibility alias
            src = os.path.join(self.DIR_MODELS, f"{ticker}_twohead.keras")
            dst = os.path.join(self.DIR_MODELS, f"{ticker}_lstm.keras")
            try:
                shutil.copyfile(src, dst)
            except Exception:
                pass

            return self.results[ticker]

        except Exception as e:
            print(f"    Error training {ticker}: {e}")
            import traceback; traceback.print_exc()
            return None

    def forecast_recursive(self, ticker: str, fe_path: str, days: int = 7):
        if ticker not in self.models:
            print("    No trained model found. Train first.")
            return None

        allfe = pd.read_csv(fe_path)
        df = allfe[allfe["Symbol"] == ticker].copy().sort_values("Date").reset_index(drop=True)
        fe, feats = self.build_features(df)
        fe = fe.dropna(subset=feats + ["Date", "Close"]).reset_index(drop=True)
        fe["Date"] = pd.to_datetime(fe["Date"])
        model = self.models[ticker]
        fs = self.scalers[ticker]["feature_scaler"]
        ys = self.scalers[ticker]["target_scaler"]

        seq = fs.transform(fe[feats].tail(self.sequence_length).values)

        try:
            lr1_idx = feats.index("Log_Return_1d")
        except ValueError:
            lr1_idx = None

        current_close = float(fe["Close"].iloc[-1])
        start_date = fe["Date"].iloc[-1]
        dates, pred_open, pred_close, ret_open, ret_close = [], [], [], [], []

        d = self._next_business_day(start_date)
        for _ in range(days):
            x = seq.reshape(1, self.sequence_length, len(feats))
            pred = ys.inverse_transform(model.predict(x, verbose=0))[0]
            oret, cret = float(pred[0]), float(pred[1])

            next_open  = current_close * (1.0 + oret)
            next_close = current_close * (1.0 + cret)

            dates.append(d); pred_open.append(next_open); pred_close.append(next_close)
            ret_open.append(oret); ret_close.append(cret)

            new_row = seq[-1].copy()
            if lr1_idx is not None:
                mu = getattr(fs, "mean_", None); sc = getattr(fs, "scale_", None)
                raw_lr1 = np.log(1.0 + cret)
                if mu is not None and sc is not None and sc[lr1_idx] != 0:
                    new_row[lr1_idx] = (raw_lr1 - mu[lr1_idx]) / sc[lr1_idx]
            seq = np.vstack([seq[1:], new_row])
            current_close = next_close
            d = self._next_business_day(d)

        out = pd.DataFrame({
            "Date": pd.to_datetime(dates).strftime("%Y-%m-%d"),
            "Pred_Open_Return":  ret_open,
            "Pred_Close_Return": ret_close,
            "Pred_Open": pred_open,
            "Pred_Close": pred_close
        })
        out_path = os.path.join(self.DIR_FCAST, f"{ticker}_forecast_{days}d.csv")
        _atomic_write_df(out, out_path)
        print(f"   {days}-day forecast saved → {out_path}")

        # Plot
        tailN = 60
        tail_dates = fe["Date"].tail(tailN).tolist()
        tail_prices= fe["Close"].tail(tailN).tolist()
        plt.figure(figsize=(12,5))
        plt.plot(tail_dates, tail_prices, label="Actual Close (last 60d)")
        plt.plot(pd.to_datetime(dates), pred_close, label=f"Forecast Close (+{days}d)")
        plt.title(f"{ticker} — Last 60d Close + {days}-Day Forecast")
        plt.legend(); plt.grid(alpha=0.3); plt.tight_layout()
        plt.savefig(os.path.join(self.DIR_PLOTS, f"{ticker}_forecast_close_{days}d.png"), dpi=220)
        plt.close()
        return out

    def save_results_summary(self):
        if not self.results:
            print(" No results to report")
            return None
        rows = []
        for t, r in self.results.items():
            te_o, te_c = r["test_open"], r["test_close"]
            rows.append({
                "Ticker": t,
                "Test_R2_Open":  round(te_o[0], 4),
                "Test_RMSE_Open":round(te_o[1], 4),
                "Test_DirAcc_Open(%)": round(te_o[2], 1),
                "Test_R2_Close":  round(te_c[0], 4),
                "Test_RMSE_Close":round(te_c[1], 4),
                "Test_DirAcc_Close(%)": round(te_c[2], 1),
                "Chosen": "lstm_gru",
            })
        df = pd.DataFrame(rows)
        _atomic_write_df(df, self.RESULTS_CSV)
        print(f" Results summary saved → {self.RESULTS_CSV}")
        return df


# ============================== LangGraph nodes ==============================
def node_init(state: Dict[str, Any]) -> Dict[str, Any]:
    run_base = _infer_run_base(state)
    seq_len  = int(state.get("seq_len", 20))
    tickers  = state.get("tickers") or ["AAPL"]
    days     = int(state.get("forecast_days", 7))
    fe_path  = state.get("fe_path") or os.path.join(run_base, "FE_Agent", "features_engineered.csv")
    if not os.path.exists(fe_path):
        raise FileNotFoundError(f"FE file not found: {fe_path}")
    trainer = TwoHeadTrainer(seq_len=seq_len, run_base=run_base)
    return {**state, "run_base": run_base, "fe_path": fe_path, "trainer": trainer,
            "tickers": tickers, "forecast_days": days, "status": "init_ok"}

def node_train_all(state: Dict[str, Any]) -> Dict[str, Any]:
    tr, fe_path = state["trainer"], state["fe_path"]
    trained = []
    for t in state["tickers"]:
        res = tr.train_model(t, fe_path)
        if res is not None:
            trained.append(t)
    return {**state, "trained": trained, "status": "trained"}

def node_forecast_all(state: Dict[str, Any]) -> Dict[str, Any]:
    tr, fe_path = state["trainer"], state["fe_path"]
    days, tickers = state["forecast_days"], state.get("trained", [])
    for t in tickers:
        tr.forecast_recursive(t, fe_path, days=days)
    return {**state, "status": "forecast_done"}

def node_save_summary(state: Dict[str, Any]) -> Dict[str, Any]:
    tr = state["trainer"]
    df = tr.save_results_summary()
    return {**state, "results_csv": getattr(tr, "RESULTS_CSV", None), "status": "complete"}

def build_predictive_workflow():
    g = StateGraph(dict)
    g.add_node("init", node_init)
    g.add_node("train", node_train_all)
    g.add_node("forecast", node_forecast_all)
    g.add_node("save", node_save_summary)
    g.set_entry_point("init")
    g.add_edge("init", "train")
    g.add_edge("train", "forecast")
    g.add_edge("forecast", "save")
    g.set_finish_point("save")
    return g.compile()


def run_predict(state: Dict[str, Any]) -> Dict[str, Any]:
    s = node_init(state)
    s = node_train_all(s)
    s = node_forecast_all(s)
    s = node_save_summary(s)

    tr: TwoHeadTrainer = s["trainer"]
    return {
        **state,
        "run_base": s["run_base"],
        "predict_root": tr.ROOT,
        "models_dir": tr.DIR_MODELS,
        "pred_dir": tr.DIR_PRED,
        "forecast_dir": tr.DIR_FCAST,
        "results_csv": tr.RESULTS_CSV,
        "trained": s.get("trained", []),
        "status": "predict_complete",
    }

if __name__ == "__main__":
    print(" Starting Predictive Agent…\n")
    app = build_predictive_workflow()
    _ = app.invoke({

        # "fe_path": "<run_base>/FE_Agent/features_engineered.csv",
        "seq_len": 20,
        "forecast_days": 7,
    })
    print("\n Done. Artifacts saved under <run_base>/Predictive_Model/")


**Model_evaluation_agent.py**

In [ ]:
%%writefile /content/drive/MyDrive/A2A_prediction_system/backend/a2a/model_evaluation_agent.py
import os, json
from typing import Dict, Any, List, Optional
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shutil


# LangGraph
try:
    from langgraph.graph import StateGraph
    HAS_LG = True
except Exception:
    HAS_LG = False

# ----------------- IO utils -----------------
def _atomic_write_df(df: pd.DataFrame, path: str) -> None:
    os.makedirs(os.path.dirname(path), exist_ok=True)
    tmp = path + ".tmp"
    df.to_csv(tmp, index=False)
    os.replace(tmp, path)

def _infer_run_base(state: Dict[str, Any]) -> str:
    if state.get("run_base"):
        return str(state["run_base"])
    pred_dir = state.get("predictions_dir")
    if pred_dir:
        # expected path .../<run_base>/Predictive_Model/predictions
        return os.path.dirname(os.path.dirname(pred_dir))
    return "."

# ----------------- Normalizer -----------------
def normalize_pred_file(df: pd.DataFrame, head: str = "close") -> pd.DataFrame:
    """
    Standardize to columns: Date, Actual_Return, Pred_Return, Actual_Close, Pred_Close.
    """
    required_single = {"Actual_Return", "Pred_Return", "Actual_Close", "Pred_Close"}
    if required_single.issubset(df.columns):
        return df

    head = (head or "close").lower().strip()
    if head not in {"close", "open"}:
        head = "close"

    if head == "close":
        req = {"Actual_Return", "Pred_Return", "Actual_Close", "Pred_Close"}
        # Predictive file already uses these names for close
        mapped = {"Actual_Close_Return": "Actual_Return", "Pred_Close_Return": "Pred_Return"}
        if req.issubset(df.columns):
            return df
        if set(mapped.keys()).issubset(df.columns) and {"Actual_Close","Pred_Close"}.issubset(df.columns):
            df2 = df.copy()
            df2.rename(columns=mapped, inplace=True)
            return df2

    if head == "open":
        req = {"Actual_Open", "Pred_Open"}
        ret = {"Actual_Open_Return": "Actual_Return", "Pred_Open_Return": "Pred_Return"}
        if req.issubset(df.columns) and set(ret.keys()).issubset(df.columns):
            df2 = df.copy()
            df2.rename(columns=ret, inplace=True)
            # For generic naming, map open->close slots so metrics/plots reuse code
            df2["Actual_Close"] = df2["Actual_Open"]
            df2["Pred_Close"]   = df2["Pred_Open"]
            return df2

    raise ValueError(
        "Prediction file missing required columns. "
        "Expected either {Actual_Return, Pred_Return, Actual_Close, Pred_Close} "
        "or open-head equivalents."
    )

# ----------------- Metrics  -----------------
def _price_metrics(a_p: np.ndarray, p_p: np.ndarray) -> Dict[str, float]:
    if len(a_p) <= 1 or len(p_p) <= 1:
        return {"RMSE_price": np.nan, "MAE_price": np.nan, "MAPE": np.nan, "R2(price)": np.nan}
    mse  = float(np.mean((a_p - p_p) ** 2))
    rmse = float(np.sqrt(mse))
    mae  = float(np.mean(np.abs(a_p - p_p)))
    r2   = float(r2_score(a_p, p_p))
    mape = float(pd.Series(a_p).pipe(lambda s: np.mean(np.abs((a_p - p_p) / np.where(s != 0, s, np.nan)))) * 100.0)
    return {"RMSE_price": rmse, "MAE_price": mae, "MAPE": mape, "R2(price)": r2}

from sklearn.metrics import r2_score, mean_squared_error

def eval_metrics(df: pd.DataFrame) -> dict:
    if df is None or df.empty:
        return {k: (np.nan) for k in
                ["RMSE_price","MAE_price","MAPE","R2(price)","RMSE_ret","R2(returns)","DirAcc(%)","Sharpe_ann",
                 "Rows","Test_Start","Test_End"]}

    if "Date" in df.columns:
        d = pd.to_datetime(df["Date"], errors="coerce")
        test_start, test_end = d.min(), d.max()
    else:
        test_start = test_end = None

    a_r = df["Actual_Return"].to_numpy()
    p_r = df["Pred_Return"].to_numpy()
    a_p = df["Actual_Close"].to_numpy()
    p_p = df["Pred_Close"].to_numpy()

    # price metrics
    price = _price_metrics(a_p, p_p)

    # returns metrics
    if len(a_r) > 1:
        rmse_ret = float(np.sqrt(mean_squared_error(a_r, p_r)))
        r2_ret   = float(r2_score(a_r, p_r))
        diracc   = float(np.mean(np.sign(a_r) == np.sign(p_r)) * 100.0)
        ex_ret   = (p_r - a_r)
        sharpe_d = float(np.mean(ex_ret) / np.std(ex_ret)) if np.std(ex_ret) > 0 else np.nan
        sharpe_a = float(sharpe_d * np.sqrt(252)) if not np.isnan(sharpe_d) else np.nan
    else:
        rmse_ret = r2_ret = diracc = sharpe_a = np.nan

    out = {
        **{k: (round(v, 6) if isinstance(v, float) and not np.isnan(v) else (v if not isinstance(v, float) else np.nan))
           for k, v in price.items()},
        "RMSE_ret": round(rmse_ret, 6) if not np.isnan(rmse_ret) else np.nan,
        "R2(returns)": round(r2_ret, 4) if not np.isnan(r2_ret) else np.nan,
        "DirAcc(%)": round(diracc, 1) if not np.isnan(diracc) else np.nan,
        "Sharpe_ann": round(sharpe_a, 3) if not np.isnan(sharpe_a) else np.nan,
        "Rows": int(len(df)),
        "Test_Start": "" if test_start is None else str(getattr(test_start, "date", lambda: "")()),
        "Test_End": "" if test_end is None else str(getattr(test_end, "date", lambda: "")()),
    }
    return out

# ----------------- Baselines -----------------
def eval_baselines(df: pd.DataFrame) -> Dict[str, float]:
    """
    Baseline A (returns): predict zero return.
    Baseline B (price): carry-forward yesterday's actual close.
    """
    out = {"BL_DirAcc(%)": np.nan, "BL_RMSE_ret": np.nan, "BL_RMSE_price": np.nan, "BL_R2(price)": np.nan}
    if df.empty:
        return out

    # A: returns baseline (0)
    a_r = df["Actual_Return"].to_numpy()
    p0_r = np.zeros_like(a_r)
    if len(a_r) > 1:
        out["BL_RMSE_ret"] = float(np.sqrt(mean_squared_error(a_r, p0_r)))
        out["BL_DirAcc(%)"] = float(np.mean(np.sign(a_r) == np.sign(p0_r)) * 100.0)

    # B: price baseline (carry-forward)
    a_p = df["Actual_Close"].to_numpy()
    if len(a_p) > 2:
        p0_p = np.r_[a_p[0], a_p[:-1]]  # yesterday's price
        pm = _price_metrics(a_p, p0_p)
        out["BL_RMSE_price"] = pm["RMSE_price"]
        out["BL_R2(price)"]  = pm["R2(price)"]

    return out

# ========================= LangGraph NODES =========================
def node_init(state: Dict[str, Any]) -> Dict[str, Any]:
    """
    Inputs:
      run_base: str
      predictions_dir: str   (defaults to <run_base>/Predictive_Model/predictions)
      eval_root: str         (defaults to <run_base>/Model_Evaluation)
      eval_head: "close"|"open" (default "close")
      strong_pct: int        (default 75)  # percentile on |Pred_Return|
    """
    run_base = _infer_run_base(state)
    predictions_dir = state.get("predictions_dir") or os.path.join(run_base, "Predictive_Model", "predictions")
    eval_root = state.get("eval_root") or os.path.join(run_base, "Model_Evaluation")
    eval_head = (state.get("eval_head", "close") or "close").lower()
    strong_pct = int(state.get("strong_pct", 75))

    if not os.path.isdir(predictions_dir):
        raise FileNotFoundError(f"Predictions folder not found: {predictions_dir}")

    sum_dir = os.path.join(eval_root, "summaries")
    plot_dir = os.path.join(eval_root, "plots")
    os.makedirs(sum_dir, exist_ok=True)
    os.makedirs(plot_dir, exist_ok=True)

    paths = {
        "CSV_FULL":    os.path.join(sum_dir,  "model_evaluation_summary.csv"),
        "JSON_FULL":   os.path.join(sum_dir,  "model_evaluation_summary.json"),
        "CSV_STRONG":  os.path.join(sum_dir,  "model_evaluation_strong_signals.csv"),
        "JSON_STRONG": os.path.join(sum_dir,  "model_evaluation_strong_signals.json"),
        "DIRACC_PNG":  os.path.join(plot_dir, "diracc_full_vs_strong.png"),
        "SCATTER_PNG": lambda ticker: os.path.join(plot_dir, f"{ticker}_strong_signal_pred_vs_actual.png"),
    }

    return {
        **state,
        "run_base": run_base,
        "predictions_dir": predictions_dir,
        "eval_root": eval_root,
        "sum_dir": sum_dir,
        "plot_dir": plot_dir,
        "paths": paths,
        "eval_head": "open" if eval_head == "open" else "close",
        "strong_pct": strong_pct,
        "status": "init_ok",
    }

def node_discover_files(state: Dict[str, Any]) -> Dict[str, Any]:
    files = [fn for fn in sorted(os.listdir(state["predictions_dir"])) if fn.endswith("_test_predictions.csv")]
    tickers = [fn.split("_")[0] for fn in files]
    return {**state, "files": files, "tickers": tickers, "status": "files_ok"}

def node_evaluate(state: Dict[str, Any]) -> Dict[str, Any]:
    head = state["eval_head"]
    pred_dir = state["predictions_dir"]

    full_rows: List[dict] = []
    strong_rows: List[dict] = []

    print(f" Model evaluation (head='{head}') — price & return metrics …\n")
    for fname in state["files"]:
        ticker = fname.split("_")[0]
        path = os.path.join(pred_dir, fname)
        try:
            raw = pd.read_csv(path)
            df = normalize_pred_file(raw, head=head)

            need = {"Actual_Return", "Pred_Return", "Actual_Close", "Pred_Close"}
            if df.empty or not need.issubset(df.columns):
                print(f"  {ticker}: normalized file missing columns — skip.")
                continue

            # Full-sample metrics
            fm = eval_metrics(df)
            bl = eval_baselines(df)
            fm.update(bl)
            fm["Ticker"] = ticker
            fm["Head"] = head
            full_rows.append(fm)

            print(f" {ticker}: R2(price)={fm['R2(price)']} | RMSEp={fm['RMSE_price']} | DirAcc={fm['DirAcc(%)']}%  "
                  f"| BL DirAcc={fm.get('BL_DirAcc(%)')}%")

            # Strong-signal subset (by |Pred_Return|)
            try:
                thr = df["Pred_Return"].abs().quantile(state["strong_pct"]/100.0)
                strong = df.loc[df["Pred_Return"].abs() >= thr].copy()
                if not strong.empty:
                    sm = eval_metrics(strong)
                    sm["Ticker"] = ticker
                    sm["Head"] = head
                    sm["Subset"] = f"|Pred_Return|>={state['strong_pct']}pct"
                    strong_rows.append(sm)
            except Exception:
                pass

        except Exception as e:
            print(f"  {ticker}: error {e}")

    return {**state, "full_rows": full_rows, "strong_rows": strong_rows, "status": "evaluated"}

def node_save_tables(state: Dict[str, Any]) -> Dict[str, Any]:
    paths = state["paths"]
    full_rows = state.get("full_rows", [])
    strong_rows = state.get("strong_rows", [])

    df_full = pd.DataFrame(full_rows).sort_values(["Ticker", "Head"]) if full_rows else pd.DataFrame()
    df_str  = pd.DataFrame(strong_rows).sort_values(["Ticker", "Head"]) if strong_rows else pd.DataFrame()

    if not df_full.empty:
        _atomic_write_df(df_full, paths["CSV_FULL"])
        with open(paths["JSON_FULL"], "w") as f:
            json.dump(full_rows, f, indent=2)

    if not df_str.empty:
        _atomic_write_df(df_str, paths["CSV_STRONG"])
        with open(paths["JSON_STRONG"], "w") as f:
            json.dump(strong_rows, f, indent=2)

    return {**state, "df_full": df_full, "df_strong": df_str, "status": "tables_saved"}

def node_make_plots(state: Dict[str, Any]) -> Dict[str, Any]:
    df_full = state.get("df_full", pd.DataFrame())
    paths = state["paths"]
    head = state["eval_head"]
    pred_dir = state["predictions_dir"]

    # Per-ticker plots from prediction files
    files = [fn for fn in sorted(os.listdir(pred_dir)) if fn.endswith("_test_predictions.csv")]
    for fname in files:
        ticker = fname.split("_")[0]
        path = os.path.join(pred_dir, fname)
        try:
            raw = pd.read_csv(path)
            df = normalize_pred_file(raw, head=head)

            # Sort by date
            if "Date" in df.columns:
                try:
                    df["Date"] = pd.to_datetime(df["Date"])
                    df = df.sort_values("Date")
                    x = df["Date"]
                    x_label = "Date"
                except Exception:
                    x = np.arange(len(df)); x_label = "Index"
            else:
                x = np.arange(len(df)); x_label = "Index"

            # Line plot: Actual vs Predicted Price
            plt.figure(figsize=(12, 5))
            plt.plot(x, df["Actual_Close"], label="Actual Price", linewidth=1.5)
            plt.plot(x, df["Pred_Close"],   label="Predicted Price", linewidth=1.5)
            plt.title(f"{ticker} — {head.capitalize()} Price: Actual vs Predicted")
            plt.xlabel(x_label); plt.ylabel("Price"); plt.legend(); plt.grid(alpha=0.3)
            out_line = os.path.join(state["plot_dir"], f"{ticker}_{head}_price_line.png")
            plt.tight_layout(); plt.savefig(out_line, dpi=300); plt.close()

            # Scatter: Predicted vs Actual Price
            plt.figure(figsize=(6.5, 6))
            plt.scatter(df["Actual_Close"], df["Pred_Close"], alpha=0.6)
            lo = float(min(df["Actual_Close"].min(), df["Pred_Close"].min()))
            hi = float(max(df["Actual_Close"].max(), df["Pred_Close"].max()))
            plt.plot([lo, hi], [lo, hi], "r--", linewidth=1)
            plt.title(f"{ticker} — {head.capitalize()} Price: Predicted vs Actual")
            plt.xlabel("Actual Price"); plt.ylabel("Predicted Price"); plt.grid(alpha=0.3)
            out_scatter = os.path.join(state["plot_dir"], f"{ticker}_{head}_price_scatter.png")
            plt.tight_layout(); plt.savefig(out_scatter, dpi=300); plt.close()

            # Copy to the Predictive_Model/evaluation_plots folder
            legacy_dir = os.path.join(state["run_base"], "Predictive_Model", "evaluation_plots")
            os.makedirs(legacy_dir, exist_ok=True)
            shutil.copyfile(out_line,    os.path.join(legacy_dir, f"{ticker}_{head}_price_line.png"))
            shutil.copyfile(out_scatter, os.path.join(legacy_dir, f"{ticker}_{head}_price_scatter.png"))

        except Exception as e:
            print(f"  {ticker}: plotting error: {e}")

    # Aggregate DirAcc bar
    df_str = state.get("df_strong", pd.DataFrame())
    if not df_full.empty and not df_str.empty:
        tickers = sorted(set(df_full["Ticker"]).intersection(set(df_str["Ticker"])))
        if tickers:
            full_da = [float(df_full[df_full["Ticker"]==t]["DirAcc(%)"].values[0]) for t in tickers]
            str_da  = [float(df_str[df_str["Ticker"]==t]["DirAcc(%)"].values[0]) for t in tickers]
            x = np.arange(len(tickers))
            plt.figure(figsize=(10,5))
            plt.bar(x - 0.2, full_da, width=0.4, label="Full")
            plt.bar(x + 0.2, str_da,  width=0.4, label="Strong")
            plt.xticks(x, tickers)
            plt.ylabel("Direction Accuracy (%)")
            plt.title("DirAcc: Full vs Strong-signal subset")
            plt.legend(); plt.grid(axis="y", alpha=0.3)
            plt.tight_layout(); plt.savefig(paths["DIRACC_PNG"], dpi=300); plt.close()

    print("\nSaved plots in:", state["plot_dir"])
    return {**state, "status": "plots_saved"}

# ========================= Build + run LangGraph =========================
def build_evaluation_workflow():
    g = StateGraph(dict)
    g.add_node("init", node_init)
    g.add_node("discover", node_discover_files)
    g.add_node("evaluate", node_evaluate)
    g.add_node("tables", node_save_tables)
    g.add_node("plots", node_make_plots)

    g.set_entry_point("init")
    g.add_edge("init", "discover")
    g.add_edge("discover", "evaluate")
    g.add_edge("evaluate", "tables")
    g.add_edge("tables", "plots")
    g.set_finish_point("plots")
    return g.compile()

# -------- Simple function for orchestrator --------
def run_evaluate(state: Dict[str, Any]) -> Dict[str, Any]:
    s = node_init(state)
    s = node_discover_files(s)
    s = node_evaluate(s)
    s = node_save_tables(s)
    s = node_make_plots(s)
    return {
        **state,
        "run_base": s["run_base"],
        "eval_root": s["eval_root"],
        "eval_summary_csv": s["paths"]["CSV_FULL"],
        "eval_summary_json": s["paths"]["JSON_FULL"],
        "eval_strong_csv": s["paths"]["CSV_STRONG"],
        "eval_diracc_png": s["paths"]["DIRACC_PNG"],
        "status": "evaluation_complete",
    }

if __name__ == "__main__":
    print(" Running Evaluation Agent …")
    app = build_evaluation_workflow()
    _ = app.invoke({
        # Provide run_base or predictions_dir; defaults to <run_base>/Predictive_Model/predictions

        "eval_head": "close",   # or "open"
        "strong_pct": 75
    })


**Explainability_agent.py - IG + SHAP**

In [ ]:
%%writefile /content/drive/MyDrive/A2A_prediction_system/backend/a2a/explainability_agent.py
# Explainability Agent — runs IG first, then SHAP

import os, json, warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

import joblib

import tensorflow as tf
from tensorflow.keras.models import load_model

# LangGraph
try:
    from langgraph.graph import StateGraph
    HAS_LG = True
except Exception:
    HAS_LG = False

# SHAP
try:
    import shap
    HAS_SHAP = True
except Exception:
    HAS_SHAP = False

np.random.seed(42)
tf.random.set_seed(42)

# ----------------------------- path helpers -----------------------------
def _infer_paths(state):
    """
    Returns (models_dir, fe_path, explain_root, ig_root, shap_root)
    """
    run_base = state.get("run_base")
    if run_base:
        models_dir   = state.get("models_dir", os.path.join(run_base, "Predictive_Model", "lstm_models"))
        fe_path      = state.get("fe_path",   os.path.join(run_base, "FE_Agent", "features_engineered.csv"))
        explain_root = state.get("explain_root", os.path.join(run_base, "Explainability"))
    else:
        models_dir   = state.get("models_dir", "Predictive_Model/lstm_models")
        fe_path      = state.get("fe_path",   "FE_Agent/features_engineered.csv")
        explain_root = state.get("explain_root", "Explainability")

    ig_root   = os.path.join(explain_root, "IG_XAI")
    shap_root = os.path.join(explain_root, "SHAP_XAI")
    return models_dir, fe_path, explain_root, ig_root, shap_root

def _atomic_write_df(df: pd.DataFrame, path: str) -> None:
    os.makedirs(os.path.dirname(path), exist_ok=True)
    tmp = path + ".tmp"
    df.to_csv(tmp, index=False)
    os.replace(tmp, path)

# ======================================================================
# =============================== IG ===============================
# ======================================================================

def build_improved_features(ticker_df: pd.DataFrame):
    df = ticker_df.copy()
    roll50 = df["Close"].rolling(50)
    df["Price_Norm"] = (df["Close"] - roll50.mean()) / roll50.std()
    df["High_Low_Ratio"]  = df["High"] / df["Low"].replace(0, np.nan)
    df["Open_Close_Ratio"] = df["Open"] / df["Close"].replace(0, np.nan)
    for w in [5, 10, 20]:
        ma = df["Close"].rolling(w).mean()
        df[f"MA_{w}"] = ma
        df[f"Close_MA{w}_Ratio"] = df["Close"] / ma.replace(0, np.nan)
        df[f"MA{w}_Slope"] = ma.diff(5) / ma.shift(5).replace(0, np.nan)
    for p in [1, 3, 5]:
        prev = df["Close"].shift(p).replace(0, np.nan)
        df[f"Log_Return_{p}d"] = np.log(df["Close"] / prev)
        df[f"Return_Volatility_{p}d"] = df[f"Log_Return_{p}d"].rolling(10).std()
    delta = df["Close"].diff()
    gain = delta.clip(lower=0).rolling(14).mean()
    loss = (-delta.clip(upper=0)).rolling(14).mean().replace(0, np.nan)
    rs = gain / loss
    df["RSI"] = 100 - (100 / (1 + rs))
    df["RSI_Norm"] = (df["RSI"] - 50) / 50
    exp1 = df["Close"].ewm(span=12, adjust=False).mean()
    exp2 = df["Close"].ewm(span=26, adjust=False).mean()
    df["MACD"] = exp1 - exp2
    df["MACD_Signal"] = df["MACD"].ewm(span=9, adjust=False).mean()
    df["MACD_Histogram"] = df["MACD"] - df["MACD_Signal"]
    df["MACD_Norm"] = df["MACD"] / df["Close"].replace(0, np.nan)
    vol_ma = df["Volume"].rolling(20).mean().replace(0, np.nan)
    df["Volume_MA"] = vol_ma
    df["Volume_Ratio"] = df["Volume"] / vol_ma
    df["Volume_Price_Correlation"] = df["Volume"].rolling(20).corr(df["Close"])
    df["Price_Volatility"] = df["Log_Return_1d"].rolling(20).std()
    df["High_Low_Volatility"] = np.log((df["High"] / df["Low"]).replace(0, np.nan)).rolling(10).mean()
    roll_min = df["Close"].rolling(20).min()
    roll_max = df["Close"].rolling(20).max()
    den = (roll_max - roll_min).replace(0, np.nan)
    df["Price_Position"] = (df["Close"] - roll_min) / den
    df["Momentum_5"] = df["Close"] / df["Close"].shift(5).replace(0, np.nan)
    df["Momentum_10"] = df["Close"] / df["Close"].shift(10).replace(0, np.nan)

    feature_columns = [
        "Price_Norm","High_Low_Ratio","Open_Close_Ratio",
        "Close_MA5_Ratio","Close_MA10_Ratio","Close_MA20_Ratio",
        "MA5_Slope","MA10_Slope","MA20_Slope",
        "Log_Return_1d","Log_Return_3d","Log_Return_5d",
        "Return_Volatility_1d","Return_Volatility_3d",
        "RSI_Norm","MACD_Norm","MACD_Histogram",
        "Volume_Ratio","Volume_Price_Correlation",
        "Price_Volatility","High_Low_Volatility",
        "Price_Position","Momentum_5","Momentum_10",
    ]
    avail = [c for c in feature_columns if c in df.columns]
    df[avail] = df[avail].replace([np.inf, -np.inf], np.nan).ffill().bfill()
    return df, avail

def prepare_sequences_ig(features: np.ndarray, seq_len: int) -> np.ndarray:
    if len(features) <= seq_len:
        return np.empty((0, seq_len, features.shape[1]))
    return np.array([features[i - seq_len:i] for i in range(seq_len, len(features))])

def integrated_gradients_for_head(model, inputs, head_idx: int, steps: int = 50):
    inputs = tf.convert_to_tensor(inputs, dtype=tf.float32)
    baseline = tf.zeros_like(inputs)
    output_dim = int(model.output_shape[-1]) if hasattr(model, "output_shape") else 1

    @tf.function
    def f_head(x):
        y = model(x, training=False)
        if output_dim == 1:
            return y
        return y[:, head_idx:head_idx+1]

    alphas = tf.reshape(tf.linspace(0.0, 1.0, steps + 1), [-1, 1, 1, 1])
    xb = tf.expand_dims(inputs, 0)
    x0 = tf.expand_dims(baseline, 0)
    path_inputs = x0 + alphas * (xb - x0)

    with tf.GradientTape() as tape:
        tape.watch(path_inputs)
        flat = tf.reshape(path_inputs, [-1, inputs.shape[1], inputs.shape[2]])
        preds = f_head(flat)
        s = tf.reduce_sum(preds)
    grads = tape.gradient(s, path_inputs)
    avg_grads = tf.reduce_mean(grads, axis=0)
    ig = (inputs - baseline) * avg_grads
    return ig.numpy()

def ig_completeness_residual(model, x_batch, ig_vals, head_idx=1):
    x = tf.convert_to_tensor(x_batch, dtype=tf.float32)
    x0 = tf.zeros_like(x)
    def f_head(z):
        y = model(z, training=False)
        return y[:, head_idx]
    delta = f_head(x) - f_head(x0)
    ig_sum = tf.reduce_sum(ig_vals, axis=[1,2])
    resid = delta - ig_sum
    return float(tf.reduce_mean(tf.abs(resid)).numpy()), resid.numpy()

def ig_discover(state):
    models_dir = state.get("models_dir", "Predictive_Model/lstm_models")
    data_path  = state.get("data_path",  "FE_Agent/features_engineered.csv")
    out_dir    = state.get("out_dir",    "IG_XAI")
    if not os.path.isdir(models_dir):
        raise FileNotFoundError(f"Models directory not found: {models_dir}")
    if not os.path.exists(data_path):
        raise FileNotFoundError(f"Engineered features not found: {data_path}")
    os.makedirs(out_dir, exist_ok=True)

    with open(os.path.join(out_dir, "environment.json"), "w") as f:
        json.dump({
            "tensorflow_version": tf.__version__,
            "numpy_version": np.__version__,
            "pandas_version": pd.__version__,
            "matplotlib_version": matplotlib.__version__,
        }, f, indent=2)

    tickers, artifacts = [], {}
    for fname in os.listdir(models_dir):
        if fname.endswith("_twohead.keras") or fname.endswith("_lstm.keras"):
            t = fname.replace("_twohead.keras", "").replace("_lstm.keras", "")
            tickers.append(t)
    tickers = sorted(set(tickers))
    for t in tickers:
        two = os.path.join(models_dir, f"{t}_twohead.keras")
        lstm= os.path.join(models_dir, f"{t}_lstm.keras")
        artifacts[t] = {
            "model_path": two if os.path.exists(two) else lstm,
            "feature_scaler_path": os.path.join(models_dir, f"{t}_feature_scaler.pkl"),
            "feature_names_path":  os.path.join(models_dir, f"{t}_feature_names.json"),
            "seq_len_path":        os.path.join(models_dir, f"{t}_seq_len.txt"),
        }
    return {**state, "tickers": tickers, "artifacts": artifacts}

def ig_prepare(state):
    df = pd.read_csv(state["data_path"])
    if "Date" in df.columns:
        df["Date"] = pd.to_datetime(df["Date"])
    per_ticker = {}
    for t in state["tickers"]:
        tdf = df[df["Symbol"] == t].copy().sort_values("Date").reset_index(drop=True)
        if tdf.empty:
            continue
        fe_df, rebuilt_feats = build_improved_features(tdf)
        feats_path = state["artifacts"][t]["feature_names_path"]
        if os.path.exists(feats_path):
            with open(feats_path, "r") as f:
                trained_feats = json.load(f)
            feats = [c for c in trained_feats if c in fe_df.columns]
        else:
            feats = rebuilt_feats
        fe_df = fe_df.dropna(subset=feats).copy()
        per_ticker[t] = {"full_df": fe_df, "feature_columns": feats}
    return {**state, "per_ticker": per_ticker}

def ig_compute(state):
    out_dir  = state.get("out_dir", "IG_XAI")
    steps    = int(state.get("ig_steps", 50))
    samples  = int(state.get("ig_samples", 80))
    head     = state.get("ig_head", "close")
    head_idx = 0 if str(head).lower() == "open" else 1

    results = {}
    for t, bundle in state.get("per_ticker", {}).items():
        try:
            art = state["artifacts"][t]
            model = load_model(art["model_path"])
            scaler = joblib.load(art["feature_scaler_path"])

            seq_len = 20
            if os.path.exists(art["seq_len_path"]):
                with open(art["seq_len_path"], "r") as f:
                    seq_len = int(f.read().strip())

            features = bundle["feature_columns"]
            fe_df = bundle["full_df"].copy()
            X_all = fe_df[features].values
            N = len(X_all)
            if N < seq_len + 10:
                print(f"[IG] {t}: not enough rows, skipping")
                continue
            trn = int(0.70 * N); val = int(0.15 * N)
            X_test = X_all[trn + val:]
            X_test_s = scaler.transform(pd.DataFrame(X_test, columns=features))
            X_seq = prepare_sequences_ig(X_test_s, seq_len)
            if len(X_seq) == 0:
                print(f"[IG] {t}: no test sequences, skipping")
                continue
            K = min(samples, len(X_seq))
            X_exp = X_seq[-K:]
            ig_vals = integrated_gradients_for_head(model, X_exp, head_idx=head_idx, steps=steps)
            res_mean, resid = ig_completeness_residual(model, X_exp, ig_vals, head_idx=head_idx)

            mean_abs_ig = np.mean(np.abs(ig_vals), axis=(0,1))
            imp = pd.DataFrame({"feature": features, "mean_abs_ig": mean_abs_ig}).sort_values("mean_abs_ig", ascending=False)

            tdir = os.path.join(out_dir, t); os.makedirs(tdir, exist_ok=True)
            np.savetxt(os.path.join(tdir, f"ig_completeness_residual_{head}.txt"), resid, fmt="%.6e")
            imp.to_csv(os.path.join(tdir, f"ig_feature_importance_{head}.csv"), index=False)

            avg_time_feat = np.mean(np.abs(ig_vals), axis=0)
            vmax = np.percentile(avg_time_feat, 99) if np.isfinite(avg_time_feat).all() else None
            plt.figure(figsize=(10, 4.8))
            plt.imshow(avg_time_feat.T, aspect='auto', interpolation='nearest', vmin=0.0, vmax=(vmax if (vmax and vmax>0) else None))
            plt.colorbar(label="mean |IG|")
            plt.yticks(ticks=np.arange(len(features)), labels=features)
            plt.xlabel("Time step (old → new)"); plt.ylabel("Feature")
            plt.title(f"{t} — IG Heatmap ({head} head)")
            plt.tight_layout(); plt.savefig(os.path.join(tdir, f"ig_heatmap_{head}.png"), dpi=300); plt.close()

            topN = min(20, len(features))
            plt.figure(figsize=(8, max(4, int(topN*0.35))))
            plt.barh(imp["feature"].head(topN)[::-1], imp["mean_abs_ig"].head(topN)[::-1])
            plt.title(f"{t} — Global Feature Importance (mean |IG|, {head} head)")
            plt.tight_layout(); plt.savefig(os.path.join(tdir, f"ig_global_importance_{head}.png"), dpi=300); plt.close()

            results[t] = {
                "head": head, "seq_len": int(seq_len), "rows_used": int(K),
                "ig_steps": int(steps), "ig_completeness_mean_abs_residual": float(res_mean),
                "top_features": imp.head(10).to_dict(orient="records"),
                "outputs_dir": tdir
            }
            print(f"[IG] {t}: saved → {tdir}")
        except Exception as e:
            print(f"[IG] {t}: failed — {e}")


    summary_path = os.path.join(out_dir, "ig_xai_summary.json")
    with open(summary_path, "w") as f:
        json.dump({"historical": results, "forecast": {}}, f, indent=2)
    print(f" Saved IG summary → {summary_path}")
    return {**state, "ig_results": results}

# ======================================================================
# ============================== SHAP ==============================
# ======================================================================

def build_features_like_trainer(ticker_df: pd.DataFrame):
    df = ticker_df.copy()
    roll50 = df["Close"].rolling(50)
    df["Price_Norm"] = (df["Close"] - roll50.mean()) / roll50.std()
    df["High_Low_Ratio"] = df["High"] / df["Low"].replace(0, np.nan)
    df["Open_Close_Ratio"] = df["Open"] / df["Close"].replace(0, np.nan)
    for w in [5, 10, 20]:
        ma = df["Close"].rolling(w).mean()
        df[f"MA_{w}"] = ma
        df[f"Close_MA{w}_Ratio"] = df["Close"] / ma.replace(0, np.nan)
        df[f"MA{w}_Slope"] = ma.diff(5) / ma.shift(5).replace(0, np.nan)
    for p in [1, 3, 5]:
        prev = df["Close"].shift(p).replace(0, np.nan)
        df[f"Log_Return_{p}d"] = np.log(df["Close"] / prev)
        df[f"Return_Volatility_{p}d"] = df[f"Log_Return_{p}d"].rolling(10).std()
    delta = df["Close"].diff()
    gain = delta.clip(lower=0).rolling(14).mean()
    loss = (-delta.clip(upper=0)).rolling(14).mean().replace(0, np.nan)
    rs = gain / loss
    df["RSI"] = 100 - (100 / (1 + rs))
    df["RSI_Norm"] = (df["RSI"] - 50) / 50
    exp1 = df["Close"].ewm(span=12, adjust=False).mean()
    exp2 = df["Close"].ewm(span=26, adjust=False).mean()
    df["MACD"] = exp1 - exp2
    df["MACD_Signal"] = df["MACD"].ewm(span=9, adjust=False).mean()
    df["MACD_Histogram"] = df["MACD"] - df["MACD_Signal"]
    df["MACD_Norm"] = df["MACD"] / df["Close"].replace(0, np.nan)
    vol_ma = df["Volume"].rolling(20).mean().replace(0, np.nan)
    df["Volume_MA"] = vol_ma
    df["Volume_Ratio"] = df["Volume"] / vol_ma
    df["Volume_Price_Correlation"] = df["Volume"].rolling(20).corr(df["Close"])
    df["Price_Volatility"] = df["Log_Return_1d"].rolling(20).std()
    df["High_Low_Volatility"] = np.log((df["High"] / df["Low"]).replace(0, np.nan)).rolling(10).mean()
    roll_min = df["Close"].rolling(20).min()
    roll_max = df["Close"].rolling(20).max()
    den = (roll_max - roll_min).replace(0, np.nan)
    df["Price_Position"] = (df["Close"] - roll_min) / den
    df["Momentum_5"] = df["Close"] / df["Close"].shift(5).replace(0, np.nan)
    df["Momentum_10"] = df["Close"] / df["Close"].shift(10).replace(0, np.nan)

    feature_columns = [
        "Price_Norm", "High_Low_Ratio", "Open_Close_Ratio",
        "Close_MA5_Ratio", "Close_MA10_Ratio", "Close_MA20_Ratio",
        "MA5_Slope", "MA10_Slope", "MA20_Slope",
        "Log_Return_1d", "Log_Return_3d", "Log_Return_5d",
        "Return_Volatility_1d", "Return_Volatility_3d",
        "RSI_Norm", "MACD_Norm", "MACD_Histogram",
        "Volume_Ratio", "Volume_Price_Correlation",
        "Price_Volatility", "High_Low_Volatility",
        "Price_Position", "Momentum_5", "Momentum_10",
    ]
    avail = [c for c in feature_columns if c in df.columns]
    df[avail] = df[avail].replace([np.inf, -np.inf], np.nan).ffill().bfill()
    return df, avail

def prepare_sequences_shap(features: np.ndarray, seq_len: int) -> np.ndarray:
    return np.array([features[i - seq_len:i] for i in range(seq_len, len(features))])

def shap_discover(state):
    models_dir = state.get("models_dir", "Predictive_Model/lstm_models")
    if not os.path.exists(models_dir):
        print(f" Models dir not found: {models_dir}")
        return {**state, "tickers": [], "artifacts": {}}

    tickers, artifacts = [], {}
    for fname in os.listdir(models_dir):
        if fname.endswith("_twohead.keras"):
            tickers.append(fname.split("_twohead.keras")[0])
        elif fname.endswith("_lstm.keras"):
            tickers.append(fname.split("_lstm.keras")[0])
    tickers = sorted(set(tickers))
    for t in tickers:
        model_path = os.path.join(models_dir, f"{t}_twohead.keras")
        if not os.path.exists(model_path):
            model_path = os.path.join(models_dir, f"{t}_lstm.keras")
        artifacts[t] = {
            "model_path": model_path,
            "feature_scaler_path": os.path.join(models_dir, f"{t}_feature_scaler.pkl"),
            "feature_names_path":   os.path.join(models_dir, f"{t}_feature_names.json"),
            "seq_len_path":         os.path.join(models_dir, f"{t}_seq_len.txt"),
        }
    return {**state, "tickers": tickers, "artifacts": artifacts}

def shap_prepare(state):
    data_path = state.get("data_path", "FE_Agent/features_engineered.csv")
    if not os.path.exists(data_path):
        print(f" Engineered data not found: {data_path}")
        return {**state, "per_ticker": {}}

    df = pd.read_csv(data_path)
    if "Date" in df.columns:
        df["Date"] = pd.to_datetime(df["Date"])
    per_ticker = {}
    for t in state.get("tickers", []):
        tdf = df[df["Symbol"] == t].copy().sort_values("Date").reset_index(drop=True)
        if len(tdf) < 200:
            print(f"   {t}: too few rows; skipping")
            continue
        fe_df, built_features = build_features_like_trainer(tdf)
        art = state["artifacts"][t]
        features = built_features
        fpath = art["feature_names_path"]
        if os.path.exists(fpath):
            try:
                with open(fpath, "r") as f:
                    saved_feats = json.load(f)
                features = [c for c in saved_feats if c in fe_df.columns]
            except Exception:
                pass
        fe_df = fe_df.dropna(subset=features + ["Date"]).copy()
        per_ticker[t] = {"full_df": fe_df, "feature_columns": features}
    return {**state, "per_ticker": per_ticker}

def shap_compute(state):
    if not HAS_SHAP:
        print("[SHAP] shap not installed; skipping SHAP")
        return {**state, "results": {}}

    out_dir = state.get("out_dir", "SHAP_XAI")
    os.makedirs(out_dir, exist_ok=True)

    head_name = str(state.get("head", "close")).lower()
    head_idx = 0 if head_name == "open" else 1
    k_last = int(state.get("k_last", 120))
    bg_cap = int(state.get("bg_cap", 100))

    results = {}
    for t, bundle in state.get("per_ticker", {}).items():
        try:
            art = state["artifacts"][t]
            model = load_model(art["model_path"])
            feat_scaler = joblib.load(art["feature_scaler_path"])

            seq_len = 20
            if os.path.exists(art["seq_len_path"]):
                try:
                    with open(art["seq_len_path"], "r") as f:
                        seq_len = int(f.read().strip())
                except Exception:
                    pass

            features = bundle["feature_columns"]
            fe_df = bundle["full_df"].copy()
            X_all = fe_df[features].values
            dates_all = fe_df["Date"].values

            total = len(X_all)
            trn = int(total * 0.70)
            val = int(total * 0.15)
            X_train = X_all[:trn]
            X_val   = X_all[trn: trn + val]
            X_test  = X_all[trn + val:]
            dates_test = dates_all[trn + val:]

            X_train_s = feat_scaler.transform(pd.DataFrame(X_train, columns=features))
            X_val_s   = feat_scaler.transform(pd.DataFrame(X_val,   columns=features))
            X_test_s  = feat_scaler.transform(pd.DataFrame(X_test,  columns=features))

            def _seq(x): return prepare_sequences_shap(x, seq_len)
            X_train_seq = _seq(X_train_s)
            X_val_seq   = _seq(X_val_s)
            X_test_seq  = _seq(X_test_s)
            if len(X_test_seq) < 5 or len(X_train_seq) < 20:
                print(f"    {t}: not enough sequences; skipping")
                continue

            B = min(bg_cap, len(X_train_seq))
            bg_idx = np.random.RandomState(42).choice(len(X_train_seq), size=B, replace=False)
            background = X_train_seq[bg_idx]

            k = min(k_last, len(X_test_seq))
            X_shap = X_test_seq[-k:]

            shap_vals = None
            try:
                explainer = shap.DeepExplainer(model, background)
                sv = explainer.shap_values(X_shap)
                shap_vals = sv[head_idx] if isinstance(sv, list) else sv
            except Exception as e:
                print(f"    {t}: DeepExplainer failed ({e}). Trying GradientExplainer…")
                try:
                    explainer = shap.GradientExplainer(model, background)
                    sv = explainer.shap_values(X_shap)
                    shap_vals = sv[head_idx] if isinstance(sv, list) else sv
                except Exception as e2:
                    print(f"    {t}: GradientExplainer failed ({e2}). Using KernelExplainer (slow)…")
                    X_bg_flat   = background.reshape((background.shape[0], -1))
                    X_shap_flat = X_shap.reshape((X_shap.shape[0], -1))
                    def f(z):
                        z3 = z.reshape((-1, seq_len, len(features)))
                        preds = model.predict(z3, verbose=0)
                        if preds.shape[-1] == 2:
                            return preds[:, head_idx]
                        return preds.reshape((-1,))
                    explainer = shap.KernelExplainer(f, X_bg_flat[:min(50, len(X_bg_flat))])
                    sv = explainer.shap_values(X_shap_flat, nsamples=200)
                    shap_vals = np.array(sv).reshape((X_shap.shape[0], seq_len, len(features)))

            try:
                exp_val_raw = explainer.expected_value
                base_value = float(exp_val_raw[head_idx]) if isinstance(exp_val_raw, (list, tuple, np.ndarray)) else float(exp_val_raw)
            except Exception:
                preds_bg = model.predict(background, verbose=0)
                base_value = float(np.mean(preds_bg[:, head_idx])) if preds_bg.ndim == 2 else float(np.mean(preds_bg))

            mean_abs_shap = np.mean(np.abs(shap_vals), axis=(0, 1))
            feat_importance = pd.DataFrame({"feature": features, "mean_abs_shap": mean_abs_shap}).sort_values("mean_abs_shap", ascending=False)

            ticker_dir = os.path.join(out_dir, t); os.makedirs(ticker_dir, exist_ok=True)
            suffix = head_name.lower()

            # SHAP files:
            feat_importance.to_csv(os.path.join(ticker_dir, f"{t}_feature_importance_{suffix}.csv"), index=False)

            topN = min(20, len(features))
            plt.figure(figsize=(8, max(4, int(topN * 0.35))))
            plt.barh(feat_importance["feature"].head(topN)[::-1],
                     feat_importance["mean_abs_shap"].head(topN)[::-1])
            plt.title(f"{t} — Global Feature Importance (mean |SHAP|, head='{suffix}')")
            plt.tight_layout()
            plt.savefig(os.path.join(ticker_dir, f"{t}_global_importance_{suffix}.png"), dpi=300)
            plt.close()

            heat = np.mean(np.abs(shap_vals), axis=0)
            vmax = np.percentile(heat, 99) if np.isfinite(heat).all() else None
            plt.figure(figsize=(12, 5))
            plt.imshow(heat.T, aspect='auto', interpolation='nearest',
                       vmin=0.0, vmax=vmax if (vmax is not None and vmax > 0) else None)
            plt.colorbar(label="mean |SHAP|")
            plt.yticks(ticks=np.arange(len(features)), labels=features)
            plt.xlabel("Time step (old → new)"); plt.ylabel("Feature")
            plt.title(f"{t} — SHAP Heatmap (head='{suffix}')")
            plt.tight_layout()
            plt.savefig(os.path.join(ticker_dir, f"{t}_heatmap_{suffix}.png"), dpi=300)
            plt.close()

            # plots
            X_last_step = X_shap[:, -1, :]
            shap_last_step = shap_vals[:, -1, :]

            try:
                shap.summary_plot(shap_last_step, X_last_step,
                                  feature_names=features, show=False,
                                  plot_type="dot", max_display=20)
                plt.title(f"{t} — SHAP Summary (last step, head='{suffix}')")
                plt.tight_layout()
                plt.savefig(os.path.join(ticker_dir, f"{t}_summary_beeswarm_{suffix}.png"),
                            dpi=300, bbox_inches="tight")
                plt.close()
            except Exception as e:
                print(f"    {t}: summary_plot failed ({e}) — continuing.")

            try:
                idx = -1
                fv = X_last_step[idx]
                sv = shap_last_step[idx]
                order = np.argsort(-np.abs(sv))
                vals = shap.Explanation(
                    values=sv[order],
                    base_values=np.array([base_value]),
                    data=fv[order],
                    feature_names=[features[i] for i in order]
                )
                shap.plots.waterfall(vals, max_display=20, show=False)
                plt.title(f"{t} — Local Waterfall (last sample, head='{suffix}')")
                plt.tight_layout()
                plt.savefig(os.path.join(ticker_dir, f"{t}_local_waterfall_last_{suffix}.png"),
                            dpi=300, bbox_inches="tight")
                plt.close()
            except Exception as e:
                print(f"    {t}: local waterfall failed ({e}) — continuing.")

            try:
                force = shap.force_plot(base_value,
                                        shap_last_step[-1],
                                        X_last_step[-1],
                                        feature_names=features,
                                        matplotlib=False)
                shap.save_html(os.path.join(ticker_dir, f"{t}_force_last_{suffix}.html"), force)
            except Exception as e:
                print(f"    {t}: force_plot (HTML) failed ({e})")
            try:
                plt.figure(figsize=(10, 2.8))
                shap.force_plot(base_value,
                                shap_last_step[-1],
                                X_last_step[-1],
                                feature_names=features,
                                matplotlib=True,
                                show=False)
                plt.title(f"{t} — SHAP Force (last sample, head='{suffix}')")
                plt.tight_layout()
                plt.savefig(os.path.join(ticker_dir, f"{t}_force_last_{suffix}.png"),
                            dpi=300, bbox_inches="tight")
                plt.close()
            except Exception as e:
                print(f"    {t}: force_plot (PNG) failed ({e})")

            try:
                n_overlay = min(50, shap_last_step.shape[0])
                plt.figure(figsize=(10, 5))
                shap.decision_plot(base_value,
                                   shap_last_step[-n_overlay:],
                                   feature_names=features,
                                   show=False)
                plt.title(f"{t} — SHAP Decision Plot (last {n_overlay} samples, head='{suffix}')")
                plt.tight_layout()
                plt.savefig(os.path.join(ticker_dir, f"{t}_decision_overlay_{suffix}.png"),
                            dpi=300, bbox_inches="tight")
                plt.close()
            except Exception as e:
                print(f"    {t}: decision_plot (overlay) failed ({e})")
            try:
                plt.figure(figsize=(10, 4))
                shap.decision_plot(base_value,
                                   shap_last_step[-1],
                                   feature_names=features,
                                   show=False)
                plt.title(f"{t} — SHAP Decision Plot (last sample, head='{suffix}')")
                plt.tight_layout()
                plt.savefig(os.path.join(ticker_dir, f"{t}_decision_last_{suffix}.png"),
                            dpi=300, bbox_inches="tight")
                plt.close()
            except Exception as e:
                print(f"    {t}: decision_plot (single) failed ({e})")

            results[t] = {
                "head": head_name, "rows_used": int(len(X_shap)),
                "seq_len": int(seq_len), "features": features,
                "top_features": feat_importance.head(10).to_dict(orient="records"),
                "outputs_dir": ticker_dir
            }
            print(f"    {t}: SHAP completed (head='{suffix}')")

        except Exception as e:
            print(f"   {t}: SHAP failed — {e}")

    # summary file
    summary_path = os.path.join(out_dir, "shap_xai_summary.json")
    with open(summary_path, "w") as f:
        json.dump(results, f, indent=2)
    print(f" Saved summary → {summary_path}")
    return {**state, "results": results}

# ======================================================================
# =========================== COMBINED RUNNER ===========================
# ======================================================================

def run_explainability(state: dict) -> dict:
    """
    Runs IG first (saving under Explainability/IG_XAI), then SHAP
    (Explainability/SHAP_XAI). Artifacts & filenames saved under this folder.
    """
    models_dir, fe_path, explain_root, ig_root, shap_root = _infer_paths(state)
    os.makedirs(explain_root, exist_ok=True)

    # -------- IG phase --------
    ig_state = {
        "models_dir": models_dir,
        "data_path":  fe_path,
        "out_dir":    ig_root,
        "ig_steps":   int(state.get("ig_steps", 50)),
        "ig_samples": int(state.get("ig_samples", 80)),
        "ig_head":    state.get("ig_head", "close"),
    }
    ig_state = ig_discover(ig_state)
    ig_state = ig_prepare(ig_state)
    ig_state = ig_compute(ig_state)

    # -------- SHAP phase --------
    shap_state = {
        "models_dir": models_dir,
        "data_path":  fe_path,
        "out_dir":    shap_root,
        "head":       state.get("shap_head", state.get("ig_head", "close")),
        "k_last":     int(state.get("k_last", 120)),
        "bg_cap":     int(state.get("bg_cap", 100)),
    }
    shap_state = shap_discover(shap_state)
    shap_state = shap_prepare(shap_state)
    shap_state = shap_compute(shap_state)

    # Combined summary
    summary_path = os.path.join(explain_root, "explainability_summary.json")
    with open(summary_path, "w") as f:
        json.dump({
            "IG": ig_state.get("ig_results", {}),
            "SHAP": shap_state.get("results", {}),
            "roots": {"IG_XAI": ig_root, "SHAP_XAI": shap_root}
        }, f, indent=2)
    print(f"[Explainability] Combined summary → {summary_path}")

    return {
        "explain_root": explain_root,
        "ig_root": ig_root,
        "shap_root": shap_root,
        "ig_summary": os.path.join(ig_root, "ig_xai_summary.json"),
        "shap_summary": os.path.join(shap_root, "shap_xai_summary.json"),
        "combined_summary": summary_path,
    }

# LangGraph wrapper (runs IG → SHAP)
def build_explainability_workflow():
    if not HAS_LG:
        raise RuntimeError("LangGraph not available. Install with: pip install langgraph")

    def _runner(state):
        return {**state, **run_explainability(state)}
    g = StateGraph(dict)
    g.add_node("run", _runner)
    g.set_entry_point("run")
    g.set_finish_point("run")
    return g.compile()

if __name__ == "__main__":
    print(" Running Explainability Agent (IG → SHAP)…")
    _ = run_explainability({
        # "run_base": "/content/drive/MyDrive/A2A_prediction_system/RUN_YYYYMMDD_HHMMSS",
        "models_dir": "Predictive_Model/lstm_models",
        "fe_path":    "FE_Agent/features_engineered.csv",
        "ig_head": "close", "ig_steps": 50, "ig_samples": 80,
        "shap_head": "close", "k_last": 120, "bg_cap": 100,
    })


**risk_assessment.py**

In [ ]:
%%writefile /content/drive/MyDrive/A2A_prediction_system/backend/a2a/risk_assessment_agent.py

import os, json, warnings
warnings.filterwarnings("ignore")

from typing import Dict, Any, Optional, List, Tuple
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

try:
    from langgraph.graph import StateGraph
    HAS_LANGGRAPH = True
except Exception:
    HAS_LANGGRAPH = False

# ---------- Defaults ----------
FE_PATH_DEFAULT           = "FE_Agent/features_engineered.csv"
PRED_DIR_DEFAULT          = "Predictive_Model/predictions"
FCAST_DIR_DEFAULT         = "Predictive_Model/advanced_forecasts"
SHAP_DIR_DEFAULT          = "SHAP_XAI"
OUT_DIR_DEFAULT           = "Risk_Assessment"

ALPHA_DEFAULT             = 0.10         # 90% PIs
CALIB_FRAC_DEFAULT        = 0.70         # residuals split: first 70% for calibration
CALIB_MIN_DEFAULT         = 60           # min calibration points
ROLL_COVER_WIN_DEFAULT    = 60
BOOTSTRAP_PATHS_DEFAULT   = 200
VOL_LOOKBACK_DEFAULT      = 20
ADAPT_WIDTH_WIN           = 252          # derive "wide band" threshold from last 1y widths

rng = np.random.RandomState(42)

# ---------- run_base helper ----------
def _under_run_base(state: Dict[str, Any], rel_or_abs: str) -> str:
    """If `run_base` is present, resolve paths under it; otherwise use as-is."""
    if not rel_or_abs:
        return rel_or_abs
    rb = state.get("run_base")
    if rb and not os.path.isabs(rel_or_abs):
        return os.path.join(rb, rel_or_abs)
    return rel_or_abs

# ---------- Basic helpers ----------
def pct_returns(series: pd.Series) -> pd.Series:
    return series.pct_change().dropna()

def rolling_vol(returns: pd.Series, window: int = VOL_LOOKBACK_DEFAULT) -> pd.Series:
    return returns.rolling(window).std()

def drawdown(close: pd.Series) -> pd.Series:
    peak = close.cummax()
    return (close - peak) / peak

def sharpe_ratio(returns: pd.Series, annualization: int = 252) -> float:
    r = returns.dropna()
    if len(r) < 2 or r.std() == 0: return float("nan")
    return float(np.sqrt(annualization) * r.mean() / r.std())

def var_percentile(returns: pd.Series, q: float = 0.05) -> float:
    r = returns.dropna()
    if len(r) == 0: return float("nan")
    return float(np.quantile(r, q))

def winkler_score(y: np.ndarray, lo: np.ndarray, hi: np.ndarray, alpha: float) -> np.ndarray:
    width = hi - lo
    below, above = y < lo, y > hi
    score = width.copy()
    score[below] += (2.0/alpha) * (lo[below] - y[below])
    score[above] += (2.0/alpha) * (y[above] - hi[above])
    return score

# ---------- I/O helpers ----------
def load_test_predictions(pred_dir: str, ticker: str) -> Optional[pd.DataFrame]:
    path = os.path.join(pred_dir, f"{ticker}_test_predictions.csv")
    if not os.path.exists(path): return None
    df = pd.read_csv(path)
    # normalize names
    ren = {}
    if "Actual_Close_Return" in df.columns: ren["Actual_Close_Return"] = "Actual_Return"
    if "Pred_Close_Return"   in df.columns: ren["Pred_Close_Return"]   = "Pred_Return"
    df = df.rename(columns=ren)
    need = {"Actual_Return","Pred_Return"}
    if not need.issubset(df.columns): return None
    if "Date" in df.columns: df["Date"] = pd.to_datetime(df["Date"])
    return df

def load_tomorrow(pred_dir: str, ticker: str) -> Optional[dict]:
    path = os.path.join(pred_dir, f"{ticker}_tomorrow.csv")
    if not os.path.exists(path): return None
    return pd.read_csv(path).iloc[0].to_dict()

def load_forecast(fdir: str, ticker: str) -> Optional[pd.DataFrame]:
    path = os.path.join(fdir, f"{ticker}_forecast_7d.csv")
    if not os.path.exists(path): return None
    df = pd.read_csv(path, parse_dates=["Date"])
    if "Pred_Close_Return" in df.columns: df = df.rename(columns={"Pred_Close_Return":"Pred_Return"})
    if "Predicted_Close" in df.columns: df["Pred_Close"] = df["Predicted_Close"]
    need = {"Date","Pred_Return","Pred_Close"}
    return df[list(need)] if need.issubset(df.columns) else None

def load_top_features(shap_dir: str, ticker: str, head: str = "close", top_n: int = 5) -> Optional[List[str]]:
    path = os.path.join(shap_dir, ticker, f"{ticker}_feature_importance_{head}.csv")
    if not os.path.exists(path): return None
    try:
        imp = pd.read_csv(path)
        col_feat = "feature" if "feature" in imp.columns else imp.columns[0]
        col_score= "mean_abs_shap" if "mean_abs_shap" in imp.columns else imp.columns[1]
        imp = imp.sort_values(col_score, ascending=False)
        return imp[col_feat].head(top_n).tolist()
    except Exception:
        return None

# ---------- Risk helpers ----------
def volatility_marks(returns: pd.Series, lookback: int = VOL_LOOKBACK_DEFAULT):
    rv = returns.rolling(lookback).std().dropna()
    if len(rv) < 30: return None
    return {"recent": float(rv.iloc[-1]),
            "p75": float(np.quantile(rv, 0.75)),
            "p90": float(np.quantile(rv, 0.90))}

def risk_label(interval_pct: float, crosses_zero: bool, marks: Optional[dict], width_thresh: float):
    triggers = []
    if interval_pct >= width_thresh: triggers.append("uncertainty_wide")
    if crosses_zero: triggers.append("direction_uncertain")
    if marks:
        if marks["recent"] >= marks["p90"]: triggers.append("volatility_extreme")
        elif marks["recent"] >= marks["p75"]: triggers.append("volatility_high")
    if "volatility_extreme" in triggers or ("uncertainty_wide" in triggers and "direction_uncertain" in triggers):
        lvl = "HIGH"
    elif triggers:
        lvl = "MEDIUM"
    else:
        lvl = "LOW"
    return lvl, triggers

def return_to_price_interval(latest_close: float, pred_return: float, q_abs_ret: float):
    lo_r, hi_r = pred_return - q_abs_ret, pred_return + q_abs_ret
    lo_p = latest_close * (1.0 + lo_r)
    hi_p = latest_close * (1.0 + hi_r)
    return (lo_r, hi_r), (lo_p, hi_p)

# ---------- Conformal calibration ----------
def calibrate_from_residuals(df_test: pd.DataFrame,
                             alpha: float,
                             calib_frac: float,
                             calib_min: int,
                             roll_cov_win: int) -> Dict[str, Any]:
    y  = df_test["Actual_Return"].values.astype(float)
    yh = df_test["Pred_Return"].values.astype(float)
    resid = y - yh
    abs_resid = np.abs(resid)
    n = len(abs_resid)
    if n < max(calib_min, 20):
        return {"ok": False, "reason": "not_enough_points"}

    n_cal = max(calib_min, int(calib_frac * n))
    n_cal = min(n_cal, n - 5)
    cal = abs_resid[:n_cal].copy()
    lo, hi = np.quantile(cal, [0.01, 0.99])
    cal = np.clip(cal, lo, hi)
    q = float(np.quantile(cal, 1 - alpha))

    # Diagnostics on evaluation tail
    y_ev, yh_ev = y[n_cal:], yh[n_cal:]
    lo_ev, hi_ev = yh_ev - q, yh_ev + q
    cov = float(np.mean((y_ev >= lo_ev) & (y_ev <= hi_ev))) if len(y_ev) else np.nan
    wink = float(np.mean(winkler_score(y_ev, lo_ev, hi_ev, alpha))) if len(y_ev) else np.nan

    # Rolling coverage (on full series)
    lo_all, hi_all = yh - q, yh + q
    ok_all = (y >= lo_all) & (y <= hi_all)
    roll_cov = pd.Series(ok_all).rolling(roll_cov_win).mean().values

    # Adaptive width history
    width_hist = None
    if {"Pred_Close","Actual_Close"}.issubset(df_test.columns):
        base = df_test["Actual_Close"].shift(1).values.astype(float)
        width_price = base * (2.0 * q)  # approximate price width from return width
        rel_width = width_price / np.maximum(df_test["Pred_Close"].values.astype(float), 1e-8)
        width_hist = pd.Series(rel_width).rolling(ADAPT_WIDTH_WIN).mean().dropna().values

    return {
        "ok": True,
        "alpha": alpha,
        "q_abs_ret": q,
        "coverage_eval": cov,
        "winkler_mean_eval": wink,
        "indices": {"calib_end": int(n_cal), "total": int(n)},
        "roll_cov": roll_cov.tolist(),
        "width_hist": width_hist.tolist() if width_hist is not None else None
    }

# ---------- Fan chart (residual bootstrapping) ----------
def bootstrap_fan_chart(start_price: float,
                        pred_returns: np.ndarray,
                        calib_residuals: np.ndarray,
                        n_paths: int = BOOTSTRAP_PATHS_DEFAULT) -> Dict[str, np.ndarray]:
    T = len(pred_returns)
    paths = np.zeros((n_paths, T))
    for i in range(n_paths):
        p = start_price
        eps = rng.choice(calib_residuals, size=T, replace=True)
        for t in range(T):
            r = float(pred_returns[t]) + float(eps[t])
            p = p * (1.0 + r)
            paths[i, t] = p
    return {"p10": np.percentile(paths, 10, axis=0),
            "p50": np.percentile(paths, 50, axis=0),
            "p90": np.percentile(paths, 90, axis=0)}

# =========================================================
# -------------------- Pipeline nodes ---------------------
# =========================================================
def node_load(state: Dict[str, Any]) -> Dict[str, Any]:

    fe_path   = _under_run_base(state, state.get("data_path", FE_PATH_DEFAULT))
    pred_dir  = _under_run_base(state, state.get("predictions_dir", PRED_DIR_DEFAULT))
    fcast_dir = _under_run_base(state, state.get("forecasts_dir", FCAST_DIR_DEFAULT))
    shap_dir  = _under_run_base(state, state.get("shap_dir", SHAP_DIR_DEFAULT))

    tickers   = state.get("tickers")
    days_win  = state.get("days_window", None)

    if not os.path.exists(fe_path):
        raise FileNotFoundError(f"Engineered features not found: {fe_path}")

    fe = pd.read_csv(fe_path, parse_dates=["Date"])
    need = {"Date","Symbol","Close"}
    if not need.issubset(fe.columns):
        raise ValueError(f"{fe_path} missing columns {need}")
    fe = fe[["Date","Symbol","Close"]].dropna().sort_values(["Symbol","Date"]).reset_index(drop=True)
    if not tickers:
        tickers = sorted(fe["Symbol"].unique().tolist())

    per_ticker = {}
    for t in tickers:
        hist = fe[fe["Symbol"] == t].copy()
        if hist.empty: continue
        if days_win and len(hist) > days_win:
            hist = hist.tail(days_win).copy()

        test = load_test_predictions(pred_dir, t)
        tom  = load_tomorrow(pred_dir, t)
        fc   = load_forecast(fcast_dir, t)
        top5 = load_top_features(shap_dir, t, head=state.get("head","close"))

        per_ticker[t] = {
            "hist": hist,              # Date, Close
            "test_preds": test,        # residuals source
            "tomorrow": tom,           # next-day point forecast
            "forecast": fc,            # 7-day point forecasts
            "top_features": top5       # enrichment
        }

    return {**state,
            "per_ticker": per_ticker,
            "tickers": list(per_ticker.keys()),
            "fe_path": fe_path,
            "pred_dir": pred_dir,
            "fcast_dir": fcast_dir,
            "shap_dir": shap_dir,
            "status": "loaded"}

def node_assess(state: Dict[str, Any]) -> Dict[str, Any]:
    alpha      = float(state.get("alpha", ALPHA_DEFAULT))
    calib_frac = float(state.get("calib_frac", CALIB_FRAC_DEFAULT))
    calib_min  = int(state.get("calib_min", CALIB_MIN_DEFAULT))
    roll_win   = int(state.get("roll_cov_win", ROLL_COVER_WIN_DEFAULT))
    ann        = int(state.get("annualization", 252))
    lookback   = int(state.get("vol_lookback", VOL_LOOKBACK_DEFAULT))

    for t, bundle in state["per_ticker"].items():
        hist = bundle["hist"].copy().set_index("Date")
        close = hist["Close"].astype(float)
        ret   = pct_returns(close)
        vol   = rolling_vol(ret, window=lookback)
        dd    = drawdown(close)
        marks = volatility_marks(ret, lookback=lookback)

        # Metrics (historical)
        metrics = {
            "Obs": int(len(close)),
            "Daily_Vol": float(ret.std()) if len(ret) else np.nan,
            "Sharpe": sharpe_ratio(ret, ann),
            "Max_Drawdown": float(dd.min()) if len(dd) else np.nan,
            "VaR_5pct_1d": var_percentile(ret, 0.05),
        }

        # Conformal calibration
        calib = {"ok": False, "reason": "no_test_predictions"}
        if bundle["test_preds"] is not None and len(bundle["test_preds"]) >= max(calib_min, 20):
            calib = calibrate_from_residuals(bundle["test_preds"], alpha, calib_frac, calib_min, roll_win)

        # Adaptive width threshold
        width_thresh = 0.02
        if calib.get("width_hist"):
            w = np.array(calib["width_hist"])
            if len(w) > 20:
                width_thresh = float(np.nanpercentile(w, 80))

        # Tomorrow assessment
        risk = None
        predicted_rows = []
        if bundle["tomorrow"] and calib.get("ok"):
            tom   = bundle["tomorrow"]
            last_close = float(tom.get("Latest_Actual_Close", close.iloc[-1]))
            r_pred = float(tom.get("Pred_Close_Return", tom.get("Pred_Return", 0.0)))
            p_pred = float(tom.get("Pred_Close", last_close * (1 + r_pred)))
            (lo_r, hi_r), (lo_p, hi_p) = return_to_price_interval(last_close, r_pred, calib["q_abs_ret"])
            band_pct = (hi_p - lo_p) / max(p_pred, 1e-8)
            crosses0 = (lo_r <= 0.0 <= hi_r)
            label, triggers = risk_label(band_pct, crosses0, marks, width_thresh)

            snr = abs(r_pred) / max(calib["q_abs_ret"], 1e-12)

            risk = {
                "Pred_Close": p_pred,
                "PI_Return": [lo_r, hi_r],
                "PI_Close": [lo_p, hi_p],
                "Uncertainty_Band_Pct": float(band_pct),
                "Crosses_Zero": bool(crosses0),
                "Risk_Label": label,
                "Risk_Triggers": triggers,
                "SNR": float(snr),
                "Width_Thresh_Used": width_thresh,
                "Top_Features": bundle.get("top_features")
            }

            predicted_rows.append({
                "For_Date": tom.get("For_Date"),
                "Pred_Close": p_pred,
                "PI_Close_Lo": lo_p,
                "PI_Close_Hi": hi_p,
                "BandPct": band_pct,
                "CrossesZero": crosses0,
                "Risk_Label": label,
                "Triggers": ";".join(triggers),
                "Pred_Close_Return": r_pred,
                "SNR": snr,
                "Top_Features": ",".join(bundle.get("top_features") or [])
            })

        # 7-day fan chart (forecast + calibration)
        forecast_bands = None
        if bundle["forecast"] is not None and calib.get("ok"):
            df_test = bundle["test_preds"]
            n_cal = calib["indices"]["calib_end"] if calib.get("indices") else max(60, int(0.7*len(df_test)))
            eps = (df_test["Actual_Return"].values[:n_cal] - df_test["Pred_Return"].values[:n_cal])
            fc = bundle["forecast"].copy()
            fan = bootstrap_fan_chart(float(close.iloc[-1]),
                                      fc["Pred_Return"].values.astype(float),
                                      eps,
                                      n_paths=state.get("bootstrap_paths", BOOTSTRAP_PATHS_DEFAULT))
            fc["PI10"] = fan["p10"]; fc["PI50"] = fan["p50"]; fc["PI90"] = fan["p90"]
            forecast_bands = fc


            q = calib["q_abs_ret"]
            base = float(close.iloc[-1])
            for i, r_k in enumerate(fc["Pred_Return"].values.astype(float)):
                (lo_r, hi_r), (lo_p, hi_p) = return_to_price_interval(base, r_k, q)
                band_pct = (hi_p - lo_p) / max(fc["Pred_Close"].iloc[i], 1e-8)
                crosses0 = (lo_r <= 0.0 <= hi_r)
                label, triggers = risk_label(band_pct, crosses0, marks, width_thresh)
                predicted_rows.append({
                    "For_Date": fc["Date"].iloc[i].strftime("%Y-%m-%d"),
                    "Pred_Close": float(fc["Pred_Close"].iloc[i]),
                    "PI_Close_Lo": lo_p,
                    "PI_Close_Hi": hi_p,
                    "BandPct": band_pct,
                    "CrossesZero": crosses0,
                    "Risk_Label": label,
                    "Triggers": ";".join(triggers),
                    "Pred_Close_Return": r_k,
                    "SNR": abs(r_k)/max(q,1e-12),
                    "Top_Features": ",".join(bundle.get("top_features") or [])
                })

        # Persist in bundle
        bundle.update({
            "close": close, "returns": ret, "rolling_vol": vol, "drawdown": dd,
            "vol_marks": marks, "calibration": calib, "risk": risk,
            "forecast_bands": forecast_bands,
            "metrics": metrics,
            "predicted_rows": predicted_rows
        })

    return {**state, "status": "assessed"}

def node_save(state: Dict[str, Any]) -> Dict[str, Any]:
    out_dir = _under_run_base(state, state.get("out_dir", OUT_DIR_DEFAULT))
    os.makedirs(out_dir, exist_ok=True)
    os.makedirs(os.path.join(out_dir, "plots"), exist_ok=True)

    summary, rows = {}, []

    for t, b in state["per_ticker"].items():
        hist_idx = b["close"].index
        # ---- Historical CSV
        pd.DataFrame({
            "Date": hist_idx,
            "Close": b["close"].values,
            "Return": b["returns"].reindex(hist_idx).values,
            "RollingVol20": b["rolling_vol"].reindex(hist_idx).values,
            "Drawdown": b["drawdown"].reindex(hist_idx).values
        }).to_csv(os.path.join(out_dir, f"{t}_historical_risk.csv"), index=False)

        # ---- Predicted CSV (tomorrow + 7-day)
        if b.get("predicted_rows"):
            pd.DataFrame(b["predicted_rows"]).to_csv(os.path.join(out_dir, f"{t}_predicted_risk.csv"), index=False)

        # ---- Plots
        idx = b["close"].index
        n = len(idx)
        i_tr = int(0.70 * n)          # end of train
        i_va = int(0.85 * n)          # end of validation
        split_dates = []
        if n >= 10:  # safety
            split_dates = [idx[i_tr], idx[i_va]]

        def _plot_with_splits(series, title, ylabel, fname):
            plt.figure(figsize=(12, 4))
            plt.plot(series.index, series.values, linewidth=1.3)

            for x in split_dates:
                plt.axvline(x=x, linestyle="--", linewidth=1)

            if split_dates:
                ylim_top = plt.gca().get_ylim()[1]
                plt.text(split_dates[0], ylim_top, "Train/Val split", va="bottom", ha="left", fontsize=8)
                plt.text(split_dates[1], ylim_top, "Val/Test split",  va="bottom", ha="left", fontsize=8)
            plt.title(title)
            plt.xlabel("Date")
            plt.ylabel(ylabel)
            plt.grid(alpha=0.3)
            plt.tight_layout()
            plt.savefig(os.path.join(out_dir, "plots", fname), dpi=300)
            plt.close()


        _plot_with_splits(b["close"],      f"{t} — Close",                    "Price (USD)",          f"{t}_price.png")
        # Drawdown and rolling volatility plot
        _plot_with_splits(b["drawdown"],   f"{t} — Drawdown",                 "Drawdown (fraction)",  f"{t}_drawdown.png")
        _plot_with_splits(b["rolling_vol"],f"{t} — Rolling Volatility (20d)", "Std. dev. (daily)",    f"{t}_rolling_vol.png")

        # Daily returns histogram
        rets = b["returns"].dropna()
        plt.figure(figsize=(8, 5))
        plt.hist(rets, bins=60, alpha=0.85)
        plt.axvline(0.0, linestyle="--", linewidth=1)
        if len(rets) > 0:
            plt.axvline(rets.mean(), linestyle=":", linewidth=1)
        plt.title(f"{t} — Daily Returns Histogram")
        plt.xlabel("Daily return")
        plt.ylabel("Frequency")
        plt.grid(alpha=0.25)
        plt.tight_layout()
        plt.savefig(os.path.join(out_dir, "plots", f"{t}_return_hist.png"), dpi=300)
        plt.close()

        # ---- Residual histogram & rolling coverage
        calib = b.get("calibration", {})
        if b.get("test_preds") is not None:
            resid = (b["test_preds"]["Actual_Return"] - b["test_preds"]["Pred_Return"]).values
            plt.figure(figsize=(8, 5))
            plt.hist(resid, bins=40, alpha=0.85)
            if calib.get("ok"):
                q = calib["q_abs_ret"]
                plt.axvline(+q, color="r", linestyle="--", label=f"+q @ {(1-calib['alpha'])*100:.0f}%")
                plt.axvline(-q, color="r", linestyle="--")
            plt.title(f"{t} — Test Residuals (returns)")
            plt.xlabel("Residual (Actual − Pred)")
            plt.ylabel("Count")
            plt.legend()
            plt.grid(alpha=0.3)
            plt.tight_layout()
            plt.savefig(os.path.join(out_dir, "plots", f"{t}_residual_hist.png"), dpi=300)
            plt.close()

        if calib.get("ok") and calib.get("roll_cov"):
            rc = np.array(calib["roll_cov"])
            plt.figure(figsize=(10, 3.5))
            plt.plot(rc, label="Rolling coverage")
            plt.axhline(1 - calib["alpha"], color="k", linestyle="--", label="Target")
            plt.ylim(0, 1)
            plt.title(f"{t} — Rolling Coverage")
            plt.xlabel("Window index")
            plt.ylabel("Coverage")
            plt.legend()
            plt.grid(alpha=0.3)
            plt.tight_layout()
            plt.savefig(os.path.join(out_dir, "plots", f"{t}_rolling_coverage.png"), dpi=300)
            plt.close()

        # ---- Tomorrow band plot
        if b.get("risk"):
            r = b["risk"]
            p_mid = r["Pred_Close"]
            lo_p, hi_p = r["PI_Close"]
            plt.figure(figsize=(7.8, 5))
            plt.errorbar([0], [p_mid], yerr=[[p_mid - lo_p], [hi_p - p_mid]], fmt="o")
            ttl = f"{t} — Tomorrow Close Uncertainty ({r['Risk_Label']})"
            plt.title(ttl)
            plt.xticks([])
            plt.ylabel("Price")
            plt.grid(alpha=0.3)
            txt = (
                f"Pred: {p_mid:.2f}\n"
                f"PI: [{lo_p:.2f}, {hi_p:.2f}] (width {100*r['Uncertainty_Band_Pct']:.2f}%)\n"
                f"Signal-to-noise ratio: {r['SNR']:.2f}"
            )
            plt.text(0.05, 0.95, txt, transform=plt.gca().transAxes, va="top")
            plt.tight_layout()
            plt.savefig(os.path.join(out_dir, "plots", f"{t}_tomorrow_band.png"), dpi=300)
            plt.close()

        # ---- Fan chart
        if isinstance(b.get("forecast_bands"), pd.DataFrame):
            fc = b["forecast_bands"]
            plt.figure(figsize=(10, 4))
            plt.plot(fc["Date"], fc["PI50"], label="Median", linewidth=2)
            plt.fill_between(fc["Date"], fc["PI10"], fc["PI90"], alpha=0.25, label="10–90% band")
            plt.plot(fc["Date"], fc["Pred_Close"], linestyle="--", label="Model path")
            plt.title(f"{t} — 7-Day Forecast Fan Chart")
            plt.xlabel("Date")
            plt.ylabel("Price")
            plt.legend()
            plt.grid(alpha=0.3)
            plt.tight_layout()
            plt.savefig(os.path.join(out_dir, "plots", f"{t}_fan_chart.png"), dpi=300)
            plt.close()
            fc.to_csv(os.path.join(out_dir, f"{t}_forecast_fanchart.csv"), index=False)

        # Summary
        summary[t] = {
            "metrics": b["metrics"],
            "calibration": {k:v for k,v in calib.items() if k in
                            {"ok","alpha","q_abs_ret","coverage_eval","winkler_mean_eval","indices"}},
            "risk": b.get("risk"),
            "top_features": b.get("top_features")
        }
        rows.append({
            "Ticker": t,
            "Obs": b["metrics"]["Obs"],
            "Daily_Vol": round(b["metrics"]["Daily_Vol"],6) if not np.isnan(b["metrics"]["Daily_Vol"]) else np.nan,
            "Sharpe": round(b["metrics"]["Sharpe"],3) if not np.isnan(b["metrics"]["Sharpe"]) else np.nan,
            "Max_Drawdown": round(b["metrics"]["Max_Drawdown"],4) if not np.isnan(b["metrics"]["Max_Drawdown"]) else np.nan,
            "VaR_5pct_1d": round(b["metrics"]["VaR_5pct_1d"],4) if not np.isnan(b["metrics"]["VaR_5pct_1d"]) else np.nan,
            "Risk_Label_Tomorrow": b.get("risk",{}).get("Risk_Label") if b.get("risk") else None
        })

    # Write summary files
    with open(os.path.join(out_dir, "summary.json"), "w") as f:
        json.dump(summary, f, indent=2)
    if rows:
        pd.DataFrame(rows).sort_values("Ticker").to_csv(os.path.join(out_dir, "summary.csv"), index=False)

    print(f" Risk assessment saved → {out_dir}")
    return {**state, "status": "done", "report_dir": out_dir}

# ---------- Build / run ----------
def build_risk_workflow():
    g = StateGraph(dict)
    g.add_node("load", node_load)
    g.add_node("assess", node_assess)
    g.add_node("save", node_save)
    g.set_entry_point("load")
    g.add_edge("load", "assess")
    g.add_edge("assess", "save")
    g.set_finish_point("save")
    return g.compile()

if __name__ == "__main__":
    payload = {

        # "run_base": "/content/drive/MyDrive/A2A_prediction_system/RUN_YYYYMMDD_HHMMSS",

        "data_path": FE_PATH_DEFAULT,
        "predictions_dir": PRED_DIR_DEFAULT,
        "forecasts_dir": FCAST_DIR_DEFAULT,
        "shap_dir": SHAP_DIR_DEFAULT,
        "out_dir": OUT_DIR_DEFAULT,

        # Order of agents: Prediction → SHAP/IG → Risk (this agent) → Evaluation
        # tickers: omit to auto-detect
        # "tickers": ["AAPL"],

        # Calibration & diagnostics
        "alpha": ALPHA_DEFAULT,
        "calib_frac": CALIB_FRAC_DEFAULT,
        "calib_min": CALIB_MIN_DEFAULT,
        "roll_cov_win": ROLL_COVER_WIN_DEFAULT,

        # Risk context
        "annualization": 252,
        "vol_lookback": VOL_LOOKBACK_DEFAULT,

        # Fan chart
        "bootstrap_paths": BOOTSTRAP_PATHS_DEFAULT,

        # Data window
        "days_window": 252,

        # SHAP head (for top-features enrichment)
        "head": "close"
    }
    if HAS_LANGGRAPH:
        app = build_risk_workflow()
        _ = app.invoke(payload)
    else:
        s = node_load(payload)
        s = node_assess(s)
        s = node_save(s)


**optimisation_agent.py**

In [ ]:
%%writefile /content/drive/MyDrive/A2A_prediction_system/backend/a2a/optimisation_agent.py


import os, json, warnings
warnings.filterwarnings("ignore")

from typing import Dict, Any, Optional
import numpy as np
import pandas as pd

# ---- LangGraph wiring ----
try:
    from langgraph.graph import StateGraph
    HAS_LANGGRAPH = True
except Exception:
    HAS_LANGGRAPH = False

# ---------------- utilities ----------------
def ensure_dir(p: str): os.makedirs(p, exist_ok=True)

def _under_run_base(state: Dict[str, Any], path: str) -> str:
    """
    If state has run_base and 'path' is relative, join them.
    Otherwise, return path unchanged.
    """
    rb = state.get("run_base")
    if rb and not os.path.isabs(path):
        return os.path.join(rb, path)
    return path

def kpis(equity: pd.Series, daily_ret: pd.Series) -> Dict[str, float]:
    equity = equity.dropna(); daily_ret = daily_ret.dropna()
    if len(equity) < 2 or len(daily_ret) < 2:
        return {k: float("nan") for k in
                ["CAGR","Sharpe","MaxDrawdown","HitRate","Volatility_Daily","TotalReturn"]}
    total_return = float(equity.iloc[-1] / equity.iloc[0] - 1.0)
    years = max((pd.to_datetime(equity.index[-1]) - pd.to_datetime(equity.index[0])).days / 365.25, 1e-9)
    cagr = float((equity.iloc[-1] / equity.iloc[0]) ** (1/years) - 1.0)
    vol = float(daily_ret.std())
    sharpe = float(np.sqrt(252) * daily_ret.mean() / vol) if vol > 0 else float("nan")
    dd = float((equity / equity.cummax() - 1.0).min())
    hit = float((daily_ret > 0).mean() * 100.0)
    return {"CAGR": cagr, "Sharpe": sharpe, "MaxDrawdown": dd,
            "HitRate": hit, "Volatility_Daily": vol, "TotalReturn": total_return}

# ------------- schema helpers -------------
def _norm_predictions(df: pd.DataFrame, head: str) -> pd.DataFrame:
    """Return Date, Pred_Return, Actual_Return."""
    df = df.copy()
    if "Date" not in df.columns: raise ValueError("Predictions CSV missing 'Date'")
    pred_two = f"Pred_{head.capitalize()}_Return"
    act_two  = f"Actual_{head.capitalize()}_Return"
    if {"Pred_Return","Actual_Return"}.issubset(df.columns):
        out = df[["Date","Pred_Return","Actual_Return"]].copy()
    elif {pred_two, act_two}.issubset(df.columns):
        out = df[["Date", pred_two, act_two]].rename(
            columns={pred_two:"Pred_Return", act_two:"Actual_Return"})
    else:
        raise ValueError("Predictions CSV must have Pred_Return & Actual_Return (or head-specific columns).")
    out["Date"] = pd.to_datetime(out["Date"])
    return out.sort_values("Date").reset_index(drop=True)

def _norm_forecast(df: pd.DataFrame, head: str) -> pd.DataFrame:
    """Return Date, Pred_Return."""
    df = df.copy()
    if "Date" not in df.columns: raise ValueError("Forecast CSV missing 'Date'")
    pr_two = f"Pred_{head.capitalize()}_Return"; pp_two = f"Pred_{head.capitalize()}"
    if {"Pred_Return","Pred_Close"}.issubset(df.columns):
        out = df[["Date","Pred_Return","Pred_Close"]].copy()
    elif {pr_two, pp_two}.issubset(df.columns):
        out = df[["Date", pr_two, pp_two]].rename(columns={pr_two:"Pred_Return", pp_two:"Pred_Close"})
    else:
        if "Pred_Return" in df.columns:
            out = df[["Date","Pred_Return"]].copy(); out["Pred_Close"] = np.nan
        elif pr_two in df.columns:
            out = df[["Date", pr_two]].rename(columns={pr_two:"Pred_Return"}); out["Pred_Close"] = np.nan
        else:
            raise ValueError("Forecast CSV must contain Pred_Return (and ideally Pred_Close).")
    out["Date"] = pd.to_datetime(out["Date"])
    return out.sort_values("Date").reset_index(drop=True)

# ---------------- risk helpers ---------------
def _load_q_from_summary(risk_dir: str, ticker: str) -> Optional[float]:
    p = os.path.join(risk_dir, "summary.json")
    if not os.path.exists(p): return None
    try:
        obj = json.load(open(p, "r"))
        r = obj.get(ticker, {})
        cal = r.get("calibration", {})
        q = cal.get("q_abs_ret")
        return float(q) if q is not None else None
    except Exception:
        return None

def _load_predicted_risk_rows(risk_dir: str, ticker: str) -> Optional[pd.DataFrame]:
    p = os.path.join(risk_dir, f"{ticker}_predicted_risk.csv")
    if not os.path.exists(p): return None
    df = pd.read_csv(p)
    if "For_Date" in df.columns: df["Date"] = pd.to_datetime(df["For_Date"])
    elif "Date" in df.columns:   df["Date"] = pd.to_datetime(df["Date"])
    else: return None
    return df

# compute q from predictions if Risk summary not present ---
def _compute_q_from_predictions(pred_df: pd.DataFrame, alpha: float) -> Optional[float]:
    try:
        y  = pred_df["Actual_Return"].astype(float).values
        yh = pred_df["Pred_Return"].astype(float).values
        resid = np.abs(y - yh)
        if len(resid) < 30:
            return None
        lo, hi = np.quantile(resid, [0.01, 0.99])
        resid = np.clip(resid, lo, hi)
        return float(np.quantile(resid, 1 - alpha))
    except Exception:
        return None

# =========================================================
#                       NODES
# =========================================================
def node_load(state: Dict[str, Any]) -> Dict[str, Any]:

    # ticker handling (single ticker for now)
    tickers = state.get("tickers") or ["AAPL"]
    ticker = tickers[0]

    mode     = str(state.get("mode","historical")).lower()
    head     = str(state.get("head","close")).lower()

    pred_dir = _under_run_base(state, state.get("predictions_dir","Predictive_Model/predictions"))
    fcast_dir= _under_run_base(state, state.get("forecasts_dir","Predictive_Model/advanced_forecasts"))
    fe_path  = _under_run_base(state, state.get("features_path","FE_Agent/features_engineered.csv"))
    risk_dir = _under_run_base(state, state.get("risk_dir","Risk_Assessment"))
    out_dir  = _under_run_base(state, state.get("out_dir","Opt_results"))
    ensure_dir(out_dir)

    # try to get q from Risk assessment
    q = _load_q_from_summary(risk_dir, ticker)

    risk_rows = _load_predicted_risk_rows(risk_dir, ticker)

    if mode == "historical":
        ppath = os.path.join(pred_dir, f"{ticker}_test_predictions.csv")
        if not os.path.exists(ppath):
            raise FileNotFoundError(f"Missing predictions: {ppath}")
        pred = _norm_predictions(pd.read_csv(ppath), head=head)
        hist = pred[["Date","Actual_Return"]].copy()
        hist["NextDay_Return"] = hist["Actual_Return"].shift(-1)
        hist["Vol20"] = hist["Actual_Return"].rolling(20).std()

        # fallback q if Risk summary not available
        if q is None:
            alpha = float(state.get("alpha", 0.10))
            q = _compute_q_from_predictions(pred, alpha)

        bundle = {"pred": pred, "hist": hist, "q": q, "risk_rows": None}

    elif mode == "forecast":
        if not os.path.exists(fe_path):
            raise FileNotFoundError(f"Engineered features not found: {fe_path}")
        feats = pd.read_csv(fe_path, parse_dates=["Date"])
        feats = feats[feats["Symbol"] == ticker].sort_values("Date").reset_index(drop=True)
        if len(feats) < 30:
            raise RuntimeError(f"Not enough {ticker} rows in engineered features.")
        fpath = os.path.join(fcast_dir, f"{ticker}_forecast_7d.csv")
        if not os.path.exists(fpath):
            raise FileNotFoundError(f"Missing forecast: {fpath}")
        fc = _norm_forecast(pd.read_csv(fpath), head=head)
        feats["Actual_Return"] = feats["Close"].pct_change()
        feats["Vol20"] = feats["Actual_Return"].rolling(20).std()
        bundle = {"forecast": fc, "hist": feats[["Date","Actual_Return","Vol20"]],
                  "q": q, "risk_rows": risk_rows}
    else:
        raise ValueError("mode must be 'historical' or 'forecast'")

    print(f"[Opt] Loaded mode={mode}, ticker={ticker}, head={head}")
    if q is None:
        print("[Opt] Note: q (uncertainty band) unavailable — SNR gating will only use strong-pct / risk rows.")
    else:
        print(f"[Opt] q_abs_ret={q:.6f}")

    return {**state, "ticker": ticker, "bundle": bundle, "head": head, "out_dir": out_dir, "status":"loaded"}

def node_signals(state: Dict[str, Any]) -> Dict[str, Any]:

    mode = state.get("mode","historical").lower()
    allow_shorts = bool(state.get("allow_shorts", False))
    w_max   = float(state.get("w_max", 0.30))
    snr_min = float(state.get("snr_min", 1.0))
    strong_pct = int(state.get("strong_pct", 75))

    b = state["bundle"]
    rows = []

    if mode == "historical":
        df = b["pred"].copy()  # Date, Pred_Return, Actual_Return
        hist = b["hist"].copy()
        q = b.get("q", None)

        # strong-signal threshold from |Pred_Return|
        x = np.abs(df["Pred_Return"].values)
        thr = float(np.percentile(x, strong_pct)) if len(x) >= 10 else 0.0

        for _, r in df.iterrows():
            dt = pd.to_datetime(r["Date"])
            r_pred = float(r["Pred_Return"])
            vol = hist[hist["Date"] <= dt]["Vol20"].dropna()
            vol = float(vol.iloc[-1]) if len(vol) else float(hist["Vol20"].median() or 1e-6)

            passes = True
            if q is None:
                passes = (abs(r_pred) >= thr)
            else:
                snr = abs(r_pred) / max(q, 1e-12)
                crosses_zero = (r_pred - q) <= 0.0 <= (r_pred + q)
                passes = (snr >= snr_min) and (not crosses_zero) and (abs(r_pred) >= thr)

            if not passes or vol == 0 or np.isnan(vol):
                w = 0.0
            else:
                w = r_pred / vol
                if not allow_shorts and w < 0: w = 0.0
                w = float(np.clip(w, -w_max, w_max))
            rows.append({"Date": dt, "Weight": w})

    else:  # forecast
        fc = b["forecast"].copy()   # Date, Pred_Return (+ Pred_Close)
        hist = b["hist"].copy()
        risk_rows = b.get("risk_rows")
        q = b.get("q", None)

        for _, r in fc.iterrows():
            dt = pd.to_datetime(r["Date"])
            r_pred = float(r["Pred_Return"])
            vol = hist[hist["Date"] <= dt]["Vol20"].dropna()
            vol = float(vol.iloc[-1]) if len(vol) else float(hist["Vol20"].median() or 1e-6)

            passes = True
            if isinstance(risk_rows, pd.DataFrame):
                m = risk_rows[risk_rows["Date"] == dt]
                if not m.empty:
                    lbl = str(m["Risk_Label"].iloc[0]) if "Risk_Label" in m.columns else None
                    crosses = bool(m["CrossesZero"].iloc[0]) if "CrossesZero" in m.columns else None
                    snr_val = float(m["SNR"].iloc[0]) if "SNR" in m.columns else None
                    if lbl and lbl.upper() == "HIGH": passes = False
                    if crosses is True:               passes = False
                    if (snr_val is not None) and (snr_val < snr_min): passes = False
                elif q is not None:
                    snr = abs(r_pred) / max(q, 1e-12)
                    crosses_zero = (r_pred - q) <= 0.0 <= (r_pred + q)
                    passes = (snr >= snr_min) and (not crosses_zero)
            elif q is not None:
                snr = abs(r_pred) / max(q, 1e-12)
                crosses_zero = (r_pred - q) <= 0.0 <= (r_pred + q)
                passes = (snr >= snr_min) and (not crosses_zero)

            if not passes or vol == 0 or np.isnan(vol):
                w = 0.0
            else:
                w = r_pred / vol
                if not allow_shorts and w < 0: w = 0.0
                w = float(np.clip(w, -w_max, w_max))
            rows.append({"Date": dt, "Weight": w})

    W = pd.DataFrame(rows).set_index("Date").sort_index()
    print(f"[Opt] Built weights: {len(W)} rows (nonzero {int((W['Weight']!=0).sum())})")
    return {**state, "weights": W, "status":"signals_built"}

def node_simulate(state: Dict[str, Any]) -> Dict[str, Any]:

    cost_rate = float(state.get("cost_bps", 0.001))   # e.g., 10 bps per unit turnover
    alignment = str(state.get("alignment","next_day")).lower()

    W = state["weights"].copy()              # index Date, col Weight
    b = state["bundle"]
    if "NextDay_Return" in b["hist"].columns:
        R = b["hist"][["Date","NextDay_Return"]].copy().set_index("Date").rename(columns={"NextDay_Return":"Ret"})
    else:
        # forecast fallback
        H = b["hist"].copy()
        H["NextDay_Return"] = H["Actual_Return"].shift(-1)
        R = H[["Date","NextDay_Return"]].set_index("Date").rename(columns={"NextDay_Return":"Ret"})

    idx = W.index.intersection(R.index)
    if len(idx) == 0:
        equity_df = pd.DataFrame(columns=["Equity","DailyReturn"], index=W.index)
        print("[Opt] No overlap between weights and returns — empty equity curve.")
        return {**state, "equity_df": equity_df, "weights_used": W, "status":"backtested"}

    W = W.loc[idx]
    R = R.loc[idx]

    W_used = W.shift(1) if alignment == "next_day" else W
    W_used = W_used.fillna(0.0)

    turnover = W.diff().abs().fillna(0.0)["Weight"]
    gross = (W_used["Weight"] * R["Ret"])
    costs = turnover * cost_rate
    net = gross - costs

    equity = (1.0 + net).cumprod()
    equity_df = pd.DataFrame({"Equity": equity, "DailyReturn": net}, index=idx)
    print(f"[Opt] Backtest built: {len(equity_df)} days, equity {equity_df['Equity'].iloc[-1]:.4f}")
    return {**state, "equity_df": equity_df, "weights_used": W_used, "status":"backtested"}

def node_save_min(state: Dict[str, Any]) -> Dict[str, Any]:

    out_dir = state.get("out_dir","Opt_results"); ensure_dir(out_dir)
    go_min_weight = float(state.get("go_min_weight", 0.10))  # threshold to act
    ticker = state.get("ticker","AAPL")

    # Build GO/NO-GO series from weights_used
    W = state["weights_used"].copy() if "weights_used" in state else pd.DataFrame()
    if not W.empty and "Weight" in W.columns:
        go_series = np.where(np.abs(W["Weight"].values) >= go_min_weight, "GO", "NO-GO")
        go_idx = W.index
    else:
        go_series = np.array([], dtype=str)
        go_idx = pd.DatetimeIndex([])

    # Compose equity_curve.csv
    ec = state["equity_df"].copy() if "equity_df" in state else pd.DataFrame()
    if not ec.empty and len(go_series):
        go_t = pd.Series(go_series, index=go_idx).reindex(ec.index).fillna("NO-GO")
        ec_out = ec.copy()

        ec_out[f"GO_{ticker}"] = go_t.values
    elif not ec.empty:
        ec_out = ec.copy()
        ec_out[f"GO_{ticker}"] = "NO-GO"
    else:
        ec_out = pd.DataFrame(columns=["Equity","DailyReturn",f"GO_{ticker}"])

    ec_out.reset_index().rename(columns={"index":"Date"}).to_csv(
        os.path.join(out_dir, "equity_curve.csv"), index=False
    )

    # Latest position/decision/go-no-go
    if not W.empty:
        last_dt = W.index.max()
        last_w = float(W.loc[last_dt, "Weight"])
        last_decision = "LONG" if last_w > 0 else ("SHORT" if last_w < 0 else "FLAT")
        last_go = "GO" if abs(last_w) >= go_min_weight else "NO-GO"
    else:
        last_dt, last_w, last_decision, last_go = None, 0.0, "FLAT", "NO-GO"

    # KPIs
    if "Date" in ec_out.columns and len(ec_out) > 0:
        eq_series = ec_out.set_index("Date")["Equity"]
        dr_series = ec_out.set_index("Date")["DailyReturn"]
    else:
        eq_series = ec_out["Equity"]
        dr_series = ec_out["DailyReturn"]

    k = kpis(eq_series, dr_series)

    summary = {
        "ticker": ticker,
        "kpis": k,
        "params": {
            "mode": state.get("mode","historical"),
            "head": state.get("head","close"),
            "alignment": state.get("alignment","next_day"),
            "allow_shorts": bool(state.get("allow_shorts", False)),
            "w_max": float(state.get("w_max", 0.30)),
            "snr_min": float(state.get("snr_min", 1.0)),
            "strong_pct": int(state.get("strong_pct", 75)),
            "cost_bps": float(state.get("cost_bps", 0.001)),
            "go_min_weight": go_min_weight
        },
        "window": {
            "start": str(ec_out["Date"].min()) if "Date" in ec_out.columns and len(ec_out)>0 else None,
            "end":   str(ec_out["Date"].max()) if "Date" in ec_out.columns and len(ec_out)>0 else None
        },
        "latest": {
            "timestamp": str(last_dt) if last_dt is not None else None,
            "weight": last_w,
            "decision": last_decision,  # LONG / SHORT / FLAT
            "go_nogo": last_go          # GO / NO-GO
        }
    }
    json.dump(summary, open(os.path.join(out_dir, "summary.json"), "w"), indent=2)

    print(f"[Opt] Saved: equity_curve.csv  & summary.json → {out_dir}")
    print(f"[Opt] Last: {summary['latest']}")
    return {**state, "status":"saved"}

# ---------------- workflow glue ----------------
def build_workflow():
    if not HAS_LANGGRAPH:
        raise RuntimeError("LangGraph not available. Install it or call nodes manually.")
    g = StateGraph(dict)
    g.add_node("load",     node_load)
    g.add_node("signals",  node_signals)
    g.add_node("simulate", node_simulate)
    g.add_node("save",     node_save_min)

    g.set_entry_point("load")
    g.add_edge("load", "signals")
    g.add_edge("signals", "simulate")
    g.add_edge("simulate", "save")
    g.set_finish_point("save")
    return g.compile()


def build_optimization_workflow():
    g = StateGraph(dict)
    g.add_node("load", node_load)
    g.add_node("signals", node_signals)
    g.add_node("simulate", node_simulate)
    g.add_node("save", node_save_min)
    g.set_entry_point("load")
    g.add_edge("load","signals")
    g.add_edge("signals","simulate")
    g.add_edge("simulate","save")
    g.set_finish_point("save")
    return g.compile()

# --------------------- run ---------------------
if __name__ == "__main__":
    print(" Running Optimization Agent…")
    app = build_optimization_workflow()
    _ = app.invoke({
        "run_base": "/content/drive/MyDrive/A2A_prediction_system/RUN_XXXXXXXX_XXXXXX",
        "tickers": ["AAPL"],
        "mode": "historical",
        "head": "close",
        "predictions_dir": "Predictive_Model/predictions",
        "forecasts_dir": "Predictive_Model/advanced_forecasts",
        "features_path": "FE_Agent/features_engineered.csv",
        "risk_dir": "Risk_Assessment",

        # risk gating & sizing
        "allow_shorts": False,                # True = long/short
        "w_max": 0.30,
        "snr_min": 1.0,
        "strong_pct": 75,
        "alignment": "next_day",
        "cost_bps": 0.001,                    # 10 bps per unit turnover
        "go_min_weight": 0.10,                # threshold for GO / NO-GO

        "alpha": 0.10,                        # used only if q must be computed from predictions
        "out_dir": "Opt_results"
    })



**Summerizer_agent.py**

In [ ]:
%%writefile /content/drive/MyDrive/A2A_prediction_system/backend/a2a/summarizer_agent.py
import os, json
import numpy as np
import pandas as pd
from typing import Dict, Any, List

# LangGraph
try:
    from langgraph.graph import StateGraph
    HAS_LG = True
except Exception:
    HAS_LG = False

# ------------------ helpers ------------------
def _ensure_dir(p: str):
    os.makedirs(p, exist_ok=True)

def _load_json(path: str) -> Any:
    try:
        with open(path, "r") as f:
            return json.load(f)
    except Exception:
        return None

def _load_csv(path: str) -> pd.DataFrame:
    try:
        return pd.read_csv(path)
    except Exception:
        return pd.DataFrame()

def _pick_run_base(state: Dict[str, Any]) -> str:
    return state.get("run_base") or "."

def _default_paths(run_base: str) -> Dict[str, str]:

    return {
        "predictions_dir":       os.path.join(run_base, "Predictive_Model", "predictions"),
        "models_dir":            os.path.join(run_base, "Predictive_Model", "lstm_models"),
        "eval_summary_json":     os.path.join(run_base, "Model_Evaluation", "summaries", "model_evaluation_summary.json"),


        "desc_summary_json":     os.path.join(run_base, "Descriptive", "summary.json"),

        # risk / opt
        "risk_summary_json":     os.path.join(run_base, "Risk_Assessment", "summary.json"),
        "opt_summary_json":      os.path.join(run_base, "Opt_results", "summary.json"),

        # explainability
        "xai_summary_json":      os.path.join(run_base, "Explainability", "xai_summary.json"),
        "ig_dir":                os.path.join(run_base, "Explainability", "IG_XAI"),
        "shap_dir":              os.path.join(run_base, "Explainability", "SHAP_XAI"),

        # plots from agents
        "eval_plots_dir":        os.path.join(run_base, "Model_Evaluation", "plots"),
        "risk_plots_dir":        os.path.join(run_base, "Risk_Assessment", "plots"),

        # final report
        "out_dir":               os.path.join(run_base, "reports"),
        "out_payload_json":      os.path.join(run_base, "reports", "final_payload.json"),
        "out_assets_json":       os.path.join(run_base, "reports", "assets.json"),
    }

def _discover_tickers(predictions_dir: str, tickers: List[str] = None) -> List[str]:
    if tickers:
        return sorted(list({t.upper() for t in tickers}))
    if not os.path.isdir(predictions_dir):
        return []
    toks = []
    for fn in os.listdir(predictions_dir):
        if fn.endswith("_test_predictions.csv"):
            toks.append(fn.split("_")[0].upper())
    return sorted(list(set(toks)))

def _safe_get(d: dict, *path, default=None):
    cur = d
    try:
        for k in path:
            if cur is None:
                return default
            cur = cur[k]
        return cur if cur is not None else default
    except Exception:
        return default

def _latest_pred_row(pred_csv: str) -> dict:
    df = _load_csv(pred_csv)
    if df.empty or "Date" not in df.columns:
        return {}
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
    df = df.dropna(subset=["Date"]).sort_values("Date")
    return df.tail(1).to_dict(orient="records")[0] if len(df) else {}

def _eval_json_to_dict(obj: Any) -> Dict[str, Any]:
    """
    Evaluation agent writes a LIST of rows.
    Convert to { TICKER: row } for easy lookup.
    """
    if isinstance(obj, list):
        out = {}
        for row in obj:
            t = row.get("Ticker") or row.get("ticker")
            if t:
                out[str(t).upper()] = row
        return out
    return obj if isinstance(obj, dict) else {}

def _compute_strong_thr_from_preds(pred_csv: str, pct: int = 75) -> float:

    df = _load_csv(pred_csv)
    if df.empty:
        return 0.002
    if "Pred_Return" not in df.columns:
        if "Pred_Close_Return" in df.columns:
            df = df.rename(columns={"Pred_Close_Return": "Pred_Return"})
        elif "Pred_Open_Return" in df.columns:
            df = df.rename(columns={"Pred_Open_Return": "Pred_Return"})
    if "Pred_Return" not in df.columns:
        return 0.002
    try:
        thr = float(np.percentile(np.abs(df["Pred_Return"].astype(float).values), pct))
        return thr
    except Exception:
        return 0.002

def _derive_decision_for_ticker(t: str,
                                opt_json: dict,
                                risk_json: dict,
                                desc_json: dict,
                                pred_csv_path: str) -> Dict[str, Any]:

    go = _safe_get(opt_json, "latest", "go_nogo")
    w  = _safe_get(opt_json, "latest", "weight", default=None)
    reason = None

    if go is None:
        risk_label = _safe_get(risk_json, t, "risk", "Risk_Label")
        thr = _compute_strong_thr_from_preds(pred_csv_path, 75)

        last = _latest_pred_row(pred_csv_path)
        pred_ret = None
        for key in ["Pred_Return", "Pred_Close_Return", "Pred_Open_Return"]:
            if key in last:
                try:
                    pred_ret = float(last[key]); break
                except Exception:
                    pass

        if pred_ret is None:
            go, reason = "NO-GO", "No latest Pred_Return available"
        else:
            if str(risk_label).upper() == "HIGH":
                go, reason = "NO-GO", f"Risk {risk_label}"
            elif abs(pred_ret) < thr:
                go, reason = "NO-GO", f"|Pred_Return|<{thr:.4f}"
            else:
                go, reason = "GO", f"|Pred_Return|≥{thr:.4f}"
        w = None if go == "NO-GO" else 1.0

    return {"go_nogo": go, "weight": w, "reason": reason}

def _find_first(paths: List[str]) -> str:
    for p in paths:
        if p and os.path.exists(p):
            return p
    return ""

def _plots_for_ticker(paths: Dict[str, str], tslug: str) -> Dict[str, str]:

    out = {}
    tU, tl = tslug.upper(), tslug.lower()

    # Model Evaluation line/scatter (close head)
    cand_line = [
        os.path.join(paths["eval_plots_dir"], f"{tU}_close_price_line.png"),
        os.path.join(paths["eval_plots_dir"], f"{tl}_close_price_line.png"),
        os.path.join(paths["eval_plots_dir"], f"{tU}_price_line.png"),
        os.path.join(paths["eval_plots_dir"], f"{tl}_price_line.png"),
    ]
    cand_scatter = [
        os.path.join(paths["eval_plots_dir"], f"{tU}_close_price_scatter.png"),
        os.path.join(paths["eval_plots_dir"], f"{tl}_close_price_scatter.png"),
        os.path.join(paths["eval_plots_dir"], f"{tU}_price_scatter.png"),
        os.path.join(paths["eval_plots_dir"], f"{tl}_price_scatter.png"),
    ]
    p_line = _find_first(cand_line)
    p_scatter = _find_first(cand_scatter)
    if p_line:    out["actual_vs_pred_line"] = p_line
    if p_scatter: out["actual_vs_pred_scatter"] = p_scatter

    # Risk plots
    rp = paths.get("risk_plots_dir")
    for name, fnameU, fnamel in [
        ("drawdown",      f"{tU}_drawdown.png",      f"{tl}_drawdown.png"),
        ("rolling_vol",   f"{tU}_rolling_vol.png",   f"{tl}_rolling_vol.png"),
        ("tomorrow_band", f"{tU}_tomorrow_band.png", f"{tl}_tomorrow_band.png"),
        ("fanchart",      f"{tU}_fan_chart.png",     f"{tl}_fan_chart.png"),
    ]:
        p = _find_first([os.path.join(rp, fnameU), os.path.join(rp, fnamel)])
        if p:
            out[f"risk_{name}"] = p

    # ---- XAI (IG & SHAP) ----
    # IG (heatmap + importance)
    ig_heat = _find_first([
        os.path.join(paths["ig_dir"],   tU, "ig_heatmap_close.png"),
        os.path.join(paths["ig_dir"],   tl, "ig_heatmap_close.png"),
        os.path.join(paths["ig_dir"],   tU, "AAPL_heatmap_close.png"),
        os.path.join(paths["ig_dir"],   tl, "AAPL_heatmap_close.png"),
    ])
    ig_import = _find_first([
        os.path.join(paths["ig_dir"],   tU, "ig_global_importance_close.png"),
        os.path.join(paths["ig_dir"],   tl, "ig_global_importance_close.png"),
    ])

    # SHAP (global importance / beeswarm / local)
    shap_global = _find_first([
        os.path.join(paths["shap_dir"], tU, f"{tU}_global_importance_close.png"),
        os.path.join(paths["shap_dir"], tl, f"{tl}_global_importance_close.png"),
    ])
    shap_bees = _find_first([
        os.path.join(paths["shap_dir"], tU, f"{tU}_summary_beeswarm_close.png"),
        os.path.join(paths["shap_dir"], tl, f"{tl}_summary_beeswarm_close.png"),
        os.path.join(paths["shap_dir"], tU, "AAPL_summary_beeswarm_close.png"),
        os.path.join(paths["shap_dir"], tl, "AAPL_summary_beeswarm_close.png"),
    ])
    shap_local = _find_first([
        os.path.join(paths["shap_dir"], tU, f"{tU}_local_waterfall_last_close.png"),
        os.path.join(paths["shap_dir"], tl, f"{tl}_local_waterfall_last_close.png"),
        os.path.join(paths["shap_dir"], tU, "AAPL_local_waterfall_last_close.png"),
        os.path.join(paths["shap_dir"], tl, "AAPL_local_waterfall_last_close.png"),
    ])

    # What to expose to the Report Agent
    if ig_heat:    out["ig_heatmap"]   = ig_heat
    if ig_import:  out["ig_importance"] = ig_import
    if shap_global: out["shap_bar"]    = shap_global
    elif shap_bees: out["shap_bar"]    = shap_bees
    if shap_local:  out["shap_local"]  = shap_local

    return out

# ------------------ Sharpe & MaxDD from risk json ------------------
def _extract_sharpe_and_mdd(risk_json_for_ticker: dict) -> Dict[str, Any]:
    """
    Try multiple common key paths/names for Sharpe (annualised) and Max Drawdown.
    Returns dict with keys: {'sharpe_ann': float|None, 'max_drawdown': float|None}
    """
    # Check nested 'kpis'
    sharpe = _safe_get(risk_json_for_ticker, "kpis", "Sharpe_ann")
    if sharpe is None: sharpe = _safe_get(risk_json_for_ticker, "kpis", "Sharpe")
    if sharpe is None: sharpe = _safe_get(risk_json_for_ticker, "risk", "Sharpe_ann")
    if sharpe is None: sharpe = _safe_get(risk_json_for_ticker, "Sharpe_ann")
    if sharpe is None: sharpe = _safe_get(risk_json_for_ticker, "Sharpe")
    if sharpe is None: sharpe = _safe_get(risk_json_for_ticker, "SharpeRatio")

    mdd = _safe_get(risk_json_for_ticker, "kpis", "Max_Drawdown")
    if mdd is None: mdd = _safe_get(risk_json_for_ticker, "risk", "Max_Drawdown")
    if mdd is None: mdd = _safe_get(risk_json_for_ticker, "Max_Drawdown")
    if mdd is None: mdd = _safe_get(risk_json_for_ticker, "max_drawdown")
    if mdd is None: mdd = _safe_get(risk_json_for_ticker, "MDD")

    # Cast to floats where possible
    try: sharpe = None if sharpe is None else float(sharpe)
    except: sharpe = None
    try: mdd = None if mdd is None else float(mdd)
    except: mdd = None

    return {"sharpe_ann": sharpe, "max_drawdown": mdd}

# ------------------ LangGraph nodes ------------------
def node_collect(state: Dict[str, Any]) -> Dict[str, Any]:
    run_base = _pick_run_base(state)
    paths = _default_paths(run_base)

    # allow overrides
    for k, v in (state.get("paths") or {}).items():
        paths[k] = v

    tickers = _discover_tickers(paths["predictions_dir"], state.get("tickers"))

    # load json artifacts
    eval_js_raw = _load_json(paths["eval_summary_json"]) or []
    eval_js     = _eval_json_to_dict(eval_js_raw)  # normalize to dict by ticker
    desc_js     = _load_json(paths["desc_summary_json"]) or {}
    risk_js     = _load_json(paths["risk_summary_json"]) or {}
    opt_js      = _load_json(paths["opt_summary_json"])  or {}
    xai_js      = _load_json(paths["xai_summary_json"])  or {}

    _ensure_dir(paths["out_dir"])

    return {
        **state,
        "paths": paths,
        "tickers": tickers,
        "eval_json": eval_js,
        "desc_json": desc_js,
        "risk_json": risk_js,
        "opt_json": opt_js,
        "xai_json": xai_js,
        "status": "collected"
    }

def node_summarize(state: Dict[str, Any]) -> Dict[str, Any]:
    tickers = state["tickers"]
    P = state["paths"]

    # labels
    ui_labels = {
        "prediction_accuracy": "Prediction Accuracy",   # R², RMSE, MAPE
        "trend_prediction":    "Trend Prediction",      # Directional Accuracy %
        "statistical_insights":"Statistical Insights",  # Sharpe (ann.) and Max Drawdown
        "risk_assessment":     "Risk Assessment",       # Label, SNR, Crosses Zero
        "model_explainability":"Model Explainability",  # SHAP / IG status
        "final_recommendation":"Final Recommendation",  # Go / No-Go
    }

    out = {
        "as_of": None,
        "run_base": _pick_run_base(state),
        "tickers": tickers,
        "ui_labels": ui_labels,
        "per_ticker": {}
    }

    for t in tickers:
        tslug = t.upper()

        # evaluation
        e = state["eval_json"].get(tslug, {})

        # risk (summary per ticker)
        r_all = state["risk_json"] if isinstance(state["risk_json"], dict) else {}
        r_t   = r_all.get(tslug, {}) if isinstance(r_all, dict) else {}
        # Extract Sharpe & MaxDD
        kpis  = _extract_sharpe_and_mdd(r_t)

        # opt (single summary)
        o = state["opt_json"] or {}

        # Pred file path
        pred_csv = os.path.join(P["predictions_dir"], f"{tslug}_test_predictions.csv")
        decision = _derive_decision_for_ticker(tslug, o, state["risk_json"], state["desc_json"], pred_csv)

        # assemble
        per = {
            "ticker": tslug,
            "head": "close",

            # --- Prediction Accuracy ---
            "evaluation": {
                "R2_price": e.get("R2(price)"),
                "RMSE_price": e.get("RMSE_price"),
                "MAE_price": e.get("MAE_price"),
                "MAPE_pct": e.get("MAPE"),
                "DirAcc_pct": e.get("DirAcc(%)") or e.get("DirAcc"),
                "Rows": e.get("Rows"),
                "Test_Start": e.get("Test_Start"),
                "Test_End": e.get("Test_End"),
            },

            # --- Statistical Insights  ---
            "statistical": {
                "Sharpe_ann": kpis.get("sharpe_ann"),      # annualised Sharpe
                "Max_Drawdown": kpis.get("max_drawdown"),  # max drawdown
            },

            # --- Risk Assessment ---
            "risk": {
                "label": _safe_get(r_t, "risk", "Risk_Label"),
                "snr_latest": _safe_get(r_t, "risk", "SNR"),
                "crosses_zero_latest": _safe_get(r_t, "risk", "Crosses_Zero"),
            },

            # --- Optimization / Decision ---
            "optimization": {
                "decision": _safe_get(state["opt_json"], "latest", "go_nogo"),
                "weight": _safe_get(state["opt_json"], "latest", "weight"),
                "kpis": state["opt_json"].get("kpis", {})
            },

            # XAI summary
            "xai": {"ig_top": None, "shap_top": None, "notes": "Populate if you persist a combined XAI summary."},

            # Final decision
            "decision": decision,

            # Collect plots
            "plots": _plots_for_ticker(P, tslug)
        }

        out["per_ticker"][tslug] = per
        if out["as_of"] is None:

            out["as_of"] = e.get("Test_End")

    return {**state, "payload": out, "status": "summarized"}

def node_save(state: Dict[str, Any]) -> Dict[str, Any]:
    P = state["paths"]
    _ensure_dir(P["out_dir"])
    with open(P["out_payload_json"], "w") as f:
        json.dump(state["payload"], f, indent=2)

    assets = {t: state["payload"]["per_ticker"][t]["plots"] for t in state["payload"]["per_ticker"]}
    with open(P["out_assets_json"], "w") as f:
        json.dump(assets, f, indent=2)

    print(f"Summarizer wrote:\n  - {P['out_payload_json']}\n  - {P['out_assets_json']}")
    return {**state, "status": "saved", "out_payload": P["out_payload_json"], "out_assets": P["out_assets_json"]}

# ------------------ Build workflow ------------------
def build_summarizer_workflow():
    g = StateGraph(dict)
    g.add_node("collect", node_collect)
    g.add_node("summarize", node_summarize)
    g.add_node("save", node_save)
    g.set_entry_point("collect")
    g.add_edge("collect", "summarize")
    g.add_edge("summarize", "save")
    g.set_finish_point("save")
    return g.compile()

# ------------------ Standalone run ------------------
if __name__ == "__main__":
    print(" Running Agent Summarizer…")
    app = build_summarizer_workflow()
    _ = app.invoke({
        "run_base": "/content/drive/MyDrive/A2A_prediction_system/RUN_XXXXXXXX_XXXXXX",
        "tickers": ["AAPL"],
    })




**Report_agent.py**

In [ ]:
%%writefile /content/drive/MyDrive/A2A_prediction_system/backend/a2a/report_agent.py

# backend/a2a/report_agent.py
import os, json, glob, datetime, base64
from typing import Dict, Any, List, Optional

# ---------------------------- utils ----------------------------
def _ensure_dir(p: str): os.makedirs(p, exist_ok=True)

def _safe_get(d: dict, *path, default=None):
    cur = d
    for k in path:
        if cur is None:
            return default
        if isinstance(cur, dict):
            cur = cur.get(k)
        elif isinstance(cur, list) and isinstance(k, int):
            cur = cur[k] if 0 <= k < len(cur) else None
        else:
            return default
    return default if cur is None else cur

def _dash(msg: str | None = None, html: bool = True) -> str:
    if html:
        tip = (msg or "not computed").replace("'", "\\'")
        return f"<span title='{tip}'>—</span>"
    return "—"

def _fmt_str(x: Optional[str], html: bool = True) -> str:
    s = (x or "").strip()
    return s if s else _dash(html=html)

def _fmt_bool(x, html: bool = True) -> str:
    if x is True:  return "True"
    if x is False: return "False"
    return _dash("not available", html=html)

def _fmt_num(x, digits=3, money=False):
    try:
        v = float(x)
    except Exception:
        return "—"
    return (f"${v:.{digits}f}" if money else f"{v:.{digits}f}")

def _decision_badge(decision: Optional[str]) -> str:
    d = (decision or "").strip().upper()
    if d == "GO":    return "<b>GO</b>"
    if d == "NO-GO": return "<b>NO-GO</b>"
    return "—"

def _autodetect_run_base(run_base: str) -> str:
    if os.path.exists(os.path.join(run_base, "reports", "final_payload.json")):
        return run_base
    candidates = []
    for p in glob.glob(os.path.join("runs", "*", "reports", "final_payload.json")):
        try:
            mtime = os.path.getmtime(p)
            base = os.path.dirname(os.path.dirname(p))
            candidates.append((mtime, base))
        except Exception:
            pass
    if candidates:
        candidates.sort()
        return candidates[-1][1]
    return run_base

def _img_data_uri(path: str) -> Optional[str]:
    if not path or not os.path.exists(path):
        return None
    try:
        with open(path, "rb") as f:
            b64 = base64.b64encode(f.read()).decode("ascii")
        return f"data:image/png;base64,{b64}"
    except Exception:
        return None

# ---------------------------- main render ----------------------------
def render_report(run_base: str = ".", out_name: str = "llm_report.md",
                  order: Optional[List[str]] = None) -> str:

    run_base = _autodetect_run_base(run_base)

    reports_dir = os.path.join(run_base, "reports")
    payload_path = os.path.join(reports_dir, "final_payload.json")
    assets_path  = os.path.join(reports_dir, "assets.json")

    if not os.path.exists(payload_path):
        raise FileNotFoundError(
            f"Missing {payload_path}. Run the Summarizer agent for the same run_base first."
        )

    payload = json.load(open(payload_path, "r"))
    assets  = json.load(open(assets_path, "r")) if os.path.exists(assets_path) else {}

    as_of = payload.get("as_of") or datetime.date.today().isoformat()
    tickers = order or payload.get("tickers") or list((_safe_get(payload, "per_ticker") or {}).keys())
    tickers = [t.upper() for t in tickers]

    # ---- UI labels
    ui = payload.get("ui_labels") or {
        "prediction_accuracy": "Prediction Accuracy",
        "trend_prediction":    "Trend Prediction",
        "statistical_insights":"Statistical Insights",
        "risk_assessment":     "Risk Assessment",
        "model_explainability":"Model Explainability",
        "final_recommendation":"Final Recommendation",
    }

    # Helper to choose a plot path from candidates, with fuzzy/glob + path resolution
    def _get_plot(node_plots: Dict[str, str], asset_plots: Dict[str, str], key_candidates: List[str]) -> Optional[str]:
        def _resolve(p: Optional[str]) -> Optional[str]:
            if not p:
                return None
            for cand in (
                p,
                os.path.join(reports_dir, p),
                os.path.join(run_base, p),
            ):
                if os.path.exists(cand):
                    return cand
            return None

        # exact keys
        for k in key_candidates:
            p = (node_plots or {}).get(k) or (asset_plots or {}).get(k)
            rp = _resolve(p)
            if rp:
                return rp

        # fuzzy on dict keys
        def _match_any(d: Dict[str, str], must_tokens: List[str]) -> Optional[str]:
            for k, v in (d or {}).items():
                lab = (k or "").lower()
                if all(tok in lab for tok in must_tokens):
                    rp = _resolve(v)
                    if rp:
                        return rp
            return None

        toks_sets = []
        # infer token sets from candidates provided
        cand_l = " ".join(key_candidates).lower()
        if "ig" in cand_l:
            toks_sets += [["ig", "heatmap"], ["ig", "importance"]]
        if "shap" in cand_l:
            toks_sets += [["shap", "beeswarm"], ["shap", "importance"], ["shap", "global"]]

        for dsrc in (node_plots or {}), (asset_plots or {}):
            for toks in toks_sets:
                hit = _match_any(dsrc, toks)
                if hit:
                    return hit

        # glob search on disk for common patterns
        import glob as _glob
        def _glob_first(patterns: List[str]) -> Optional[str]:
            roots = [reports_dir, os.path.join(run_base, "reports"), run_base]
            for root in roots:
                for pat in patterns:
                    hits = _glob.glob(os.path.join(root, "**", pat), recursive=True)
                    if hits:
                        return hits[0]
            return None

        patterns = []
        if "ig" in cand_l:
            patterns += ["*ig*heatmap*.png", "*ig*importance*.png"]
        if "shap" in cand_l:
            patterns += ["*shap*beeswarm*.png", "*shap*importance*.png", "*global*importance*shap*.png"]

        patterns += [
            "ig_heatmap_close.png",
            "ig_global_importance_close.png",
            "*_summary_beeswarm_close.png",
            "*_local_waterfall_last_close.png",
            "*_global_importance_close.png",
        ]
        ghit = _glob_first(patterns)
        if ghit:
            return ghit

        return None

    make_html = out_name.lower().endswith(".html")
    blocks: List[str] = []

    # ---------- Header ----------
    if make_html:
        blocks.append("""<!doctype html>
<html><head><meta charset="utf-8">
<title>Stock Insights Report</title>
<style>
 body{font-family:system-ui,-apple-system,Segoe UI,Roboto,Helvetica,Arial,sans-serif; line-height:1.5; padding:24px; max-width:960px; margin:auto;}
 h1,h2{margin:0.6em 0;}
 .muted{color:#666}
 .section{margin:24px 0;}
 img{max-width:100%; height:auto; border:1px solid #eee; border-radius:8px; padding:4px; background:#fff;}
 details{margin:8px 0 16px 0;}
 table{border-collapse:collapse; width:100%; font-size:14px;}
 th,td{border:1px solid #ddd; padding:6px 8px;}
 th{background:#fafafa; text-align:left;}
 .hr{border-top:1px solid #eee; margin:24px 0;}
</style>
</head><body>
""")
        blocks.append(f"<h1>Stock Insights Report</h1>")
        blocks.append(f"<p class='muted'>Report Date <b>{as_of}</b></p>")
        blocks.append("<p><i>Note: This report summarizes model predictions and risk metrics. It is informational and <b>not investment advice</b>.</i></p>")
    else:
        blocks.append("# Stock Insights Report\n")
        blocks.append(f"_As of **{as_of}**_\n")
        blocks.append("> **Note:** This report summarizes model predictions and risk metrics. It is informational and **not investment advice**.\n")

    # ---------- Per-ticker sections ----------
    for t in tickers:
        node = _safe_get(payload, "per_ticker", t) or {}
        eval_ = node.get("evaluation", {}) or {}
        stat_ = node.get("statistical", {}) or {}
        risk_ = node.get("risk", {}) or {}
        opt_  = node.get("optimization", {}) or {}
        final = node.get("decision", {}) or {}
        node_plots = node.get("plots", {}) or {}
        asset_plots = assets.get(t, {}) if isinstance(assets, dict) else {}

        r2     = _fmt_num(_safe_get(eval_, "R2_price"), 3)
        rmse   = _fmt_num(_safe_get(eval_, "RMSE_price"), 3, money=True)
        mape   = _fmt_num(_safe_get(eval_, "MAPE_pct"), 2)
        diracc = _fmt_num(_safe_get(eval_, "DirAcc_pct"), 1)
        rows   = _safe_get(eval_, "Rows", default="—")

        # Risk
        html_mode = make_html
        raw_label = risk_.get("label")
        risk_label = (raw_label.upper() if raw_label else "—")
        snr        = _fmt_num(risk_.get("snr_latest"), 2)
        cz         = _fmt_bool(risk_.get("crosses_zero_latest"), html=html_mode)

        # Decision
        raw_decision = final.get("go_nogo") or opt_.get("decision")
        decision     = (raw_decision or "NO-GO")
        weight       = final.get("weight") if final.get("weight") is not None else opt_.get("weight")
        reason       = _fmt_str(final.get("reason") or "", html=html_mode)

        # SHAP / IG plots existence
        p_shap = _get_plot(node_plots, asset_plots, ["shap_bar", "xai_shap_bar", "shap", "beeswarm", "importance"])
        p_ig   = _get_plot(node_plots, asset_plots, ["ig_heatmap", "xai_ig_heatmap", "ig", "importance", "heatmap"])
        shap_status = "Shown" if p_shap else _dash("SHAP skipped or not available", html=html_mode)
        ig_status   = "Shown" if p_ig   else _dash("IG not available", html=html_mode)

        # Section title + executive summary
        if make_html:
            blocks.append(f"<div class='section'><h2>{t}</h2>")
            blocks.append("<h3>Executive Summary</h3>")
            blocks.append(
                f"<ul>"
                f"<li><b>{ui['prediction_accuracy']}</b>: R² <b>{r2}</b>, RMSE <b>{rmse}</b>, MAPE <b>{mape}%</b> (N={rows}).</li>"
                f"<li><b>{ui['risk_assessment']}</b>: label <b>{risk_label}</b>, SNR <b>{snr}</b>, crosses-zero=<code>{cz}</code>.</li>"
                f"<li><b>{ui['final_recommendation']}</b>: " + _decision_badge(decision) +
                (f" (weight={_fmt_num(weight, 2)})" if weight is not None else "") +
                (f" — {reason}" if reason.strip('— ').strip() else "") + ".</li></ul>"
            )

            # Metrics (HTML)
            blocks.append("<details><summary><b>Metrics</b> (click to expand)</summary>")
            blocks.append("<table><thead><tr><th>Category</th><th>Metric</th><th>Value</th></tr></thead><tbody>")
            rows_tbl = [
                (ui["prediction_accuracy"], "R² (price)", r2),
                (ui["prediction_accuracy"], "RMSE (price)", rmse),
                (ui["prediction_accuracy"], "MAPE", f"{mape}%"),
                (ui["trend_prediction"], "Directional Accuracy", f"{diracc}%"),
                (ui["statistical_insights"], "Sharpe (annualised)", _fmt_num(stat_.get("Sharpe_ann"), 2)),
                (ui["statistical_insights"], "Max Drawdown", _fmt_num(stat_.get("Max_Drawdown"), 2)),
                (ui["risk_assessment"], "Label", risk_label),
                (ui["risk_assessment"], "SNR (latest)", snr),
                (ui["risk_assessment"], "Crosses zero", cz),
                (ui["model_explainability"], "SHAP status", shap_status),
                (ui["model_explainability"], "IG status", ig_status),
                (ui["final_recommendation"], "Final", decision),
            ]
            for cat, m, v in rows_tbl:
                blocks.append(f"<tr><td>{cat}</td><td>{m}</td><td>{v}</td></tr>")
            blocks.append("</tbody></table></details>")
        else:
            blocks.append(f"## {t}\n")
            blocks.append("**Executive Summary**\n")
            blocks.append(f"- **{ui['prediction_accuracy']}**: R² **{r2}**, RMSE **{rmse}**, MAPE **{mape}%** _(N={rows})_.")
            blocks.append(f"- **{ui['risk_assessment']}**: label **{risk_label}**, SNR **{snr}**, crosses-zero={cz}.")
            badges = _decision_badge(decision)
            wtxt = (f" (weight={_fmt_num(weight, 2)})" if weight is not None else "")
            rtxt = (f" — {reason}" if reason.strip('— ').strip() else "")
            blocks.append(f"- **{ui['final_recommendation']}**: {badges}{wtxt}{rtxt}.\n")

            # Metrics (Markdown + HTML <details> for collapsible)
            blocks.append("<details><summary><b>Metrics</b> (click to expand)</summary>\n")
            blocks.append("| Category | Metric | Value |")
            blocks.append("|---|---|---|")
            md_rows = [
                (ui["prediction_accuracy"], "R² (price)", r2),
                (ui["prediction_accuracy"], "RMSE (price)", rmse),
                (ui["prediction_accuracy"], "MAPE", f"{mape}%"),
                (ui["trend_prediction"], "Directional Accuracy", f"{diracc}%"),
                (ui["statistical_insights"], "Sharpe (annualised)", _fmt_num(stat_.get("Sharpe_ann"), 2)),
                (ui["statistical_insights"], "Max Drawdown", _fmt_num(stat_.get("Max_Drawdown"), 2)),
                (ui["risk_assessment"], "Label", risk_label),
                (ui["risk_assessment"], "SNR (latest)", snr),
                (ui["risk_assessment"], "Crosses zero", cz),
                (ui["model_explainability"], "SHAP status", ("Shown" if p_shap else "— (SHAP skipped or not available)")),
                (ui["model_explainability"], "IG status", ("Shown" if p_ig else "— (IG not available)")),
                (ui["final_recommendation"], "Final", decision),
            ]
            for cat, m, v in md_rows:
                blocks.append(f"| {cat} | {m} | {v} |")
            blocks.append("\n</details>\n")

        # Plots (auto-hide missing)
        p_line    = _get_plot(node_plots, asset_plots, ["actual_vs_pred_line"])
        p_scatter = _get_plot(node_plots, asset_plots, ["actual_vs_pred_scatter"])
        p_dd      = _get_plot(node_plots, asset_plots, ["risk_drawdown"])
        p_vol     = _get_plot(node_plots, asset_plots, ["risk_rolling_vol"])
        p_tb      = _get_plot(node_plots, asset_plots, ["risk_tomorrow_band"])
        p_fc      = _get_plot(node_plots, asset_plots, ["risk_fanchart"])
        # XAI (broaden keys; fuzzy+glob matching inside _get_plot handles variations)
        p_shap    = _get_plot(node_plots, asset_plots, ["shap_bar", "xai_shap_bar", "shap", "beeswarm", "importance"])
        p_ig      = _get_plot(node_plots, asset_plots, ["ig_heatmap", "xai_ig_heatmap", "ig", "importance", "heatmap"])

        ordered = [
            ("Actual vs Predicted Close (test)", p_line),
            ("Actual vs Predicted Close (scatter)", p_scatter),
            ("Drawdown", p_dd),
            ("Rolling Volatility (20d)", p_vol),
            ("Tomorrow Close Uncertainty", p_tb),
            ("7-Day Forecast Fan Chart", p_fc),
            ("Explainability — SHAP", p_shap),
            ("Explainability — Integrated Gradients", p_ig),
        ]
        for title, src in ordered:
            if not src:
                continue
            if make_html:
                data_uri = _img_data_uri(src)
                if data_uri:
                    blocks.append(f"<p><b>{title}</b><br><img src='{data_uri}' alt='{title}'></p>")
                elif os.path.exists(src):
                    # Fallback to relative path if base64 fails
                    rel = os.path.relpath(src, reports_dir) if os.path.exists(reports_dir) else src
                    blocks.append(f"<p><b>{title}</b><br><img src='{rel}' alt='{title}'></p>")
            else:
                blocks.append(f"**{title}**\n")
                blocks.append(f"![{title}]({src})\n")

        if make_html:
            blocks.append("<div class='hr'></div></div>")
        else:
            blocks.append("---\n")

    # ---------- Footer ----------
    if make_html:
        blocks.append("<p class='muted'><i>This report is automatically generated from model outputs and stock market data. "
                      "It is <b>not</b> investment advice.</i></p></body></html>")
        content = "\n".join(blocks)
    else:
        blocks.append("> _This report is automatically generated from model outputs and stock market data. It is **not** investment advice._\n")
        content = "\n".join(blocks)

    _ensure_dir(reports_dir)
    out_path = os.path.join(reports_dir, out_name)
    with open(out_path, "w", encoding="utf-8") as f:
        f.write(content)
    print(f"Report written → {out_path}")
    return out_path

# ---------------------------- LangGraph wrapper ----------------------------
try:
    from langgraph.graph import StateGraph
    HAS_LG = True
except Exception:
    HAS_LG = False

def node_render(state: Dict[str, Any]) -> Dict[str, Any]:
    run_base = state.get("run_base") or "."
    order    = state.get("tickers")
    out_name = state.get("out_name", "llm_report.html")  # default to HTML
    path = render_report(run_base=run_base, out_name=out_name, order=order)
    return {**state, "status": "ok", "report_path": path}

def build_llm_report_workflow():
    g = StateGraph(dict)
    g.add_node("render", node_render)
    g.set_entry_point("render")
    g.set_finish_point("render")
    return g.compile()

if __name__ == "__main__":
    run_base = os.environ.get("RUN_BASE", ".")
    tickers  = os.environ.get("TICKERS")
    order = [t.strip().upper() for t in (tickers.split(",") if tickers else [])] if tickers else None
    try:
        render_report(run_base=run_base, out_name="llm_report.html", order=order)
    except Exception as e:
        print("Report generation failed:", e)





